# 🌾 Image2Biomass Prediction

## Project Overview

This notebook implements a **6-model ensemble learning system** which is used to predict dry biomass  by combining  imagery and environmental tabular features, which is  **Cheap, affordable and better** then other large models.

### Architecture

- **Vision Models** (3): ViT-Base, ResNet-50, DenseNet-121 for image patch processing  
- **Species Classification Model**: ViT-Base (16-class classifier) for predicting pasture species from imagery
- **Tabular Model**: XGBoost trained on environmental and spectral features  
- **Meta-Learner**: Fully connected neural network that fuses base model predictions and tabular features  

```
                            System Parameter Breakdown:
                            ─────────────────────────────
                            1. ViT-Base (Biomass):      86.6M
                            2. ResNet-50 (Biomass):     25.6M
                            3. DenseNet-121 (Biomass):  8.0M
                            4. Species Classifier:      86.6M
                            5. Meta-Learner:            5.1K
                            ─────────────────────────────
                            Total Parameters:          ~207M

```

### Data Processing

- **Input images**: 2000×1000 pixel  images  
- **Patch extraction**: Splits each image into 8 patches (500×500 pixels), resized to 224×224 for model inputs  
- **Tabular features**: 29-dimensional vector combining:
  - Numeric features: NDVI, height, month, day of year, quarter (normalized)
  - Categorical features: state (4 dims), species (15 dims), target type (5 dims)
- **Targets**: Predicts 5 biomass target types — Dry_Green_g, Dry_Dead_g, Dry_Clover_g, Dry_Total_g, GDM_g  

### Species Classification Model

The **Species ViT** is a ViT-Base model trained for 15-class species classification:

- **Input**: Same 8 image patches as biomass models (224×224 each)
- **Output**: Species class prediction (15 unique species)
- **Purpose**: Predicts pasture species from satellite imagery for intelligent feature fallback
- **Usage**: During test inference with missing features, species prediction improves fallback feature generation accuracy (Level 1 , 2 & 3 fallback levels)
- **Training**: Trained on training set species labels, achieves high accuracy to ensure reliable predictions

### Key Features

✅ **6-model ensemble** (3 vision + 1 species + 1 tabular + 1 meta-learner)  
✅ **Intelligent multi-level fallback** for missing test features using species prediction and statistical inference  
✅ **Species-informed feature generation** — Uses predicted species to improve tabular feature accuracy in test data  
✅ **Patch-level image processing** to capture spatial detail and improve robustness  
✅ **GPU memory optimization and model offloading** for efficient training  
✅ **Comprehensive logging** for detailed training and evaluation tracking  
✅ **Weighted R² loss** optimized per competition evaluation metrics (0.5 for Dry_Total_g, 0.2 for GDM, 0.1 for others)  
✅ **Cross-validation on 85/15 train/val split** to monitor generalization  
✅ **Robust ensemble averaging and meta-learning** for final biomass prediction  

### Data Pipeline

```
    
                                            Test Image
                                                ↓
                            Load Image → Crop 8 Patches (500×500 → 224×224)
                                                ↓
                                ┌─────────────────────────────────┐
                                │ Species Model Prediction        │
                                │ (15 classes: pasture species)   │
                                └────────────┬────────────────────┘
                                             ↓ (species label)
                                ┌─────────────────────────────────┐
                                │ Intelligent Feature Fallback    │
                                │ Level 1: Random state + species │
                                │ Level 2: Exact sample ID match  │
                                │ Level 3: Image ID + species     │
                                │ Level 4: Statistical generation │
                                └────────────┬────────────────────┘
                                             ↓ (tabular features)
                            ┌─────────────────────────────────────────┐
                            │ Base Models                             │
                            │ - Vision: ViT, ResNet, DenseNet (X pred)│
                            │ - Tabular: XGBoost (1 pred)             │
                            │ Outputs: 1 predictions per model        │
                            └────────────┬────────────────────────────┘
                                         ↓ (X base predictions)
                            ┌─────────────────────────────────────────┐
                            │ Meta-Learner                            │
                            │ Input: X predictions + 29 features      │
                            │ Output: 1 final prediction              │
                            └────────────────┬────────────────────────┘
                                             ↓
                                Final Biomass Prediction (grams)

```

### Expected Performance

- Optimizes globally weighted R² metric reflecting competition's scoring  
- Species model improves test feature fallback accuracy by providing accurate species labels
- Ensemble diversity (vision + tabular + species) provides robustness  
- Meta-learner learns optimal combination of base model predictions  
- Comprehensive validation monitoring with early stopping prevents overfitting  

### Output Format

- **File**: CSV with `sample_id` and `target` columns
- **Rows**: 5 rows (one per target type: Dry_Green_g, Dry_Dead_g, Dry_Clover_g, GDM_g, Dry_Total_g)
- **Values**: Biomass predictions in grams (float, non-negative, rounded to 3 decimal places)
- **Evaluation**: Weighted R² computed globally across all rows combined

---

**Status**: ✅ Ready | **Models**: 6 | **Features**: 29 | **Targets**: 5

##  1: IMPORTS & CONFIGURATION

This cell initializes all required dependencies for the complete pipeline.

### Libraries

#### Data Processing & ML
- `pandas`, `numpy`: Data manipulation
- `scikit-learn`: Preprocessing, metrics, splitting
- `xgboost`: Tree-based ensemble model

#### Deep Learning
- `torch`, `torchvision`: PyTorch framework
- `timm`: Vision model library (ViT, ResNet, DenseNet)
- `torch.nn.functional`: Neural network operations

#### Visualization & Logging
- `matplotlib`, `seaborn`: Data visualization
- `PIL`: Image processing
- `logging`, `tqdm`: Progress tracking
- `cv2`: Computer vision operations

#### Utilities
- `pathlib`: File path management
- `json`, `pickle`: Serialization
- `shutil`, `gc`: System utilities
- `datetime`: Timestamps
- `socket`: Network connectivity checks

### Environment Detection

- GPU availability check
- Device assignment (CUDA/CPU)
- Random seed initialization for reproducibility


In [ ]:
import os
import gc
import re
import sys
import json
import math
import socket
import pickle
import joblib
import shutil
import random 
import zipfile
import logging
import warnings
import traceback
from pathlib import Path
warnings.filterwarnings('ignore')
os.environ['OPENCV_LOG_LEVEL'] = 'OFF'

# Image processing
import cv2
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
import torch.nn.functional as FF
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, densenet121

# Progress tracking
import time
from tqdm import tqdm
from collections import Counter
from datetime import datetime, timedelta

# Data processing
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import stats
from scipy.stats import trim_mean
from scipy.stats import randint, uniform
from sklearn.preprocessing import RobustScaler
from scipy.ndimage import laplace, gaussian_filter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for h in logging.root.handlers[:]:
    logging.root.removeHandler(h)
logger = logging.getLogger(__name__)
for h in logger.handlers[:]:
    logger.removeHandler(h)

logger.propagate = False
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler(stream=sys.stdout)
    handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

log_file_path = "/kaggle/working/pipeline.log" 
file_handler = logging.FileHandler(log_file_path, mode="w", encoding="utf-8")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(file_handler)

logger.info("=" * 80)
logger.info("PASTURE BIOMASS PREDICTION - 6 MODEL ENSEMBLE PIPELINE")
logger.info("=" * 80)
start_time = datetime.now()
logger.info(f"Execution started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
logger.info("=" * 80)

## 2: PATHS AND CONFIGURATION

**Purpose**: Define directory structure, file paths, and all hyperparameters.


**Image Configuration**:

| Parameter | Value | Purpose |
|-----------|-------|---------|
| Image Size | 2000×1000 px | Raw  image |
| Patch Size | 500×500 px | Crop dimensions |
| Grid Layout | 4 horizontal × 2 vertical | Total 8 patches per image |
| Model Input | 224×224 px | Vision model standard |
| Total Patches Per Image | 8 | Spatial diversity |

**Training Hyperparameters**:

| Setting | Value | Notes |
|---------|-------|-------|
| Batch Size | 2 images | = 16 patches per batch |
| Learning Rate | 1e-4 to 5e-4 | Model-specific |
| Epochs | 50 (ViT/ResNet/DenseNet), 100 (XGBoost/Meta) | Training iterations |
| Dropout | 0.1-0.3 | Regularization |
| Weight Decay | 0.05 | L2 regularization in AdamW |
| Early Stopping | patience=20 | Stop if no improvement |
| Gradient Clipping | 1.0 | Stability |
| Optimizer | AdamW | Weight decay support |

**Target Importance Weighting** (Weighted R²):
```
Dry_Total_g:   0.5  ← Primary metric (50%)
GDM_g:         0.2  ← Secondary (20%)
Dry_Green_g:   0.1  ← Supporting (10%)
Dry_Dead_g:    0.1  ← Supporting (10%)
Dry_Clover_g:  0.1  ← Supporting (10%)
```

**Output**: Configuration dictionary logged with all settings, device detected, paths created.

In [1]:
# Define directory structure
BASE_DIR = Path('./').resolve()
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models'
OUTPUTS_DIR = BASE_DIR / 'outputs'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
RAW_DATA_DIR = Path('input_data/')
ASSETS = Path('assets/')

# Data file paths
TRAIN_CSV = RAW_DATA_DIR / 'train.csv'
TEST_CSV = RAW_DATA_DIR / 'test.csv'
SYN_CSV = OUTPUTS_DIR / 'train_synthetic_matching.csv'
SAMPLE_SUBMISSION = RAW_DATA_DIR / 'sample_submission.csv'

# Model checkpoint paths
VIT_CHECKPOINT = MODELS_DIR / 'vit_best.pth'
RESNET_CHECKPOINT = MODELS_DIR / 'resnet_best.pth'
DENSENET_CHECKPOINT = MODELS_DIR / 'densenet_best.pth'
XGBOOST_CHECKPOINT = MODELS_DIR / 'xgboost_best.pkl'
METALEARNER_CHECKPOINT = MODELS_DIR / 'metalearner_best.pth'
SPECIES_VIT_CHECKPOINT = MODELS_DIR / 'species_best.pth'

# Output paths
SUBMISSION_PATH = BASE_DIR / 'submission.csv'
METRICS_PATH = OUTPUTS_DIR / 'metrics.json'

TOTAL_TABULAR_DIM = 30
MAX_TARGET = 200.0

# Configuration constants
CONFIG = {
  
    'image_width': 2000,          
    'image_height': 1000,         
    'patch_size': 500,            
    'patches_horizontal': 4,      
    'patches_vertical': 2,        
    'num_patches': 8,             

    'random_seed': 42,
    'numpy_seed': 42,
    'torch_seed': 42,
    'tensorflow_seed': 42,
    
    # Deep Learning hyperparameters
    'model_input_size': 224,     
    'batch_size': 8,              
    'num_workers': 0,
    'pin_memory': False,
    
    # Training parameters
    'vit_epochs': 15,  #15-15 Total 30   
    'resnet_epochs': 15, #15-15 Total 30 
    'densenet_epochs': 15, #15-15 Total 30
    'metalearner_epochs': 8, #8
    'metalearner_repeat' : 10, #10

    #  Species Model Configuration
    'species_vit_epochs': 15,  #15-15 Total 30
    'species_vit_lr': 2e-4,
    'num_species_classes': 16,  
    'meta_weight_decay': 0.01,    
    'meta_dropout_rate': 0.15,  
    
    "use_mil": True,   # Toggle MIL 
    "mil_dropout": 0.25,         
    
    'vit_lr': 2e-4,
    'resnet_lr': 2e-4,
    'densenet_lr': 2e-4,
    'metalearner_lr': 1e-3,
    
    'weight_decay': 0.03,
    'gradient_clip': 1.0,
    'early_stopping_patience': 5,

    # XGBoost parameters
    'xgb_n_estimators': 1000,            
    'xgb_max_depth': 4,                   
    'xgb_learning_rate': 0.05,             
    'xgb_subsample': 0.75,                 
    'xgb_colsample_bytree': 0.75,          
    'xgb_min_child_weight': 5,             
    'xgb_reg_alpha': 0.5,                  
    'xgb_reg_lambda': 1.5,                 
    'xgb_gamma': 1.0,                      
    'xgb_early_stopping_rounds': 50,       
     

    # Data split
    'val_split': 0.15,
    'test_size_for_split': 0.15,
    
    # Weighted R² weights
    'r2_weights': {
        'Dry_Green_g': 0.1,
        'Dry_Dead_g': 0.1,
        'Dry_Clover_g': 0.1,
        'GDM_g': 0.2,
        'Dry_Total_g': 0.5,
    }
}

# Create directories if they don't exist
for directory in [DATA_DIR, PROCESSED_DATA_DIR, MODELS_DIR, OUTPUTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    logger.info(f"✓ Directory ready: {directory}")
    
# Device detection
if torch.cuda.is_available():
    device = torch.device('cuda')
    logger.info(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
    logger.info(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device('cpu')
    logger.info("⚠ GPU not available, using CPU")

CONFIG['xgb_noise_scale'] = 0.05      
CONFIG['xgb_augment_rounds'] = 2
CONFIG['dry_total_tolerance'] = 0.1
CONFIG['device'] = device

# Set random seeds for reproducibility
np.random.seed(CONFIG['numpy_seed'])
torch.manual_seed(CONFIG['torch_seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(CONFIG['torch_seed'])

logger.info("✓ Paths and configuration")

NameError: name 'Path' is not defined

## 3: EXPLORATORY DATA ANALYSIS

**Purpose**: Load, explore, analyze data, and build intelligent lookup tables for test feature fetching.

**Dataset Statistics**:

| Aspect | Training | Test |
|--------|----------|------|
| Rows | 1,785 | 5 |
| Features | 30 total | 3 (ID, path, target) |
| Targets | 5 types | TBD |
| Per Target | 357 samples | 1 sample |

**Numeric Features (5)**:
- `Pre_GSHH_NDVI`: Vegetation greenness [0.0, 1.0]
- `Height_Ave_cm`: Pasture height [0, 100] cm
- `Month`: Sampling month [1, 12]
- `DayOfYear`: Day since Jan 1 [1, 365]
- `Quarter`: Season [1, 4]

**Categorical Features** (One-Hot Encoded):
- `State` (4): NSW, WA, Tasmania, Victoria
- `Species` (16): Ryegrass, Clover, Phalaris, Lucerne, etc.
- `Target Type` (5): Dry_Green_g, Dry_Dead_g, Dry_Clover_g, GDM_g, Dry_Total_g

**Total Feature Dimension**: 5 + 4 + 16 + 5 = **30 features**

**Target Variables** (5 Biomass Types):

| Target | Description | Weight | Typical Range |
|--------|-------------|--------|---------------|
| Dry_Green_g | Green leaf dry matter | 0.1 | 0-500g |
| Dry_Dead_g | Dead plant material | 0.1 | 0-300g |
| Dry_Clover_g | Clover dry matter | 0.1 | 0-200g |
| GDM_g | Green dry matter | 0.2 | 100-1000g |
| Dry_Total_g | Total dry matter | 0.5 | 100-1500g |

**Test Data Challenge**:
- Test CSV contains only 5 rows with minimal features
- Missing: Sampling_Date, State, Species, Pre_GSHH_NDVI, Height_Ave_cm
- Solution: Intelligent 4-level fallback feature fetching


**Output**: 
- Data loaded and explored
- Lookup tables built for all 4 levels
- EDA visualizations saved
- Summary statistics computed

In [ ]:
FIXED_SPECIES_LIST = [
    'Ryegrass_Clover', 'Lucerne', 'SubcloverDalkeith', 'Ryegrass',
    'Phalaris_Clover', 'SubcloverLosa', 'Clover', 'Fescue_CrumbWeed',
    'Phalaris_Ryegrass_Clover', 'Phalaris', 'WhiteClover', 'Fescue',
    'Phalaris_BarleyGrass_SilverGrass_SpearGrass_Clover_Capeweed',
    'Phalaris_Clover_Ryegrass_Barleygrass_Bromegrass', 'Mixed', 'Nothing']

def _normalize_for_vocab(s):
    if s is None:
        return ''
    s = str(s).strip()
    if s == '' or s.lower() in ('nan', 'none', 'null'):
        return ''
    return s

def map_to_fixed_species(raw):
    """
    Map arbitrary raw species text to one of FIXED_SPECIES_LIST (returning the exact list item),
    or 'Mixed' if no reasonable mapping. Blanks/NA -> 'Nothing'.
    """
    r = _normalize_for_vocab(raw)
    if r == '':
        return 'Nothing'

    # normalized comparable form
    rl = re.sub(r'[\s\-_\.]+', ' ', r.strip().lower())

    for fs in FIXED_SPECIES_LIST:
        fs_norm = re.sub(r'[\s\-_\.]+', ' ', fs.strip().lower())
        if rl == fs_norm:
            return fs
    rl_tokens = set(rl.split())
    best = None
    best_score = 0
    for fs in FIXED_SPECIES_LIST:
        fs_norm = re.sub(r'[\s\-_\.]+', ' ', fs.strip().lower())
        fs_tokens = set(fs_norm.split())
        overlap = len(rl_tokens & fs_tokens)
        contain = 1 if (rl in fs_norm or fs_norm in rl) else 0
        score = overlap * 10 + contain
        if score > best_score:
            best_score = score
            best = fs
    if best_score > 0:
        return best
    return 'Mixed'

def create_species_lookup_tables(train_df):
    """
    Create species-based lookup tables for Level 2 fallbacks.

    Returns a tuple (rows_map, stats_map) where:
      - rows_map:  { target_name_norm: { (image_id_norm, species_norm): [row_dict, ...] } }
      - stats_map: { target_name_norm: { (image_id_norm, species_norm): {count, mean, std, median} } }
    """
    logger.info("\n" + "="*80)
    logger.info("CREATING SPECIES-BASED LOOKUP TABLES")
    logger.info("="*80)

    if train_df is None or train_df.shape[0] == 0:
        logger.warning("create_species_lookup_tables: empty or None train_df provided; returning empty lookups")
        return {}, {}

    required_cols = ['sample_id', 'Species', 'target_name', 'target']
    for c in required_cols:
        if c not in train_df.columns:
            logger.warning(f"create_species_lookup_tables: missing column {c}; filling with defaults")
            if c == 'target':
                train_df[c] = pd.to_numeric(train_df.get(c, np.nan), errors='coerce')
            else:
                train_df[c] = train_df.get(c, '').astype(str)

    def _norm_str(x):
        s = str(x).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = s.replace('.', '')
        return s

    def _norm_image_id(sample_id):
        try:
            return str(sample_id).split('__')[0].strip().lower()
        except Exception:
            return str(sample_id).strip().lower()
            
    tmp = train_df.copy()
    tmp['_image_id'] = tmp['sample_id'].apply(_norm_image_id).fillna('').astype(str)
    if 'Species_canon' in tmp.columns:
        tmp['_species_n'] = tmp['Species_canon'].astype(str).fillna('').apply(lambda x: re.sub(r"[\s\-]+", "_", str(x).strip().lower()).replace('.', ''))
    else:
        tmp['_species_n'] = tmp['Species'].astype(str).fillna('').apply(_norm_str)
    
    tmp['_target_n'] = tmp['target_name'].astype(str).fillna('').apply(_norm_str)

    TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS = {}
    for idx, row in tmp.iterrows():
        t = row['_target_n']
        key = (row['_image_id'], row['_species_n'])
        TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS.setdefault(t, {}).setdefault(key, []).append(row.to_dict())

    TRAIN_BY_IMAGE_SPECIES_TARGET_STATS = {}
    grouped = tmp.groupby(['_target_n', '_image_id', '_species_n'])['target']
    for (t, img, sp), ser in grouped:
        TRAIN_BY_IMAGE_SPECIES_TARGET_STATS.setdefault(t, {})[(img, sp)] = {
            'count': int(ser.count()),
            'mean': float(ser.mean()) if ser.count() > 0 else 0.0,
            'std': float(ser.std()) if ser.count() > 1 else 0.0,
            'median': float(ser.median()) if ser.count() > 0 else 0.0}
        
    total_keys = sum(len(d) for d in TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS.values())
    logger.debug(f"✓ Level 2 species lookup (rows) built: targets={len(TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS)} total_keys={total_keys}")
    for t, combos in TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS.items():
        logger.info(f"  {t}: {len(combos)} (image,species) combos")

    return TRAIN_BY_IMAGE_SPECIES_TARGET_ROWS, TRAIN_BY_IMAGE_SPECIES_TARGET_STATS


def load_and_explore_data(TRAINDATA=TRAIN_CSV):
    """
    Load and perform comprehensive EDA on training and test datasets.
    Creates intelligent lookup tables for test data feature fetching.

    Returns:
        tuple: (train_df, test_df, STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS,
                TOTAL_TABULAR_DIM, lookups_dict, original_test_df)
    """
    logger.info("="*80)
    logger.info("LOADING DATA WITH INTELLIGENT TEST FEATURE FETCHING")
    logger.info("="*80)
    lookups_dict = {}
    
    # LOAD DATA
    try:
        train_df = pd.read_csv(TRAINDATA)
        test_df = pd.read_csv(TEST_CSV)
        original_test_df = test_df.copy()
        logger.debug(f"✓ Training data loaded: {len(train_df)} rows, {len(train_df.columns)} columns")
        logger.info(f"✓ Test data loaded: {len(test_df)} rows, {len(test_df.columns)} columns")
    except FileNotFoundError as e:
        logger.error(f"✗ Error loading data: {e}")
        sys.exit(1)
        
    try:
        HEIGHT_MAX = float(train_df['Height_Ave_cm'].max())
        if not np.isfinite(HEIGHT_MAX) or HEIGHT_MAX <= 0:
            HEIGHT_MAX = 100
    except Exception:
        HEIGHT_MAX = 100
    
    # safety margin 
    HEIGHT_MAX = float(min(max(HEIGHT_MAX, 1.0), 150.0))
    
    logger.debug(f"✓ Using dynamic HEIGHT_MAX = {HEIGHT_MAX:.2f} cm")
    lookups_dict["HEIGHT_MAX"] = HEIGHT_MAX

    # Required train columns
    expected_train_cols = ['sample_id','target_name','target','Pre_GSHH_NDVI','Height_Ave_cm','Sampling_Date','State','Species']
    for c in expected_train_cols:
        if c not in train_df.columns:
            logger.warning(f"Train CSV missing column: {c} — filling with default/NA")
            if c in ['Pre_GSHH_NDVI','Height_Ave_cm','target']:
                train_df[c] = pd.to_numeric(train_df.get(c, np.nan), errors='coerce')
            else:
                train_df[c] = train_df.get(c, '').astype(str)

    # Expected test columns - fill if missing 
    expected_test_cols = ['sample_id','image_path','target_name','Sampling_Date','State','Species','Pre_GSHH_NDVI','Height_Ave_cm']
    for c in expected_test_cols:
        if c not in test_df.columns:
            logger.warning(f"Test CSV missing column: {c}. Will rely on fallback stats.")
            test_df[c] = None

    # SHOW CSV STRUCTURE
    logger.debug("\n" + "="*80)
    logger.debug("CSV STRUCTURE ANALYSIS")
    logger.debug("="*80)
    logger.debug("\n[TRAIN CSV]")
    logger.debug(f"Columns: {list(train_df.columns)}")
    logger.debug("\n[TEST CSV]")
    logger.debug(f"Columns: {list(test_df.columns)}")

    # EXTRACT TEMPORAL FEATURES (TRAIN ONLY) 
    logger.debug("\n" + "="*80)
    logger.debug("EXTRACT TEMPORAL FEATURES (TRAIN ONLY)")
    logger.debug("="*80)

    train_df['Sampling_Date'] = pd.to_datetime(train_df['Sampling_Date'], errors='coerce')
    # create Month/DayOfYear/Quarter only if datetime parsed
    train_df['Month'] = train_df['Sampling_Date'].dt.month.fillna(1).astype(int)
    train_df['DayOfYear'] = train_df['Sampling_Date'].dt.dayofyear.fillna(1).astype(int)
    train_df['Quarter'] = train_df['Sampling_Date'].dt.quarter.fillna(1).astype(int)

    logger.debug("✓ Temporal features extracted from TRAIN ")

    # Ensure State and Species and target_name exist and normalize (safe)
    train_df['State'] = train_df['State'].astype(str).fillna('').str.strip()
    train_df['Species'] = train_df['Species'].astype(str).fillna('').str.strip()
    train_df['target_name'] = train_df['target_name'].astype(str).fillna('').str.strip()

    def _state_canon_or_unknown(x):
        s = str(x).strip()
        if s == "":
            return "unknown"
        return _normalize_state_for_onehot(x)
    
    train_df['State_canon'] = train_df['State'].apply(_state_canon_or_unknown)
    state_cols_train = pd.get_dummies(train_df['State_canon'], prefix='State')
    train_df = pd.concat([train_df, state_cols_train], axis=1)
    logger.debug(f"✓ State encoded: {len(state_cols_train.columns)} categories")
    logger.debug(f" Unique states in train: {train_df['State_canon'].nunique()}")
    
    train_df['Species_mapped'] = train_df['Species'].apply(map_to_fixed_species)
    train_df['Species_canon'] = pd.Categorical(train_df['Species_mapped'], categories=FIXED_SPECIES_LIST)
    species_cols_train = pd.get_dummies(train_df['Species_canon'], prefix='Species').reindex(
        columns=[f"Species_{s}" for s in FIXED_SPECIES_LIST], fill_value=0)
    train_df = pd.concat([train_df, species_cols_train], axis=1)

    # ONE-HOT ENCODE TARGET TYPE (TRAIN ONLY)
    target_onehot = pd.get_dummies(train_df['target_name'], prefix='target_type')
    train_df = pd.concat([train_df, target_onehot], axis=1)
    logger.debug(f"✓ Target types encoded: {len(target_onehot.columns)} categories")


    STATE_COLS = [
        col for col in train_df.columns
        if col.startswith('State_') and col not in ('State_canon', 'State_') and re.match(r'^State_[A-Za-z0-9]', col)]

    if len(STATE_COLS) < 4:
        for i in range(len(STATE_COLS), 4):
            col_name = f"State_extra_{i}"
            train_df[col_name] = 0
            STATE_COLS.append(col_name)
    
    SPECIES_COLS = [
        col for col in train_df.columns
        if col.startswith('Species_') and col not in ('Species_mapped','Species_canon', 'Species_') and re.match(r'^Species_[A-Za-z0-9]', col)]
    
    TARGET_TYPE_COLS = [
        col for col in train_df.columns
        if col.startswith('target_type_') and re.match(r'^target_type_[A-Za-z0-9]', col)]

    numeric_cols = []
    if 'Pre_GSHH_NDVI' in train_df.columns:
        numeric_cols.append('Pre_GSHH_NDVI')
    if 'Height_Ave_cm' in train_df.columns:
        numeric_cols.append('Height_Ave_cm')
    for col in ['Month', 'DayOfYear', 'Quarter']:
        if col in train_df.columns:
            numeric_cols.append(col)
    
    TOTAL_TABULAR_DIM = len(numeric_cols) + len(STATE_COLS) + len(SPECIES_COLS) + len(TARGET_TYPE_COLS)

    #sanity check for tabular dimension 
    calc = len(numeric_cols) + len(STATE_COLS) + len(SPECIES_COLS) + len(TARGET_TYPE_COLS)
    if calc != TOTAL_TABULAR_DIM:
        logger.warning(
            f"TOTAL_TABULAR_DIM mismatch: computed={calc}, declared={TOTAL_TABULAR_DIM}")
    else:
        logger.info(f"TOTAL_TABULAR_DIM consistent {TOTAL_TABULAR_DIM}")

    scaler = None
    if len(numeric_cols) > 0:
        df_num = train_df[numeric_cols].copy()
        if 'Height_Ave_cm' in df_num.columns:
            df_num['Height_Ave_cm'] = df_num['Height_Ave_cm'].astype(float).fillna(10.0)
            df_num['Height_Ave_cm'] = np.clip(df_num['Height_Ave_cm'], 0.0, 100.0)
            
        if 'Month' in df_num.columns:
            df_num['Month'] = df_num['Month'].astype(float).fillna(6)
            df_num['Month'] = np.clip(df_num['Month'], 1.0, 12.0)
            
        if 'DayOfYear' in df_num.columns:
            df_num['DayOfYear'] = df_num['DayOfYear'].astype(float).fillna(1)
            df_num['DayOfYear'] = np.clip(df_num['DayOfYear'], 1.0, 365.0)
            
        if 'Quarter' in df_num.columns:
            df_num['Quarter'] = df_num['Quarter'].astype(float).fillna(1)
            df_num['Quarter'] = np.clip(df_num['Quarter'], 1.0, 4.0)
            
        if 'Pre_GSHH_NDVI' in df_num.columns:
            df_num['Pre_GSHH_NDVI'] = df_num['Pre_GSHH_NDVI'].astype(float).fillna(0.5)
            df_num['Pre_GSHH_NDVI'] = np.clip(df_num['Pre_GSHH_NDVI'], 0.0, 1.0)
    
        scaler = RobustScaler()
        scaler.fit(df_num.values) 
        
        try:
            joblib.dump(scaler, OUTPUTS_DIR / "tabular_scaler.joblib")
            logger.debug("✓ Saved tabular scaler to outputs/tabular_scaler.joblib")
        except Exception:
            logger.debug("Could not persist tabular scaler to disk (continuing).")
            
        # store scaler in lookups dict 
        lookups_dict['TAB_SCALER'] = scaler
        lookups_dict['TAB_NUMERIC_COLS'] = numeric_cols
        lookups_dict['STATE_COLS'] = STATE_COLS
        lookups_dict['UNIQUE_SPECIES'] = [s.lower().replace('.', '').replace(' ', '_') for s in FIXED_SPECIES_LIST]
        SPECIES_COLS = list(species_cols_train.columns)
        lookups_dict['SPECIES_COLS'] = SPECIES_COLS
        lookups_dict['TARGET_TYPE_COLS'] = TARGET_TYPE_COLS
        lookups_dict['TOTAL_TABULAR_DIM'] = TOTAL_TABULAR_DIM  
        
    else:
        lookups_dict['TAB_SCALER'] = None

    logger.info("\n" + "="*80)
    logger.info("TABULAR FEATURE DIMENSIONS")
    logger.info("="*80)
    logger.info(f"✓ Feature dimensions:")
    logger.info(f" Numeric base count: {len(numeric_cols)}")
    logger.info(f" State one-hot: {len(STATE_COLS)}")
    logger.info(f" Species one-hot: {len(SPECIES_COLS)}")
    logger.info(f" Target Type one-hot: {len(TARGET_TYPE_COLS)}")
    logger.info(f" TOTAL_TABULAR_DIM: {TOTAL_TABULAR_DIM}")

    # CREATE TARGET-SPECIFIC LOOKUP INDEXES FOR ALL LEVELS
    logger.info("\n" + "="*80)
    logger.info("BUILDING TARGET-SPECIFIC LOOKUP INDEXES")
    logger.info("="*80)

    # LEVEL 1: By STATE + SPECIES + TARGET 
    TRAIN_BY_TARGET_STATE_SPECIES = {}
    
    def _norm_key(x):
        if pd.isna(x):
            return 'unknown'
        s = str(x).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)  
        s = s.replace('.', '')
        return s
    
    for target in train_df['target_name'].astype(str).unique():
        target_name = str(target).strip().lower()
        TRAIN_BY_TARGET_STATE_SPECIES[target_name] = {}
        target_df_filtered = train_df[train_df['target_name'].astype(str).str.strip().str.lower() == target_name]
    
        for idx, row in target_df_filtered.iterrows():
            raw_state = row.get('State', '')
            state_norm_onehot = _normalize_state_for_onehot(raw_state)   
            state_lookup_key = _norm_key(raw_state)
            species_lookup = _norm_key(row.get('Species', 'mixed'))
            rd = row.to_dict()
            rd['_state_onehot_repr'] = state_norm_onehot
            rd['_state_lookup'] = state_lookup_key
        
            TRAIN_BY_TARGET_STATE_SPECIES[target_name] \
                .setdefault(state_lookup_key, {}) \
                .setdefault(species_lookup, []) \
                .append(rd)

    
    # Diagnostics: log counts and examples
    logger.debug(f"\n✓ LEVEL 1 INDEX: By (Target, State, Species) — normalized keys")
    total_row_count = 0
    for target_name, state_dict in TRAIN_BY_TARGET_STATE_SPECIES.items():
        unique_combos = sum(len(species_dict) for species_dict in state_dict.values())
        rows_for_target = sum(sum(len(species_dict[k]) for k in species_dict) for species_dict in state_dict.values())
        total_row_count += rows_for_target
        logger.info(f" {target_name}: {unique_combos} unique (State,Species) combos — {rows_for_target} rows indexed")
    
    logger.debug(f"Total rows indexed in LEVEL 1: {total_row_count} (train_df rows: {len(train_df)})")

    # LEVEL 2: By SAMPLE_ID + TARGET
    TRAIN_BY_SAMPLE_ID_TARGET = {}
    for idx, row in train_df.iterrows():
        target_name = str(row.get('target_name', '')).strip().lower()
        sample_id_key = str(row.get('sample_id', '')).strip().lower()
        TRAIN_BY_SAMPLE_ID_TARGET.setdefault(target_name, {}).setdefault(sample_id_key, []).append(row.to_dict())

    logger.debug(f"\n✓ LEVEL 2 INDEX: By (Target, sample_id)")
    logger.info(f" Size: {sum(len(samples) for samples in TRAIN_BY_SAMPLE_ID_TARGET.values())} total entries")

    # LEVEL 3: By IMAGE_ID + TARGET 
    TRAIN_BY_IMAGE_ID_TARGET = {}
    for idx, row in train_df.iterrows():
        target_name = str(row.get('target_name', '')).strip().lower()
        image_id = str(row.get('sample_id', '')).strip().lower().split('__')[0]
        TRAIN_BY_IMAGE_ID_TARGET.setdefault(target_name, {}).setdefault(image_id, []).append(row.to_dict())
    
    logger.debug(f"\n✓ LEVEL 3: By (Target, image_id)")
    logger.debug(f" Size: {sum(len(images) for images in TRAIN_BY_IMAGE_ID_TARGET.values())} total entries")

    # LEVEL 4: STATISTICS BY TARGET
    logger.info(f"\n✓ LEVEL 4 INDEX: By Target (Statistics)")
    TABULAR_STATS = {
        'height_mean': float(train_df['Height_Ave_cm'].mean()) if 'Height_Ave_cm' in train_df.columns else 0.0,
        'height_std': float(train_df['Height_Ave_cm'].std()) if 'Height_Ave_cm' in train_df.columns else 0.0,
        'ndvi_mean': float(train_df['Pre_GSHH_NDVI'].mean()) if 'Pre_GSHH_NDVI' in train_df.columns else 0.0,
        'ndvi_std': float(train_df['Pre_GSHH_NDVI'].std()) if 'Pre_GSHH_NDVI' in train_df.columns else 0.0,
        'height_mean_cm': float(train_df['Height_Ave_cm'].mean()) if 'Height_Ave_cm' in train_df.columns else 0.0,
        'height_std_cm': float(train_df['Height_Ave_cm'].std()) if 'Height_Ave_cm' in train_df.columns else 0.0,
        # normalized versions 
        'height_mean_norm': (float(train_df['Height_Ave_cm'].mean())) if 'Height_Ave_cm' in train_df.columns else 0.0,
        'height_std_norm': (float(train_df['Height_Ave_cm'].std())) if 'Height_Ave_cm' in train_df.columns else 0.0,
        'sampling_date_mean': float(train_df['DayOfYear'].mean()) if 'DayOfYear' in train_df.columns else 0.0,
    }

    TABULAR_STATS_BY_TARGET = {}
    for target_name in ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']:
        tkey = str(target_name).strip().lower()
        target_df = train_df[train_df['target_name'].astype(str).str.strip().str.lower() == tkey]

        TABULAR_STATS_BY_TARGET[tkey] = {
            'ndvi_mean': float(target_df['Pre_GSHH_NDVI'].mean()) if 'Pre_GSHH_NDVI' in target_df.columns else 0.0,
            'ndvi_std': float(target_df['Pre_GSHH_NDVI'].std()) if 'Pre_GSHH_NDVI' in target_df.columns else 0.0,
        
            # original cm values
            'height_mean_cm': float(target_df['Height_Ave_cm'].mean()) if 'Height_Ave_cm' in target_df.columns else 0.0,
            'height_std_cm': float(target_df['Height_Ave_cm'].std()) if 'Height_Ave_cm' in target_df.columns else 0.0,
        
            # canonical names used elsewhere
            'height_mean': float(target_df['Height_Ave_cm'].mean()) if 'Height_Ave_cm' in target_df.columns else 0.0,
            'height_std': float(target_df['Height_Ave_cm'].std()) if 'Height_Ave_cm' in target_df.columns else 0.0,
        
            # normalized 
            'height_mean_norm': (float(target_df['Height_Ave_cm'].mean())) if 'Height_Ave_cm' in target_df.columns else 0.0,
            'height_std_norm': (float(target_df['Height_Ave_cm'].std())) if 'Height_Ave_cm' in target_df.columns else 0.0,
            'sampling_date_mean': float(target_df['DayOfYear'].mean()) if 'DayOfYear' in target_df.columns else 0.0,
            'count': int(len(target_df)),
        }

    for target_name, stats in TABULAR_STATS_BY_TARGET.items():
        logger.info(f" {target_name}:")
        logger.info(f" NDVI: {stats['ndvi_mean']:.3f} ± {stats['ndvi_std']:.3f}")
        logger.info(f" Height: {stats['height_mean']:.1f} ± {stats['height_std']:.3f}")
        logger.info(f" Count: {stats['count']} samples")

    try:
        raw_states = train_df.get('State_canon')
        if raw_states is None:
            UNIQUE_STATES = []
        else:
            # If categorical, use categories 
            if pd.api.types.is_categorical_dtype(raw_states):
                states_list = list(raw_states.cat.categories)
            else:
                states_list = raw_states.astype(str).str.strip().replace({'nan': ''}).loc[lambda x: x != ''].unique().tolist()
            UNIQUE_STATES = [str(s).strip() for s in states_list if str(s).strip() != '']
        UNIQUE_STATES = sorted(list(dict.fromkeys(UNIQUE_STATES))) 
    except Exception:
        UNIQUE_STATES = []
    
    try:
        if 'FIXED_SPECIES_LIST' in globals():
            UNIQUE_SPECIES = [
                re.sub(r"[\s\-]+", "_", s.strip()).replace('.', '')
                for s in FIXED_SPECIES_LIST]
        else:
            sp = train_df.get('Species_canon')
            if sp is None:
                UNIQUE_SPECIES = []
            else:
                if pd.api.types.is_categorical_dtype(sp):
                    raw_list = list(sp.cat.categories)
                else:
                    raw_list = sp.astype(str).str.strip().replace({'nan': ''}).loc[lambda x: x != ''].unique().tolist()
                UNIQUE_SPECIES = [re.sub(r"[\s\-]+", "_", str(s).strip()).replace('.', '') for s in raw_list if str(s).strip() != '']
        UNIQUE_SPECIES = list(dict.fromkeys(UNIQUE_SPECIES))
    except Exception:
        UNIQUE_SPECIES = []
    
    try:
        if 'TARGET_TYPE_COLS' in locals() or 'TARGET_TYPE_COLS' in globals():
            cols = TARGET_TYPE_COLS if 'TARGET_TYPE_COLS' in locals() else globals().get('TARGET_TYPE_COLS', [])
            UNIQUE_TARGET_TYPES = [c.replace('target_type_', '') for c in cols]
        else:
            UNIQUE_TARGET_TYPES = train_df['target_name'].astype(str).str.strip().replace({'nan': ''}).loc[lambda x: x != ''].unique().tolist()
        UNIQUE_TARGET_TYPES = [str(t).strip() for t in UNIQUE_TARGET_TYPES if str(t).strip() != '']
    except Exception:
        UNIQUE_TARGET_TYPES = []

    logger.info(f"\n✓ Unique values (for fallback logic):")
    logger.info(f" States: {UNIQUE_STATES}")
    logger.info(f" Species: {UNIQUE_SPECIES}")
    logger.info(f" Target Types: {UNIQUE_TARGET_TYPES}")

    rows_map, stats_map = create_species_lookup_tables(train_df)
    
    # CREATE LOOKUPS DICTIONARY 
    lookups_dict_target = {
        # Target-specific indexes
        'TRAIN_BY_SAMPLE_ID_TARGET': TRAIN_BY_SAMPLE_ID_TARGET,
        'TRAIN_BY_IMAGE_ID_TARGET': TRAIN_BY_IMAGE_ID_TARGET,
        'TRAIN_BY_TARGET_STATE_SPECIES': TRAIN_BY_TARGET_STATE_SPECIES,
        'TABULAR_STATS_BY_TARGET': TABULAR_STATS_BY_TARGET,
    
        # Global stats
        'TABULAR_STATS': TABULAR_STATS,
    
        # Utility data
        'UNIQUE_STATES': UNIQUE_STATES,
        'UNIQUE_SPECIES': UNIQUE_SPECIES,
        'UNIQUE_TARGET_TYPES': UNIQUE_TARGET_TYPES,
        'STATE_COLS': STATE_COLS,
        'SPECIES_COLS': SPECIES_COLS,
        'TARGET_TYPE_COLS': TARGET_TYPE_COLS,
        'TRAIN_DF': train_df,
    
        # Species lookup 
        'TRAIN_BY_IMAGE_SPECIES_TARGET': rows_map,
        'TRAIN_BY_IMAGE_SPECIES_TARGET_STATS': stats_map,
    
        # numeric / scaler for dataset pipeline 
        'TAB_NUMERIC_COLS': numeric_cols,
        'TAB_SCALER': lookups_dict.get('TAB_SCALER', None) if isinstance(lookups_dict, dict) else None,
        'TOTAL_TABULAR_DIM': TOTAL_TABULAR_DIM,
    }

    if isinstance(lookups_dict, dict):
        lookups_dict.update(lookups_dict_target)
    else:
        lookups_dict = lookups_dict_target
        
    # DISPLAY DATASET OVERVIEW
    logger.info("\n" + "="*80)
    logger.info("DATASET OVERVIEW")
    logger.info("="*80)
    logger.info("\n[TRAIN SET]")
    logger.info(f"Shape: {train_df.shape}")
    logger.info(f"Columns: {len(train_df.columns)} total")
    logger.info(f"Missing values: {train_df.isnull().sum().sum()} total")
   
    logger.info("\n[TEST SET]")
    logger.info(f"Shape: {test_df.shape}")
    logger.info(f"Columns: {len(test_df.columns)} total")
    logger.info(f" Columns present: sample_id, image_path, target_name")
    logger.info(f" Columns missing: Sampling_Date, State, Species, Pre_GSHH_NDVI, Height_Ave_cm")
    logger.info(f"Missing values: {test_df.isnull().sum().sum()} total")
   
    # STATISTICAL SUMMARIES
    logger.debug("\n" + "="*80)
    logger.debug("STATISTICAL SUMMARY (TRAINING SET)")
    logger.debug("="*80)
   
    logger.debug("\n[Target Variable Statistics]")
    logger.debug(f" Count: {train_df['target'].count()}")
    logger.debug(f" Mean: {train_df['target'].mean():.4f}")
    logger.debug(f" Std: {train_df['target'].std():.4f}")
    logger.debug(f" Min: {train_df['target'].min():.4f}")
    logger.debug(f" Max: {train_df['target'].max():.4f}")
   
    logger.info("\n[NDVI Statistics]")
    if 'Pre_GSHH_NDVI' in train_df.columns:
        logger.info(f" Range: [{train_df['Pre_GSHH_NDVI'].min():.3f}, {train_df['Pre_GSHH_NDVI'].max():.3f}]")
        logger.info(f" Mean: {train_df['Pre_GSHH_NDVI'].mean():.3f}")
        logger.info(f" Std: {train_df['Pre_GSHH_NDVI'].std():.3f}")
    else:
        logger.info(" Pre_GSHH_NDVI: not available")
   
    logger.info("\n[Height Statistics]")
    if 'Height_Ave_cm' in train_df.columns:
        logger.info(f" Range: [{train_df['Height_Ave_cm'].min():.2f}, {train_df['Height_Ave_cm'].max():.2f}] cm")
        logger.info(f" Mean: {train_df['Height_Ave_cm'].mean():.2f} cm")
        logger.info(f" Std: {train_df['Height_Ave_cm'].std():.2f} cm")
    else:
        logger.info(" Height_Ave_cm: not available")
   
    # TARGET DISTRIBUTION
    logger.info("\n" + "="*80)
    logger.info("TARGET DISTRIBUTION")
    logger.info("="*80)
    target_counts = train_df['target_name'].value_counts()
    for target, count in target_counts.items():
        logger.info(f" {target}: {count} samples")
   
    # STATE DISTRIBUTION
    logger.info("\n" + "="*80)
    logger.info("STATE DISTRIBUTION")
    logger.info("="*80)
    state_counts = train_df['State'].value_counts()
    for state, count in state_counts.items():
        logger.info(f" {state}: {count} samples")
   
    # VISUALIZATIONS
    logger.info("\n" + "="*80)
    logger.info("CREATING VISUALIZATIONS")
    logger.info("="*80)
   
    try:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
       
        # Target distribution by type
        target_means = train_df.groupby('target_name')['target'].mean()
        target_means.plot(kind='bar', ax=axes[0, 0])
        axes[0, 0].set_title('Average Target Value by Type', fontsize=12, fontweight='bold')
        axes[0, 0].set_ylabel('Target Value (g)')
        axes[0, 0].tick_params(axis='x', rotation=45)
       
        # NDVI distribution
        if 'Pre_GSHH_NDVI' in train_df.columns:
            axes[0, 1].hist(train_df['Pre_GSHH_NDVI'].dropna(), bins=30)
            axes[0, 1].set_title('NDVI Distribution', fontsize=12, fontweight='bold')
            axes[0, 1].set_xlabel('NDVI Value')
            axes[0, 1].set_ylabel('Frequency')
       
        # Height distribution
        if 'Height_Ave_cm' in train_df.columns:
            axes[1, 0].hist(train_df['Height_Ave_cm'].dropna(), bins=30)
            axes[1, 0].set_title('Height Distribution', fontsize=12, fontweight='bold')
            axes[1, 0].set_xlabel('Height (cm)')
            axes[1, 0].set_ylabel('Frequency')
       
        # Target value distribution by type (box plot)
        train_df.boxplot(column='target', by='target_name', ax=axes[1, 1])
        axes[1, 1].set_title('Target Distribution by Type', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Target Value (g)')
       
        plt.tight_layout()
        try:
            plt.savefig(OUTPUTS_DIR / 'eda_visualizations.png', dpi=100, bbox_inches='tight')
            logger.info(f"✓ EDA visualizations saved to {OUTPUTS_DIR / 'eda_visualizations.png'}")
        except Exception:
            # Fallback: save locally if OUTPUTS_DIR not available
            plt.savefig('eda_visualizations.png', dpi=100, bbox_inches='tight')
            logger.info("✓ EDA visualizations saved to eda_visualizations.png")
        plt.close()
    except Exception as e:
        logger.warning(f"⚠ Could not save visualizations: {e}")
   
    # CORRELATION ANALYSIS
    logger.info("\n" + "="*80)
    logger.info("CORRELATION ANALYSIS")
    logger.info("="*80)
    numeric_cols = [c for c in ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'target'] if c in train_df.columns]
    if len(numeric_cols) >= 2:
        corr_matrix = train_df[numeric_cols].corr()
        logger.info("\nCorrelation Matrix:")
        logger.info(corr_matrix.to_string())
    else:
        logger.info("Not enough numeric columns for correlation analysis")
   
    # DATA QUALITY CHECK
    logger.info("\n" + "="*80)
    logger.info("DATA QUALITY CHECK")
    logger.info("="*80)
   
    duplicates = train_df.duplicated(subset=['image_path','target_name']).sum() if 'image_path' in train_df.columns else 0
    logger.info(f"Duplicate (image, target) pairs: {duplicates}")
   
    if 'target' in train_df.columns and train_df['target'].std() > 0:
        z_scores = np.abs((train_df['target'] - train_df['target'].mean()) / train_df['target'].std())
        outliers = (z_scores > 3).sum()
    else:
        outliers = 0
    logger.info(f"Potential outliers (|Z-score| > 3): {outliers}")
   
    logger.info("\n✓ EDA completed successfully")
    logger.debug("="*80 + "\n")
    # MEMORY USAGE TRACKING
    logger.debug("\n" + "="*80)
    logger.debug("MEMORY USAGE")
    logger.debug("="*80)
    logger.debug(f"Train DataFrame: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    logger.debug(f"Test DataFrame: {test_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
   
    # LOOKUP INTEGRITY VALIDATION
    logger.info("\n" + "="*80)
    logger.info("LOOKUP VALIDATION")
    logger.info("="*80)
    total_level1 = sum(sum(len(rows) for rows in target_dict.values())
                       for target_dict in TRAIN_BY_SAMPLE_ID_TARGET.values())
    total_level2 = sum(sum(1 for _ in target_dict.keys())
                       for target_dict in TRAIN_BY_IMAGE_ID_TARGET.values())
    total_level3 = sum(sum(sum(len(rows) for rows in species_dict.values()) for species_dict in state_dict.values())
                       for state_dict in TRAIN_BY_TARGET_STATE_SPECIES.values())
    logger.info(f"✓ LEVEL 1: {total_level1} rows indexed (expected: {len(train_df)})")
    logger.info(f"✓ LEVEL 2: {total_level2} rows indexed (expected: {len(train_df)})")
    logger.info(f"✓ LEVEL 3: {total_level3} rows indexed (expected: {len(train_df)})")
   
    # FEATURE ENCODING VALIDATION
    logger.info("\n" + "="*80)
    logger.info("FEATURE ENCODING VALIDATION")
    logger.info("="*80)
    
    def _safe_row_sum(row, cols):
        """
        Given a pandas Series row and a list-like cols (may be empty),
        return numeric sum of those columns coercing non-numeric -> 0.0.
        """
        if not cols:
            return 0.0
        if isinstance(cols, str):
            cols = [cols]
        vals = row.loc[cols] if hasattr(row, 'loc') else pd.Series(dtype=float)
        numeric_vals = pd.to_numeric(vals, errors='coerce').fillna(0.0)
        return float(numeric_vals.sum())
    
    try:
        sample_row = train_df.iloc[0]
    except Exception:
        sample_row = pd.Series(dtype=float)

    if isinstance(STATE_COLS, str):
        STATE_COLS = [STATE_COLS]
    if isinstance(SPECIES_COLS, str):
        SPECIES_COLS = [SPECIES_COLS]
    if isinstance(TARGET_TYPE_COLS, str):
        TARGET_TYPE_COLS = [TARGET_TYPE_COLS]
    
    state_sum = _safe_row_sum(sample_row, STATE_COLS)
    species_sum = _safe_row_sum(sample_row, SPECIES_COLS)
    target_sum = _safe_row_sum(sample_row, TARGET_TYPE_COLS)
    
    tol = 1e-6
    logger.info(f"✓ State one-hot sum: {state_sum} (expected: ~1.0)")
    logger.info(f"✓ Species one-hot sum: {species_sum} (expected: ~1.0)")
    logger.info(f"✓ Target one-hot sum: {target_sum} (expected: ~1.0)")
    if abs(state_sum - 1.0) < tol and abs(species_sum - 1.0) < tol and abs(target_sum - 1.0) < tol:
        logger.info(f"✓ All encodings valid!")
    else:
        logger.warning("One-hot encoding sums deviate from 1.0 within tolerance; check categories and missing values.")

    return train_df, test_df, STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS, TOTAL_TABULAR_DIM, lookups_dict, original_test_df

logger.info("✓ Exploratory Data Analysis")

## 4: DATA AUGMENTATION AND PREPARATION

**Purpose**: Create dataset classes and data loaders with patch-based processing, improved feature estimation, and intelligent fallback.

### BiomassDataset Class

**Modes**:
- `train`: Augmentation enabled, extract from training data
- `val`: No augmentation, extract from training data
- `test`: No augmentation, intelligent feature fetching with 4-level fallback

**Image Processing Pipeline**:
```
2000×1000 Image
    ↓
Extract 8 patches (500×500 each)
├─ Patches 1-4: Top half (y: 0-500)
└─ Patches 5-8: Bottom half (y: 500-1000)
    ↓
Resize each to 224×224
    ↓
Stack into [8, 3, 224, 224] tensor
    ↓
Normalize (ImageNet mean/std)
```

**Data Augmentation** (Training Only):
- Horizontal flip: 50% probability
- Vertical flip: 50% probability
- Random rotation: ±15°
- Color jitter: brightness, contrast, saturation variation
- Affine transform: ±10% translation

**Validation/Test**: Deterministic (no augmentation)

**Feature Extraction** (IMPROVED):

For each sample, extract/estimate tabular features:

| Feature | Train/Val | Test Strategy |
|---------|-----------|---|
| NDVI | From training data | Level 1-3: training data lookup, Level 3a: estimated from image |
| Height | From training data | Level 1-3: training data lookup, Level 3a: **estimated from image (4-factor: texture+green+NDVI+brightness)** |
| Month | From training date | Level 1-3: training data lookup, Level 3a: **data-driven from NDVI (Gaussian likelihood, not random)** |
| Species | From training data | Level 1-3: training data lookup, Level 2a-3a: **predicted from image via species model** |
| State | From training data | Level 1-3: training data lookup, Level 4: random from available states |

**Intelligent Fallback (4 Levels)**:

| Level | Trigger | Features From | Quality |
|-------|---------|---|---|
| **1** | State + predicted species | Hybrid (lookup + image estimate) | ⭐⭐⭐ Medium |
| **2** | Exact match (sample_id + target) | Training data | ⭐⭐⭐⭐⭐ Highest |
| **3** | Image ID + species | Training data lookup | ⭐⭐⭐⭐ High |
| **4** | Not found (guaranteed to succeed) |  Statistics | ⭐⭐ Low |


**Success Rates**:
- Level 1: ~50-70% (hybrid match)
- Level 2: ~5-10% (exact match rare)
- Level 3: ~20-30% (image+species match)
- Level 4: 100% (always succeeds)

**Key Improvements**:- 
- ✅ **Better NDVI calculation:** Proper NIR channel (RGBN) with smart RGB fallback. Eliminates negative R² in vegetation areas.
- ✅ **Better height (4-factor):** Combines NDVI proxy, texture features, species scaling, and seasonal context.  more realistic across species.
- ✅ **Better month (data-driven):** Bayesian inference from training NDVI distribution. Prediction accuracy +45%, captures seasonal growth patterns.
- ✅ **Species integration:** ViT image classifier with 4-patch majority voting. Enables species-specific ensemble routing.
- ✅ **Robust fallback:** 5-level cascading fallbacks (full → minimal → constants). 100% pipeline success rate, never fails on edge cases.

```

                TEST SAMPLE (Missing: NDVI, Height, Month, Species, State)
                                            ↓

                ┌─────────────────────────────────────────────────────────────────┐
                │      LEVEL 1a v1: STATE + PREDICTED SPECIES (PRIMARY)           │
                │         Uses Estimated Features from Image                      │
                │              85% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_TARGET_STATE_SPECIES[state][pred_species]
                    Hit rate: ~35% (624/1,785)
                    
                    IF FOUND (any state):
                    ✅ Use any state with matching predicted species
                    ├─ NDVI (estimated from image via multi-factor)
                    ├─ Height (estimated from image texture/color/NDVI)
                    ├─ Month (estimated via Gaussian likelihood on NDVI)
                    ├─ DayOfYear, Quarter (derived from month)
                    ├─ Species (predicted from ViT)
                    └─ State (from matched training row)
                    
                                    IF NOT FOUND ↓
                    
                ┌─────────────────────────────────────────────────────────────────┐
                │      LEVEL 1a v2: STATE + PREDICTED SPECIES (RETRY)             │
                │        Uses Training Data Features (Fallback Features)          │
                │              75% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_TARGET_STATE_SPECIES[state][pred_species]
                
                    
                    IF FOUND:
                    ✅ Use any state with matching predicted species
                    ├─ NDVI (from training data)
                    ├─ Height (from training data)
                    ├─ Month (from training data)
                    ├─ DayOfYear, Quarter (from training data)
                    ├─ Species (predicted from ViT)
                    └─ State (from matched training row)
                    
                                    IF NOT FOUND ↓
                    
                ┌─────────────────────────────────────────────────────────────────┐
                │       LEVEL 1b: RANDOM STATE + SPECIES                          │
                │         Guaranteed Hit (Safety Net)                             │
                │              70% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_TARGET_STATE_SPECIES (any entry)
                    
                    IF FOUND:
                    ✅ Random state + random species from this target
                    ├─ NDVI (from training data)
                    ├─ Height (from training data)
                    ├─ Month (from training data)
                    ├─ DayOfYear, Quarter (from training data)
                    ├─ Species (random from training)
                    └─ State (random from training)
                    
                                    IF NOT FOUND ↓
                    
                ┌─────────────────────────────────────────────────────────────────┐
                │           LEVEL 2: EXACT (sample_id + target)                   │
                │                    100% Accuracy Fallback                       │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_SAMPLE_ID_TARGET[sample_id]
                    
                    IF FOUND:
                    ✅ Use exact training row
                    └─ NDVI, Height, Month, Species, State (all from training)
                    
                                        IF NOT FOUND ↓
                    
                ┌─────────────────────────────────────────────────────────────────┐
                │       LEVEL 3a: IMAGE + PREDICTED SPECIES                       │
                │              95% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_IMAGE_SPECIES_TARGET[image_id][pred_species]
                    
                    IF FOUND:
                    ✅ Use same image with matching predicted species
                    ├─ NDVI, Height, Month (from training data)
                    ├─ Species (predicted from ViT)
                    └─ State (from matched training row)
                    
                                        IF NOT FOUND ↓
                    
                ┌─────────────────────────────────────────────────────────────────┐
                │          LEVEL 3b: IMAGE ONLY FALLBACK                          │
                │              85% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                    Check: TRAIN_BY_IMAGE_ID_TARGET[image_id]
                    Hit rate: ~12% (214/1,785)
                    
                    IF FOUND:
                    ✅ Use same image (any species)
                    ├─ NDVI, Height, Month (from training data)
                    ├─ Species (from training data)
                    └─ State (from matched training row)
                    
                                 IF NOT FOUND ↓

                    
                ┌─────────────────────────────────────────────────────────────────┐
                │          LEVEL 4: STATISTICAL GENERATION                        │
                │     Data-Driven Synthetic Features (GUARANTEED 100%)            │
                │              60% Accuracy Fallback                              │
                └─────────────────────────────────────────────────────────────────┘
                            Check: TABULAR_STATS[target_name]
                                       ALWAYS WORKS
                    
                    ✅ ALWAYS SUCCEEDS - Generates synthetic features
                    
                    
                    GUARANTEES:
                    ✅ Zero NaN values
                    ✅ Valid ranges (NDVI ∈ [0, 1], Height ∈ [1, 70])
                    ✅ Statistically consistent with training data
                    ✅ Reproducible (seeded random)
                    ✅ Always returns 29-dim feature vector
                
                                    ↓ GUARANTEED SUCCESS
                
                ═══════════════════════════════════════════════════════════════════════
                            COMPLETE FEATURE VECTOR [29 dims]
                ─────────────────────────────────────────────────────────────────────
                [NDVI, Height, Month, DayOfYear, Quarter] + [State_OH, Species_OH]
                ═══════════════════════════════════════════════════════════════════════
                                              ↓
                                        MODEL INFERENCE
                                              ↓
                                          PREDICTION 
                ═══════════════════════════════════════════════════════════════════════


```


**Feature Replication** (Critical Design):
- 8 patches from one image → 8 identical tabular features
- 8 patches → 8 identical targets
- Result: Consistent training signal with spatial diversity

**Custom Collate Function**:
```
Input: 8 image items from DataLoader
    ↓
Each image: 8 patches + 29 tabular features + 1 target
    ↓
Concatenate across batch:
- images: [64, 3, 224, 224]    (8 images × 8 patches)
- tabular: [64, 29]             (replicated features)
- targets: [64]                 (replicated targets)
```

**Data Loaders Created**:

| Loader | Size | Batches | Purpose |
|--------|------|---------|---------|
| Training | 1,520 rows | 190 batches | Training (85% split) + augmentation |
| Validation | 265 rows | 34 batches | Validation (15% split) + no augmentation |
| Test | 5 rows | 1 batch | Prediction + intelligent fallback |

**Output**: 
- BiomassDataset class ready with improved feature extraction
- Train/val/test loaders created
- Collate function verified for patch batching
- Species model integrated for fallback matching
- Feature caching enabled for efficiency

In [ ]:
class BiomassDataset(Dataset):
    """
    Custom PyTorch Dataset for pasture biomass prediction with intelligent test feature fetching.
    
    Features:
    - Crops each 2000×1000 image into 8 patches of 500×500 pixels (4×2 grid)
    - Resizes patches to 224×224 for model input
    - Intelligent test data feature fetching with multi-level fallback 
    - Handles training, validation, and test modes
    """
    def __init__(self, dataframe, image_dir, transform=None, mode='train',
                 train_df=None,lookups_dict=None, species_model=None, device=None ):
        """
        Initialize dataset.
        Args:
            dataframe (pd.DataFrame): Input dataframe with image paths and targets
            image_dir (str): Directory containing images
            transform: Image augmentation transforms
            mode (str): 'train', 'val', or 'test'
            train_df (pd.DataFrame): Training dataframe (for test feature fetching)
            lookups_dict (dict): Lookup tables (for test feature fetching)
        """
        self.mode = mode
        self.species_model = species_model
        try:
            if device is None:
                self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            else:
                self.device = device if isinstance(device, torch.device) else torch.device(device)
        except Exception:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.species_cache = {}
        self.test_features_cache = {}
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.train_df = train_df
        self.lookups_dict = {} if lookups_dict is None else lookups_dict
        self.image_width = CONFIG['image_width']
        self.image_height = CONFIG['image_height']
        self.patch_size = CONFIG['patch_size']
        self.patches_horizontal = CONFIG['patches_horizontal']
        self.patches_vertical = CONFIG['patches_vertical']
        self.num_patches = CONFIG['num_patches']
        self.model_input_size = (CONFIG['model_input_size'], CONFIG['model_input_size'])
        self.df = dataframe.reset_index(drop=True).copy()
        self.mark_missing_images()
        if self.mode == 'test':
            logger.info(f"✓ Test data: {len(self.df)} rows (one per target type)")
            self.test_features_cache = {}
        try:
            hmax = self.lookups_dict.get("HEIGHT_MAX", None) 
            if hmax is None and self.train_df is not None and 'Height_Ave_cm' in self.train_df.columns:
                try:
                    hmax = float(self.train_df['Height_Ave_cm'].dropna().astype(float).max())
                except Exception:
                    hmax = None
            if hmax is None or (not np.isfinite(hmax)) or (hmax <= 0):
                hmax = 80.0
            hmax = float(min(max(hmax, 1.0), 150.0))
    
        except Exception:
            hmax = 80.0
        self.HEIGHT_MAX = hmax
        self.lookups_dict['HEIGHT_MAX'] = hmax
        logger.debug(f"✓ Using dynamic HEIGHT_MAX = {self.HEIGHT_MAX:.2f} cm")
        try:
            if getattr(self, 'train_df', None) is not None and 'Species' in self.train_df.columns:
                uniq_species = sorted(self.train_df['Species'].dropna().unique().tolist())
                self.species_to_idx = {s: i for i, s in enumerate(uniq_species)}
            else:
                self.species_to_idx = {}
        except Exception:
            self.species_to_idx = {}

    def __len__(self):
        return len(self.df)
        
    # Sentinel color for missing/corrupt images (magenta)
    SENTINEL_COLOR = (255, 0, 255)
    SENTINEL_THRESHOLD = 0.90  # 90%+ magenta → invalid image

    
    def _is_sentinel_image(self, img_pil):
        try:
            if not isinstance(img_pil, Image.Image):
                return False
            arr = np.asarray(img_pil.convert("RGB")).astype(np.int16)
            r_ok = (arr[:, :, 0] >= 240)
            g_ok = (arr[:, :, 1] <= 15)
            b_ok = (arr[:, :, 2] >= 240)
            mask = r_ok & g_ok & b_ok
            frac = mask.sum() / mask.size
            return float(frac) >= float(self.SENTINEL_THRESHOLD)
        except Exception:
            return False

    
    def _resolve_image_path(self, img_path_str):
        if img_path_str is None or (isinstance(img_path_str, str) and str(img_path_str).strip() == ""):
            return Path("")  # empty Path => .exists() is False, handled downstream as missing
        s = str(img_path_str).strip().lstrip("/")
        # remove repeated leading train/ or test/
        s_clean = re.sub(r'^(?:train/|test/)+', '', s, flags=re.IGNORECASE)
        candidates = [
            Path(s),                                 
            Path(self.image_dir) / s,                 
            Path(self.image_dir) / s_clean,           
            Path(self.image_dir) / Path(s).name,      
            Path(self.image_dir) / "train" / Path(s).name,
            Path(self.image_dir) / "test" / Path(s).name,]
        for p in candidates:
            if p.exists():
                return p.resolve()
        try:
            return (Path(self.image_dir) / s).resolve()
        except Exception:
            return Path(self.image_dir)

    
    @staticmethod
    def _normalize_state_for_onehot(s):
        if s is None:
            return ""
        s = str(s).strip().lower()
        s = re.sub(r'[^a-z0-9_]', '_', s)
        s = re.sub(r'^state_', '', s)
        return s

    
    def mark_missing_images(self):
        missing_count = 0
        missing_indices = []
        for idx, row in self.df.iterrows():
            img_path_raw = row.get('image_path', '')
            img_path = self._resolve_image_path(img_path_raw)
            
            # CASE 1: file missing
            file_missing = not img_path.exists()
    
            # CASE 2: file exists but unreadable (cv2 returns None)
            unreadable = False
            if not file_missing:
                try:
                    test_img = cv2.imread(str(img_path))
                    if test_img is None:
                        unreadable = True
                except:
                    unreadable = True
    
            # If missing or unreadable → set species = Nothing
            if file_missing or unreadable:
                self.df.at[idx, 'is_missing'] = True
    
                # Set target 0.0 for training only 
                if 'target' in self.df.columns and self.mode != 'test':
                    self.df.at[idx, 'target'] = 0.0
    
                # Set SPECIES = "Nothing"
                if 'Species' in self.df.columns:
                    self.df.at[idx, 'Species'] = "Nothing"
                missing_count += 1
                missing_indices.append(idx)
            else:
                self.df.at[idx, 'is_missing'] = False
        if missing_count > 0:
            logger.debug(f"✓ Marked {missing_count} missing/unreadable images")
            if missing_count <= 10:
                logger.debug(f"  Missing indices: {missing_indices}")
        else:
            logger.info("✓ All images found & readable!")

    
    def _extract_train_tabular(self, row):
        """
        Build tabular feature tensor for training/validation rows.
        Numeric columns are scaled using lookups_dict['TAB_SCALER'].
        One-hot columns are appended unchanged (0/1).
        """
        numeric_cols = self.lookups_dict.get('TAB_NUMERIC_COLS', [])
        scaler = self.lookups_dict.get('TAB_SCALER', None)
    
        # build raw numeric vector in same order as numeric_cols
        raw_vals = []
        for c in numeric_cols:
            if c == 'Pre_GSHH_NDVI':
                raw_vals.append(float(row.get('Pre_GSHH_NDVI', 0.5)))
            elif c == 'Height_Ave_cm':
                raw_vals.append(float(row.get('Height_Ave_cm', 10.0)))  
            elif c == 'Month':
                raw_vals.append(float(row.get('Month', 6)))  
            elif c == 'DayOfYear':
                raw_vals.append(float(row.get('DayOfYear', 1)))  
            elif c == 'Quarter':
                raw_vals.append(float(row.get('Quarter', 1)))  
            else:
                raw_vals.append(float(row.get(c, 0.0)))
        
        # Step 2: Clip to REALISTIC ranges 
        clipped = []
        for c, v in zip(numeric_cols, raw_vals):
            if c == "Pre_GSHH_NDVI":
                # NDVI ranges from -1 to 1, 
                clipped.append(np.clip(v, 0.0, 1.0))
            elif c == "Height_Ave_cm":
                # Height in cm: realistic range 0-100 cm
                clipped.append(np.clip(v, 0.0,100.0))
            elif c == "Month":
                # Month: 1-12
                clipped.append(np.clip(v, 1.0, 12.0))
            elif c == "DayOfYear":
                # Day of year: 1-365
                clipped.append(np.clip(v, 1.0, 365.0))
            elif c == "Quarter":
                # Quarter: 1-4
                clipped.append(np.clip(v, 1.0, 4.0))
            else:
                clipped.append(v)
        raw_vals = clipped

        # scale numeric fields if scaler available
        if scaler is not None and len(raw_vals) == len(numeric_cols):
            try:
                scaled_nums = scaler.transform([np.nan_to_num(raw_vals, nan=0.0)])[0].tolist()
            except Exception as e:
                logger.warning(f"_extract_train_tabular: scaler transform failed: {e}")
                scaled_nums = raw_vals
        else:
            scaled_nums = raw_vals
        features = list(scaled_nums)
    
        # helper: safe numeric conversion
        def _safe_to_float(x):
            if isinstance(x, (bool, np.bool_)):
                return 1.0 if x else 0.0
            try:
                return float(x)
            except Exception:
                v = pd.to_numeric(x, errors='coerce')
                return float(0.0 if pd.isna(v) else v)
        
        # get lists from lookups 
        STATE_COLS = self.lookups_dict.get('STATE_COLS', []) or []
        SPECIES_COLS = self.lookups_dict.get('SPECIES_COLS', []) or []
        TARGET_TYPE_COLS = self.lookups_dict.get('TARGET_TYPE_COLS', []) or []
        
        # LOG for debugging 
        logger.debug(f"Using STATE_COLS: {STATE_COLS}")
        logger.debug(f"Using SPECIES_COLS: {SPECIES_COLS}")
        logger.debug(f"Using TARGET_TYPE_COLS: {TARGET_TYPE_COLS}")
        
        # 1) Try to use numeric dummy columns directly 
        for col in STATE_COLS:
            features.append(_safe_to_float(row.get(col, 0)))
        
        for col in SPECIES_COLS:
            features.append(_safe_to_float(row.get(col, 0)))
        
        for col in TARGET_TYPE_COLS:
            features.append(_safe_to_float(row.get(col, 0)))
        
        # 2) If the dummy lists are empty OR all zeros (i.e. missing), optionally fallback:
        # convert categorical 'State' / 'Species' / 'target_name' into one-hot matching the dummy columns.
        def _maybe_onehot_fallback(row, cols, cat_candidates):
            """
            If all values from `cols` are zeros and we have a categorical col present (e.g. 'State' or 'Species'),
            return a one-hot vector aligned to `cols`. Otherwise return None (meaning no fallback needed).
            """
            if not cols:
                return None
            vals = [row.get(c, 0) for c in cols]
            # if there exists any numeric non-zero -> no fallback
            numeric_vals = pd.to_numeric(pd.Series(vals), errors='coerce').fillna(0.0)
            if numeric_vals.sum() > 0:
                return None
        
            # try to find a categorical column among candidates
            cat_val = None
            for cand in cat_candidates:
                if cand in row.index:
                    raw = row.get(cand)
                    if pd.isna(raw) or raw == '':
                        continue
                    cat_val = str(raw).strip().lower()
                    break
            if cat_val is None:
                return None
        
            # build one-hot by tolerant matching between column names and cat_val
            onehot = []
            for c in cols:
                c_norm = c.lower()
                score = 0
                if c_norm.endswith(cat_val) or c_norm.endswith(cat_val.replace(' ', '_')):
                    score = 2
                elif cat_val in c_norm or c_norm in cat_val:
                    score = 1
                elif c_norm.split('_')[-1][:3] == cat_val[:3]:
                    score = 1
                onehot.append(1.0 if score > 0 else 0.0)
        
            logger.debug(f"Fallback one-hot for {cat_candidates}='{cat_val}' => {onehot}")
            return onehot
        
        # apply fallbacks only if initial dummy columns were all zeros / absent
        state_fallback = _maybe_onehot_fallback(row, STATE_COLS, ['State', 'state', 'State_canon', 'state_name'])
        if state_fallback is not None:
            if STATE_COLS:
                features[-len(STATE_COLS):] = state_fallback
        
        species_fallback = _maybe_onehot_fallback(row, SPECIES_COLS, ['Species', 'species', 'Species_canon'])
        if species_fallback is not None:
            if SPECIES_COLS:
                features[-len(SPECIES_COLS):] = species_fallback
        
        target_fallback = _maybe_onehot_fallback(row, TARGET_TYPE_COLS, ['target_name', 'Target', 'target'])
        if target_fallback is not None:
            if TARGET_TYPE_COLS:
                features[-len(TARGET_TYPE_COLS):] = target_fallback
        return torch.tensor(features, dtype=torch.float32)

    def predicting_species(self, test_row, species_model=None, device=None):
        """
        Predict species for a single test_row.
    
        Supports:
          - per-patch outputs (one logits row per patch) -> argmax per patch -> majority vote
          - MIL-style: image-level logits (one logits row for the image) -> single prediction
          - outputs as tensor, (tensor, ...) tuple, or dict with "logits" or "pred"
          - dict outputs like {'pred': '<name>'} are handled
    
        Returns canonical species string (via map_to_fixed_species) or None.
        
        """
        if device is None:
            device = getattr(self, "device", None)
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if species_model is None:
            logger.debug("predicting_species: species_model is None -> returning None")
            return None
    
        sample_id = test_row.get("sample_id")
        image_id = str(sample_id).split("__")[0] if sample_id is not None else None
        cache_key = image_id.lower() if image_id is not None else None
        if cache_key is not None and cache_key in getattr(self, "species_cache", {}):
            return self.species_cache[cache_key]
    
        img_path = self._resolve_image_path(test_row.get('image_path', ''))
        try:
            if not img_path.exists():
                logger.info(f"predicting_species: image not found: {img_path}")
                return None
            image = self._load_image(img_path)
            if not isinstance(image, Image.Image):
                image = Image.fromarray(np.asarray(image))
    
            patches = self._crop_patches(image)
            if not patches:
                logger.debug(f"predicting_species: no patches for {img_path}")
                return None
    
            # Prepare device and remember original device
            try:
                orig_device = next(species_model.parameters()).device
            except Exception:
                orig_device = getattr(species_model, "device", None)
    
            target_device = device
            moved_model = False
            try:
                if orig_device is None or orig_device != target_device:
                    species_model.to(target_device)
                    moved_model = True
            except Exception:
                logger.info("predicting_species: could not move species_model to target device; continuing")
    
            species_model.eval()
            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
            patch_tensors = []
            for p in patches:
                if not isinstance(p, Image.Image):
                    p = Image.fromarray(np.asarray(p))
                patch_tensors.append(transform(p))
    
            batch = torch.stack(patch_tensors, dim=0).to(target_device)
    
            with torch.no_grad():
                outputs = species_model(batch, return_features=False)
    
            logits = None
            if isinstance(outputs, dict):
                if 'pred' in outputs:
                    raw_pred = outputs.get('pred')
                    if isinstance(raw_pred, (list, tuple, np.ndarray)):
                        raw_pred = raw_pred[0] if len(raw_pred) > 0 else None
                    if raw_pred is not None:
                        raw_pred = str(raw_pred).strip()
                    mapped = map_to_fixed_species(raw_pred) if raw_pred else None
                    if cache_key is not None:
                        self.species_cache[cache_key] = mapped
                    return mapped
                if 'species' in outputs and isinstance(outputs['species'], str):
                    raw_pred = outputs['species']
                    mapped = map_to_fixed_species(raw_pred)
                    if cache_key is not None:
                        self.species_cache[cache_key] = mapped
                    return mapped
                logits = outputs.get('logits', None)
                if logits is None:
                    for v in outputs.values():
                        if torch.is_tensor(v):
                            logits = v
                            break
    
            elif isinstance(outputs, (list, tuple)):
                first = outputs[0] if len(outputs) > 0 else None
                if isinstance(first, dict):
                    if 'pred' in first:
                        raw_pred = first.get('pred')
                        if isinstance(raw_pred, (list, tuple, np.ndarray)):
                            raw_pred = raw_pred[0] if len(raw_pred) > 0 else None
                        if raw_pred is not None:
                            raw_pred = str(raw_pred).strip()
                        mapped = map_to_fixed_species(raw_pred) if raw_pred else None
                        if cache_key is not None:
                            self.species_cache[cache_key] = mapped
                        return mapped
                    logits = first.get('logits', None)
                else:
                    logits = first
            else:
                logits = outputs
    
            if logits is None or not torch.is_tensor(logits):
                logger.info("predicting_species: could not interpret model outputs -> returning None")
                return None
    
            # detach to CPU for analysis
            logits_detached = logits.detach().cpu()
    
            # Determine per-patch vs image-level
            per_patch_mode = False
            img_level_logits = None
            per_patch_logits = None
    
            if logits_detached.dim() == 1:
                # single-vector -> image-level
                img_level_logits = logits_detached.unsqueeze(0)
            elif logits_detached.dim() == 2:
                n_rows = logits_detached.shape[0]
                n_patches = len(patches)
                if n_rows == n_patches:
                    per_patch_mode = True
                    per_patch_logits = logits_detached
                elif n_rows == 1:
                    img_level_logits = logits_detached
                else:
                    img_level_logits = logits_detached.mean(dim=0, keepdim=True)
            else:
                try:
                    flat = logits_detached.view(logits_detached.shape[0], -1)
                    if flat.dim() == 2 and flat.shape[0] == len(patches):
                        per_patch_mode = True
                        per_patch_logits = flat
                    else:
                        img_level_logits = flat.mean(dim=0, keepdim=True)
                except Exception:
                    logger.info("predicting_species: unexpected logits shape; returning None")
                    return None
    
            predicted_species = None
    
            if per_patch_mode:
                preds = torch.argmax(per_patch_logits, dim=1).cpu().numpy().tolist()
                # Map indices -> species names
                predicted_list = []
                
                for idx_val in preds:
                    idx_val = int(idx_val)
                    species_name = None
                    
                    # Try model's species_list first
                    if hasattr(species_model, "species_list") and isinstance(species_model.species_list, (list, tuple)):
                        if 0 <= idx_val < len(species_model.species_list):
                            species_name = species_model.species_list[idx_val]
                        else:
                            logger.warning(f"Per-patch: Index {idx_val} out of range for species_list (len={len(species_model.species_list)})")
                    
                    # Try idx_to_species dict if species_list didn't work
                    if species_name is None:
                        idx_to_species = getattr(species_model, "idx_to_species", None) or getattr(species_model, "idx_to_class", None)
                        if isinstance(idx_to_species, dict):
                            if idx_val in idx_to_species:
                                species_name = idx_to_species[idx_val]
                            else:
                                logger.warning(f"Per-patch: Index {idx_val} not in idx_to_species. Available: {list(idx_to_species.keys())}")
                        else:
                            logger.warning("Per-patch: No species_list or idx_to_species available!")
                    
                    # Fallback: if still None, use "Unknown" marker
                    if species_name is None:
                        species_name = "Unknown"
                    
                    predicted_list.append(str(species_name).strip())
    
                predicted_list_mapped = [map_to_fixed_species(p) for p in predicted_list]
                predicted_species = max(set(predicted_list_mapped), key=predicted_list_mapped.count) if predicted_list_mapped else None
    
            else:
                # single-image logits -> take argmax
                try:
                    probs = torch.nn.functional.softmax(img_level_logits, dim=1)
                    idx = int(torch.argmax(probs, dim=1).cpu().item())
                except Exception:
                    idx = int(torch.argmax(img_level_logits, dim=1).cpu().item())
                
                raw_name = None
                if hasattr(species_model, "species_list") and isinstance(species_model.species_list, (list, tuple)):
                    if 0 <= idx < len(species_model.species_list):
                        raw_name = species_model.species_list[idx]
                    else:
                        logger.warning(f"Image-level: Index {idx} out of range for species_list (len={len(species_model.species_list)})")
                
                if raw_name is None:
                    idx_to_species = getattr(species_model, "idx_to_species", None) or getattr(species_model, "idx_to_class", None)
                    if isinstance(idx_to_species, dict):
                        if idx in idx_to_species:
                            raw_name = idx_to_species[idx]
                        else:
                            logger.warning(f"Image-level: Index {idx} not in idx_to_species. Available: {list(idx_to_species.keys())}")
                    else:
                        logger.warning("Image-level: No species mapping available!")
                
                if raw_name is None:
                    raw_name = "Unknown"
                
                predicted_species = map_to_fixed_species(str(raw_name).strip())
            
            logger.debug(f"✓ Predicted species: {predicted_species}")
            if cache_key is not None:
                if not hasattr(self, "species_cache"):
                    self.species_cache = {}
                self.species_cache[cache_key] = predicted_species
            return predicted_species
    
        except Exception as e:
            logger.exception(f"⚠ predicting_species failed: {e}")
            return None
    
        finally:
            # ensure model moved back to original device when possible
            try:
                if 'moved_model' in locals() and moved_model and orig_device is not None:
                    species_model.to(orig_device)
            except Exception:
                pass

    
    def estimate_height_final(self, image):
        try:
            try:
                if not getattr(self, "height_model", None):
                    try:
                        out_dir = OUTPUTS_DIR if isinstance(OUTPUTS_DIR, Path) else Path(OUTPUTS_DIR)
                        ridge_path = out_dir / "height_ridge.joblib"
                        pl_path = out_dir / "height_powerlaw.json"
    
                        if ridge_path.exists():
                            try:
                                ridge_model = joblib.load(str(ridge_path))
                                self.height_model = ridge_model
                                self.height_model_type = getattr(ridge_model, "height_model_type", "log1p")
                                logger.info(f"Loaded ridge height_model from {ridge_path}")
                            except Exception as e:
                                logger.info(f"Failed to load ridge model: {e}")
    
                        # Load powerlaw params if available 
                        if (not hasattr(self, "height_a") or not hasattr(self, "height_b")) and pl_path.exists():
                            try:
                                js = json.loads(pl_path.read_text())
                                self.height_a = float(js.get("a", getattr(self, "height_a", None) or 0.0))
                                self.height_b = float(js.get("b", getattr(self, "height_b", None) or 1.0))
                                # set sensible model type for powerlaw fallback
                                if not hasattr(self, "height_model_type"):
                                    self.height_model_type = "log1p_powerlaw"
                                logger.info(f"Loaded powerlaw params from {pl_path}")
                            except Exception as e:
                                logger.debug(f"Failed to read powerlaw json: {e}")
                    except Exception:
                        pass
            except Exception:
                pass
    
            # Convert to PIL for sentinel detection
            if isinstance(image, Image.Image):
                img_pil = image
            else:
                img_pil = Image.fromarray(np.uint8(image))
            if self._is_sentinel_image(img_pil):
                return 0.0
    
            # Extract features
            feats = self.compute_features_from_image(image)
            ndvi = float(np.clip(feats.get('ndvi', 0.5), 0.0, 1.0))
            texture_norm = float(np.clip(feats.get('texture', 0.02), 0.0, 1.0))
            green_intensity = float(np.clip(feats.get('green', 0.4), 0.0, 1.0))
            brightness = float(np.clip(feats.get('brightness', 0.5), 0.0, 1.0))
            height_proxy = float(np.clip(feats.get('height_proxy', 0.4), 0.0, 1.0))
            veg_frac = float(np.clip(feats.get('veg_frac', 0.5), 0.0, 1.0))
    
            # combined score 
            MIN_SCORE = 1e-2
            combined_score = (
                0.35 * ndvi +
                0.25 * texture_norm +
                0.20 * green_intensity +
                0.10 * brightness +
                0.10 * height_proxy)
            combined_score = float(
                np.clip(combined_score * (0.5 + 0.5 * feats.get('veg_frac', 1.0)),
                        MIN_SCORE, 1.0))
    
            height_cm = None
    
            # 1) USE TRAINED MODEL IF AVAILABLE
            # --------------------------------------
            if hasattr(self, "height_model") and self.height_model is not None:
                try:
                    vec = np.array([[ndvi, texture_norm, green_intensity,
                                     brightness, height_proxy, veg_frac]], dtype=float)
                    pred_val = float(self.height_model.predict(vec)[0])
    
                    model_type = getattr(self, "height_model_type", "log1p")
    
                    if model_type == "log":
                        height_cm = float(np.exp(pred_val))
                    elif model_type == "log1p":
                        height_cm = float(np.expm1(pred_val))
                    else:
                        height_cm = float(pred_val)
                except Exception as e:
                    logger.info(f"Calibrated height_model predict failed: {e}")
                    height_cm = None
    
            # 2) POWER-LAW FALLBACK (use saved a/b if available)
            # --------------------------------------
            if height_cm is None or not np.isfinite(height_cm):
                a = getattr(self, "height_a", None)
                b = getattr(self, "height_b", None)
                model_type = getattr(self, "height_model_type", "log1p_powerlaw")
    
                cs = max(float(combined_score), 1e-6)
    
                if a is not None and b is not None:
                    try:
                        # If calibration used log1p 
                        if model_type == "log1p_powerlaw":
                            height_cm = float(np.expm1(a + b * np.log1p(cs)))
                        elif model_type == "log":
                            # classic log form: log(h) = log(a) + b*log(score)
                            height_cm = float(np.exp(np.log(a) + b * np.log(max(cs, 1e-6))))
                        else:
                            # classic powerlaw a * cs ** b
                            height_cm = float(a * (cs ** b))
                    except Exception as e:
                        logger.info(f"Powerlaw computation failed: {e}")
                        height_cm = None
                else:
                    #emergency defaults
                    height_cm = 7.7
    
            # Final clip
            return float(np.clip(height_cm, 0.0, getattr(self, "HEIGHT_MAX", 100.0)))
    
        except Exception as e:
            logger.exception(f"estimate_height_final failed: {e}")
            return 7.6

    
    def compute_features_from_image(self, image):
        """
        Return a dict of features needed by calibrator:
        keys: ndvi, texture, green, brightness, height_proxy, veg_frac
        Accepts PIL Image or numpy array.
        """
        try:
            if isinstance(image, Image.Image):
                img_pil = image.convert("RGB")
                img_np = np.asarray(img_pil, dtype=np.float32)
            else:
                # if it's a numpy array, create a PIL for sentinel check
                img_np = np.asarray(image, dtype=np.float32)
                try:
                    img_pil = Image.fromarray(
                        (np.clip(img_np, 0.0, 1.0) * 255).astype('uint8')
                        if img_np.max() <= 1.0 else img_np.astype('uint8')
                    ).convert("RGB")
                except Exception:
                    img_pil = None

            # sentinel check 
            if img_pil is not None and self._is_sentinel_image(img_pil):
                return {
                    'ndvi': 0.0,
                    'texture': 0.0,
                    'green': 0.0,
                    'brightness': 0.0,
                    'height_proxy': 0.0,
                    'veg_frac': 0.0}

            if img_np.size == 0:
                return {'ndvi':0.5,'texture':0.02,'green':0.4,'brightness':0.5,'height_proxy':0.4,'veg_frac':0.5}
            if img_np.max() > 1.0:
                img_np = img_np / 255.0
            gray = np.mean(img_np, axis=2) if img_np.ndim==3 else img_np
            lap = laplace(gray)
            texture = float(np.nanmean(np.abs(lap)))
            texture = float(np.clip(texture/100.0, 0.0, 1.0))
            green = img_np[:,:,1] if img_np.ndim==3 and img_np.shape[2]>=2 else gray
            green = float(np.nanmean(green))
            green = float(np.clip(green,0.0,1.0))
            brightness = float(np.nanmean(gray))
            height_proxy = self._get_canopy_height_proxy(img_np)
            # veg_frac: fraction of pixels considered vegetation
            try:
                R = img_np[:,:,0] if img_np.ndim==3 else gray
                G = img_np[:,:,1] if img_np.ndim==3 else gray
                veg_mask = (G > R*1.02) & (G > 0.08)
                veg_frac = float(np.clip(np.sum(veg_mask)/veg_mask.size, 0.0, 1.0))
            except Exception:
                veg_frac = 0.5
            # vNDVI fallback
            try:
                R = img_np[:,:,0] if img_np.ndim==3 else gray
                G = img_np[:,:,1] if img_np.ndim==3 else gray
                denom = G + R
                denom[denom==0] = 1e-8
                vndvi = float(np.nanmean((G-R)/denom))
                vndvi = float(np.clip(vndvi, 0.0, 1.0))
            except Exception:
                vndvi = 0.5
            return {'ndvi': vndvi, 'texture': texture, 'green': green, 'brightness': brightness, 'height_proxy': height_proxy, 'veg_frac': veg_frac}
        except Exception as e:
            logger.debug(f"compute_features_from_image failed: {e}")
            return {'ndvi':0.5,'texture':0.02,'green':0.4,'brightness':0.5,'height_proxy':0.4,'veg_frac':0.5}

    
    def _get_canopy_height_proxy(self, img_np):
        """
        Extract canopy height proxy from image (robustified).
        Expects img_np normalized to [0,1] range (function will attempt to normalize).
        Returns float in [0,1].
    
        """
        try:
            arr = np.asarray(img_np, dtype=np.float32)
            if arr.size == 0:
                return 0.4
    
            if arr.max() > 1.0:
                arr = arr / 255.0
    
            # grayscale
            if arr.ndim == 3:
                gray = np.mean(arr, axis=2)
            else:
                gray = arr
    
            # vegetation mask 
            try:
                if arr.ndim == 3 and arr.shape[2] >= 2:
                    R = arr[:, :, 0]
                    G = arr[:, :, 1]
                    factor = 1.05
                    min_green = max(0.08, np.percentile(G.ravel(), 25) * 0.6)
                    veg_mask = (G > R * factor) & (G > min_green)
                    if np.sum(veg_mask) < max(10, 0.01 * gray.size):
                        veg_mask = (G > R * 1.02) & (G > np.percentile(G.ravel(), 10) * 0.4)
                    if np.sum(veg_mask) == 0:
                        veg_mask = np.ones_like(gray, dtype=bool)
                else:
                    veg_mask = np.ones_like(gray, dtype=bool)
            except Exception:
                veg_mask = np.ones_like(gray, dtype=bool)
    
            try:
                if np.any(veg_mask):
                    texture_variance = float(np.var(gray[veg_mask]))
                else:
                    texture_variance = float(np.var(gray))
            except Exception:
                texture_variance = 0.02
    
            # local contrast 
            try:
                values = gray[veg_mask].ravel()
                p90 = float(np.percentile(values, 90))
                p10 = float(np.percentile(values, 10))
                contrast = max(0.0, p90 - p10)
            except Exception:
                contrast = 0.1
    
            proxy_raw = 0.7 * texture_variance + 0.3 * (contrast ** 1.0)
            # robust normalization: divide by reasonable expected max (0.08) then clip
            height_proxy = float(np.clip(proxy_raw / (0.08 + 1e-8), 0.0, 1.0))
    
            logger.debug(f"Canopy height proxy (var={texture_variance:.4f}, contrast={contrast:.4f}) -> {height_proxy:.3f}")
            return height_proxy
    
        except Exception as e:
            logger.error(f"Canopy height proxy estimation failed: {e}")
            return 0.4

    
    def estimate_month_and_ndvi_from_image(self, image):
        """
        Estimate growing season month AND vegetation health (NDVI) from image.
    
        Args:
            image: numpy array of shape (H, W, C) or PIL Image
    
        Returns:
            tuple: (estimated_month (int), estimated_ndvi (float))
        """
        try:
            if isinstance(image, Image.Image):
                img_np = np.asarray(image, dtype=np.float32)
            else:
                img_np = np.asarray(image, dtype=np.float32)
    
            # Validate shape
            if img_np.ndim != 3:
                logger.warning(f"Unexpected image shape: {getattr(img_np, 'shape', None)}")
                return 6, 0.5
    
            h, w, c = img_np.shape
            img_np = img_np.astype(np.float32)
    
            # scale to [0,1] if necessary
            if img_np.max() > 1.0:
                img_np = img_np / 255.0
    
            RED = img_np[:, :, 0]
            GREEN = img_np[:, :, 1]
            BLUE = img_np[:, :, 2] if c >= 3 else np.zeros_like(RED)
    
            denom = GREEN + RED
            denom_safe = denom.copy()
            denom_safe[np.isclose(denom_safe, 0.0)] = 1e-8
    
            # vNDVI = (GREEN - RED) / (GREEN + RED)
            vndvi_array = (GREEN - RED) / denom_safe
            # Clip individual pixel index to reasonable range then mean
            vndvi_array_clipped = np.clip(vndvi_array, -1.0, 1.0)
            mean_vndvi = float(np.nanmean(vndvi_array_clipped))
    
            # Validate
            if np.isnan(mean_vndvi) or np.isinf(mean_vndvi):
                logger.warning("Invalid vNDVI detected, using fallback")
                return 6, 0.5
    
            # Map vNDVI (raw) to [0,1] reasonable range for calibration
            # Some vNDVI may be slightly negative for non-vegetation; clamp to 0..1 for month mapping
            mean_vndvi_clipped = float(np.clip(mean_vndvi, 0.0, 1.0))
    
            default_true_min = 0.16
            default_true_max = 0.91
    
            # Default expected raw RGB-ensemble/vNDVI range 
            rgb_raw_min = 0.0
            rgb_raw_max = 0.80
    
            # Try to read calibration stats from /mnt/data/test.csv if available and has useful columns
            try:
                calib_path = "/kaggle/working/outputs/ndvidata.csv"
                if os.path.exists(calib_path):
                    try:
                        df_cal = pd.read_csv(calib_path, nrows=5)
                        # Look for common columns names: avg, min, max OR ndvi_avg, ndvi_min, ndvi_max OR avg_ndvi, min_ndvi, max_ndvi
                        if set(['avg','min','max']).issubset(set(df_cal.columns)):
                            true_min = float(df_cal['min'].dropna().iat[0])
                            true_max = float(df_cal['max'].dropna().iat[0])
                            logger.info("Calibration loaded from test.csv using columns avg/min/max")
                        elif set(['ndvi_avg','ndvi_min','ndvi_max']).issubset(set(df_cal.columns)):
                            true_min = float(df_cal['ndvi_min'].dropna().iat[0])
                            true_max = float(df_cal['ndvi_max'].dropna().iat[0])
                            logger.info("Calibration loaded from test.csv using ndvi_avg/min/max")
                        elif set(['avg_ndvi','min_ndvi','max_ndvi']).issubset(set(df_cal.columns)):
                            true_min = float(df_cal['min_ndvi'].dropna().iat[0])
                            true_max = float(df_cal['max_ndvi'].dropna().iat[0])
                            logger.info("Calibration loaded from test.csv using avg_ndvi/min_ndvi/max_ndvi")
                        else:
                            # Fallback to using first-row numeric values if present 
                            first_row = df_cal.iloc[0].to_dict()
                            nums = [v for v in first_row.values() if isinstance(v, (int, float, np.floating, np.integer))]
                            if len(nums) >= 3:
                                # assume ordering avg,min,max or avg,min,max-like — pick min and max heuristically
                                true_min = float(min(nums))
                                true_max = float(max(nums))
                                logger.info("Calibration heuristically inferred from test.csv first row")
                            else:
                                true_min, true_max = default_true_min, default_true_max
                    except Exception as e:
                        logger.debug(f"Calibration read failed: {e}")
                        true_min, true_max = default_true_min, default_true_max
                else:
                    true_min, true_max = default_true_min, default_true_max
            except Exception:
                true_min, true_max = default_true_min, default_true_max
    
            # If the inferred true range is degenerate, fall back
            if not (np.isfinite(true_min) and np.isfinite(true_max) and (true_max > true_min)):
                true_min, true_max = default_true_min, default_true_max
    
            fused = mean_vndvi_clipped  
            denom_scale = (rgb_raw_max - rgb_raw_min) if (rgb_raw_max - rgb_raw_min) != 0 else 1e-8
            fused_calibrated = (fused - rgb_raw_min) / denom_scale
            fused_calibrated = fused_calibrated * (true_max - true_min) + true_min
            fused_calibrated = float(np.clip(fused_calibrated, true_min, true_max))
    
            # Final month estimate 
            month = self._estimate_month_from_ndvi(fused_calibrated)
            logger.info(f"Image analysis: raw_vNDVI={mean_vndvi:.3f}, clipped={mean_vndvi_clipped:.3f} "
                        f"→ calibrated_NDVI={fused_calibrated:.3f} → Month {month}")
    
            return month, fused_calibrated
        except Exception as e:
            logger.error(f"Error in NDVI estimation: {e}")
            return 6, 0.5

    
    def estimate_month_and_ndvi_ensemble(self, image, species=None):
        """
        RGB-based NDVI ensemble estimator with calibration.
    
        If `species` is provided and `/kaggle/working/outputs/ndvidata.csv` exists,
        calibration uses that species' ndvi_min/ndvi_max (and ndvi_mean) for mapping.
        Otherwise fall back to the old test.csv heuristics and defaults.
        Returns (month:int, calibrated_ndvi:float)
        """
        try:
            import PIL.Image as PILImage   
            # --- Normalize / ensure numpy array ---
            img_np = np.asarray(image, dtype=np.float32)
            # build a PIL copy for sentinel check if needed
            if not isinstance(image, PILImage.Image):
                try:
                    if img_np.max() <= 1.0:
                        img_pil_for_check = PILImage.fromarray((img_np * 255).astype('uint8'))
                    else:
                        img_pil_for_check = PILImage.fromarray((img_np).astype('uint8'))
                except Exception:
                    try:
                        img_pil_for_check = PILImage.fromarray(np.uint8(image))
                    except Exception:
                        img_pil_for_check = None
            else:
                img_pil_for_check = image
            if self._is_sentinel_image(img_pil_for_check):
                month = self._estimate_month_from_ndvi(0.0) if hasattr(self, "_estimate_month_from_ndvi") else 6
                return month, 0.0
            # scale to [0,1]
            if img_np.max() > 1.0:
                img_np = img_np / 255.0
            MAX_NDVI_SIZE = 512
            if img_np.ndim == 3:
                h, w = img_np.shape[:2]
                max_side = max(h, w)
                if max_side > MAX_NDVI_SIZE:
                    scale = MAX_NDVI_SIZE / float(max_side)
                    new_w, new_h = int(round(w * scale)), int(round(h * scale))
                    # use cv2 if available for speed; fall back to PIL
                    try:
                        img_small = cv2.resize(img_np, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
                    except Exception:
                        from PIL import Image
                        img_pil = Image.fromarray((img_np * 255).astype('uint8'))
                        img_pil = img_pil.resize((new_w, new_h), Image.BILINEAR)
                        img_small = np.asarray(img_pil, dtype=np.float32) / 255.0
                    img_np = img_small
    
            # Basic shape check
            if img_np.ndim != 3 or img_np.shape[2] < 3:
                logger.warning("estimate_month_and_ndvi_ensemble: unexpected image shape")
                return 6, 0.5

            R = img_np[:, :, 0]
            G = img_np[:, :, 1]
            B = img_np[:, :, 2]
            
            eps = 1e-6  
            
            # ---- vNDVI 
            vndvi = (G - R) / (G + R + eps)
            vndvi = np.clip(vndvi, -1.0, 1.0)
            vndvi_mean = float(np.nanmean(vndvi))
            
            # ---- RGBVI 
            rgbvi = (G*G - R*B) / (G*G + R*B + eps)
            rgbvi = np.clip(rgbvi, -1.0, 1.0)
            rgbvi_mean = float(np.nanmean(rgbvi))
            rgbvi_mean = max(rgbvi_mean, 0.0)  
            
            # ---- VARI
            den = (G + R - B)
            vari = (G - R) / (den + eps)
            vari = np.clip(vari, -1.0, 1.0)
            vari_mean = float(np.nanmean(vari))
            vari_mean = max(vari_mean, 0.0)  
            
            texture = float(np.std(G) / (np.mean(G) + eps))
            texture = np.clip(texture, 0.0, 1.0)
            
            # STEP 3 — Adaptive Fusion (raw RGB NDVI) 
            green_intensity = float(np.nanmean(G))
            
            if green_intensity < 0.30:                      
                fused_raw = (
                    0.25 * vndvi_mean +
                    0.25 * rgbvi_mean +
                    0.35 * vari_mean +
                    0.15 * texture
                )
            
            elif green_intensity > 0.65:                   
                fused_raw = (
                    0.40 * vndvi_mean +
                    0.30 * rgbvi_mean +
                    0.15 * vari_mean +
                    0.15 * texture
                )
            
            else:                                        
                fused_raw = (
                    0.33 * vndvi_mean +
                    0.33 * rgbvi_mean +
                    0.22 * vari_mean +
                    0.12 * texture
                )
            
            fused_raw = float(np.clip(fused_raw, 0.0, 1.0))
    
            #  STEP 4 — Load Calibration Stats (prefer species table) 
            default_true_min = 0.16
            default_true_max = 0.91
    
            # typical observed range of raw fused RGB index (tunable)
            rgb_raw_min = 0.0
            rgb_raw_max = 0.80
    
            true_min, true_max = default_true_min, default_true_max
    
            # 1) If species provided, try per-species NDVI table first
            try:
                if species is not None:
                    ndvi_table_path = "/kaggle/working/outputs/ndvidata.csv" 
                    if hasattr(self, "_ndvi_table") and getattr(self, "_ndvi_table") is not None:
                        ndvi_df = self._ndvi_table
                    else:
                        if pd.io.common.file_exists(ndvi_table_path):
                            ndvi_df = pd.read_csv(ndvi_table_path)
                            self._ndvi_table = ndvi_df
                        else:
                            ndvi_df = None
    
                    if ndvi_df is not None and 'species' in ndvi_df.columns:
                        # match species 
                        mask = ndvi_df['species'].astype(str).str.lower() == str(species).lower()
                        if mask.any():
                            row = ndvi_df[mask].iloc[0]
                            # prefer ndvi_min/ndvi_max or ndvi_raw_min/raw_max if present:
                            if 'ndvi_min' in row.index and 'ndvi_max' in row.index:
                                candidate_min = float(row.get('ndvi_min', np.nan))
                                candidate_max = float(row.get('ndvi_max', np.nan))
                            elif 'min' in row.index and 'max' in row.index:
                                candidate_min = float(row.get('min', np.nan))
                                candidate_max = float(row.get('max', np.nan))
                            else:
                                candidate_min = float(row.get('ndvi_mean', np.nan)) - 0.15
                                candidate_max = float(row.get('ndvi_mean', np.nan)) + 0.15
    
                            if np.isfinite(candidate_min) and np.isfinite(candidate_max) and candidate_max > candidate_min:
                                true_min, true_max = float(candidate_min), float(candidate_max)
                                logger.debug(f"Calibration using species '{species}' stats: min={true_min:.3f}, max={true_max:.3f}")
                            else:
                                logger.info(f"Species stats found but invalid for '{species}', falling back.")
            except Exception as e:
                logger.warning(f"Failed to use per-species ndvi table for calibration: {e}")
    
            # 2) If species not used or failed, try the old test.csv calibration (as fallback)
            if (true_max <= true_min) or (true_min == default_true_min and true_max == default_true_max):
                try:
                    calib_path = "/kaggle/working/outputs/ndvidata.csv" 
                    if pd.io.common.file_exists(calib_path):
                        df_cal = pd.read_csv(calib_path)
                        cols = set(df_cal.columns.str.lower())
                        # support multiple naming conventions
                        if {'avg', 'min', 'max'}.issubset(cols):
                            true_min = float(df_cal['min'].dropna().iloc[0])
                            true_max = float(df_cal['max'].dropna().iloc[0])
                        elif {'ndvi_avg', 'ndvi_min', 'ndvi_max'}.issubset(cols):
                            true_min = float(df_cal['ndvi_min'].dropna().iloc[0])
                            true_max = float(df_cal['ndvi_max'].dropna().iloc[0])
                        elif {'avg_ndvi', 'min_ndvi', 'max_ndvi'}.issubset(cols):
                            true_min = float(df_cal['min_ndvi'].dropna().iloc[0])
                            true_max = float(df_cal['max_ndvi'].dropna().iloc[0])
                        else:
                            # try first numeric min/max-looking columns
                            for c in df_cal.columns:
                                if 'min' in c.lower():
                                    true_min = float(df_cal[c].dropna().iloc[0])
                                if 'max' in c.lower():
                                    true_max = float(df_cal[c].dropna().iloc[0])
                        logger.debug(f"Calibration from {calib_path}: min={true_min:.3f}, max={true_max:.3f}")
                except Exception as e:
                    logger.info(f"No calibration csv usable or failed to parse: {e}")
    
            # final safety checks & fallback
            if not np.isfinite(true_min) or not np.isfinite(true_max) or true_max <= true_min:
                true_min, true_max = default_true_min, default_true_max
                logger.info(f"Using default calibration min={true_min}, max={true_max}")
    
            #  STEP 5 — Linear Calibration (map fused_raw -> true_min..true_max)
            denom_scale = (rgb_raw_max - rgb_raw_min) if (rgb_raw_max - rgb_raw_min) != 0 else 1e-8
            fused_calibrated = (fused_raw - rgb_raw_min) / denom_scale
            fused_calibrated = fused_calibrated * (true_max - true_min) + true_min
            fused_calibrated = float(np.clip(fused_calibrated, true_min, true_max))
    
            # STEP 6 — Compute Month and return 
            month = self._estimate_month_from_ndvi(fused_calibrated)
    
            logger.debug(
                f"RGB Ensemble: vNDVI={vndvi_mean:.3f}, RGBVI={rgbvi_mean:.3f}, VARI={vari_mean:.3f}, "
                f"FusedRaw={fused_raw:.3f} → Calibrated={fused_calibrated:.3f}, Month={month}")
            return month, fused_calibrated
    
        except Exception as e:
            logger.exception(f"Ensemble RGB NDVI calculation failed: {e}")
            return 6, 0.5

    
    def _estimate_month_from_ndvi(self, ndvi):
        """
        Estimate month from NDVI using data-driven approach.
        """
        try:
            if not hasattr(self, '_month_profiles'):
                self._build_month_profiles()
    
            if not self._month_profiles:
                logger.warning("No month profiles available, returning default month 6")
                return 6
    
            best_month = 6
            best_score = -1.0
    
            for month, profile in self._month_profiles.items():
                # Use safe std
                profile_std = float(profile.get('std', 0.05))
                profile_mean = float(profile.get('mean', 0.5))
                # z-score
                z_score = (ndvi - profile_mean) / (profile_std + 1e-8)
                likelihood = math.exp(-0.5 * (z_score * z_score))
                frequency_weight = profile.get('count', 1) / max(p['count'] for p in self._month_profiles.values())
                score = likelihood # *frequency_weight removed to prevent bias
                if score > best_score:
                    best_score = score
                    best_month = int(month)
    
            return best_month
    
        except Exception as e:
            logger.error(f"Error estimating month: {e}")
            return 6

    
    def _build_month_profiles(self):
        """
        Build NDVI statistical profiles for each month from training data.
        """
        try:
            if self.train_df is None:
                logger.warning("train_df not available, cannot build month profiles")
                self._month_profiles = {}
                return
    
            train_df_copy = self.train_df.copy()
            train_df_copy['Month'] = pd.to_datetime(train_df_copy['Sampling_Date'], errors='coerce').dt.month
    
            self._month_profiles = {}
            for month in range(1, 13):
                month_data = train_df_copy[train_df_copy['Month'] == month]['Pre_GSHH_NDVI'].dropna().astype(float)
                if len(month_data) > 0:
                    mean_val = float(month_data.mean())
                    std_val = float(month_data.std()) if month_data.std() > 0 else 0.05
                    std_val = max(std_val, 0.05)  # floor
                    self._month_profiles[month] = {
                        'mean': mean_val,
                        'std': std_val,
                        'count': int(len(month_data)),
                    }
    
            logger.debug(f"✓ Built month profiles for {len(self._month_profiles)} months")

        except Exception as e:
            logger.error(f"Error building month profiles: {e}")
            self._month_profiles = {}

    def _intelligent_feature_fetch(self, test_row, species_model=None, prediction=None, device=None):
        """
        Intelligent fallback with image-level caching to avoid recomputing NDVI/height/species 
        for rows that share the same image_id / sample_id.
        """
        sample_id = test_row.get("sample_id")
        target_name = test_row.get("target_name")
    
        # Normalize keys to match lookups built with .strip().lower()
        target_name_norm = str(target_name).strip().lower() if target_name is not None else ""
        sample_id_norm = str(sample_id).strip().lower() if sample_id is not None else ""
        image_id = sample_id_norm.split("__")[0] if sample_id_norm else None
    
        # initialize lookups safely
        TRAIN_BY_SAMPLE_ID_TARGET = self.lookups_dict.get('TRAIN_BY_SAMPLE_ID_TARGET', {})
        TRAIN_BY_IMAGE_SPECIES_TARGET = self.lookups_dict.get('TRAIN_BY_IMAGE_SPECIES_TARGET', {})
        TRAIN_BY_IMAGE_ID_TARGET = self.lookups_dict.get('TRAIN_BY_IMAGE_ID_TARGET', {})
        TRAIN_BY_TARGET_STATE_SPECIES = self.lookups_dict.get('TRAIN_BY_TARGET_STATE_SPECIES', {})
        
        # Diagnostics 
        lvl1_count = 0
        if target_name_norm in TRAIN_BY_SAMPLE_ID_TARGET:
            try:
                lvl1_count = sum(len(v) for v in TRAIN_BY_SAMPLE_ID_TARGET.get(target_name_norm, {}).values())
            except Exception:
                lvl1_count = len(TRAIN_BY_SAMPLE_ID_TARGET.get(target_name_norm, {}))
        lvl2_image_count = len(TRAIN_BY_IMAGE_ID_TARGET.get(target_name_norm, {}))
        lvl2_species_count = len(TRAIN_BY_IMAGE_SPECIES_TARGET.get(target_name_norm, {}))
        lvl3_count = len(TRAIN_BY_TARGET_STATE_SPECIES.get(target_name_norm, {}))
        logger.debug(f"_intelligent_feature_fetch: target_name_norm='{target_name_norm}', image_id='{image_id}'")
        logger.debug(f"Lookup sizes: level1={lvl1_count}, level2_image={lvl2_image_count}, level2_species={lvl2_species_count}, level3={lvl3_count}")
    
        # IMAGE-LEVEL CACHE (keyed by image_id if available, else sample_id_norm) 
        cache_key = image_id or sample_id_norm or test_row.get('image_path', '')
        if not hasattr(self, 'test_features_cache') or self.test_features_cache is None:
            # ensure cache exists
            self.test_features_cache = {}
    
        cached = self.test_features_cache.get(cache_key, None)
    
        # If we have a cached computed-image object, reuse it
        if cached is None:
            # compute and store
            img_path = self._resolve_image_path(test_row.get('image_path', ''))
            test_image = self._load_image(img_path)
            test_image_np = np.array(test_image)
    
            # expensive computations we want to cache
            try:
                estimated_height = self.estimate_height_final(test_image_np)
            except Exception as e:
                logger.debug(f"estimate_height_final failed: {e}")
                estimated_height = 7.5
    
            try:
                estimated_month, estimated_ndvi = self.estimate_month_and_ndvi_ensemble(test_image_np, species=prediction)
            except Exception as e:
                logger.debug(f"estimate_month_and_ndvi_from_image failed: {e}")
                try:
                    estimated_month, estimated_ndvi =  self.estimate_month_and_ndvi_from_image(test_image_np)
                except Exception:
                    estimated_month, estimated_ndvi = 6, 0.5

            MONTH_TO_DOY = {
                        1: 16, 2: 47, 3: 75, 4: 106, 5: 136, 6: 167,
                        7: 197, 8: 228, 9: 259, 10: 289, 11: 320, 12: 350}
            estimated_month = int(estimated_month)
            estimated_doy = MONTH_TO_DOY.get(estimated_month, 180)
            estimated_quarter = max(1, (estimated_month - 1) // 3 + 1)
    
            # veg_frac computed by compute_features_from_image 
            try:
                feats = self.compute_features_from_image(test_image)
                veg_frac = float(np.clip(feats.get('veg_frac', 0.5), 0.0, 1.0))
            except Exception:
                veg_frac = 0.5
    
            # species prediction: allow a pre-supplied 'prediction' to override; otherwise compute once
            predicted_species = prediction
            if (predicted_species is None) and (species_model is not None):
                try:
                    # predicting_species will do patching and model inference; expensive so we cache result
                    predicted_species = self.predicting_species(test_row, species_model=species_model, device=device)
                except Exception as e:
                    logger.debug(f"predicting_species failed: {e}")
                    predicted_species = None
    
            # normalize some cached values
            def _norm_species_name(s):
                if s is None:
                    return None
                s2 = str(s).strip().lower()
                s2 = re.sub(r"[\s\-]+", "_", s2)
                s2 = s2.replace('.', '')
                return s2
    
            predicted_species_norm = _norm_species_name(predicted_species)
    
            cached = {
                'image_path': str(img_path),
                'ndvi': float(np.clip(estimated_ndvi, 0.0, 1.0)),
                'month': int(estimated_month),
                'doy': int(estimated_doy),
                'quarter': int(estimated_quarter),
                'estimated_height': float(estimated_height),
                'veg_frac': float(veg_frac),
                'predicted_species': predicted_species,
                'predicted_species_norm': predicted_species_norm
            }
            cached['state'] = None
            # store in cache
            try:
                self.test_features_cache[cache_key] = cached
            except Exception:
                # if cache write fails for any reason, ignore — caching is optional
                logger.debug("Warning: failed to write to test_features_cache")
    
        else:
            # update with any new 'prediction' passed in (higher-level caller may have predicted species already)
            if prediction is not None:
                cached['predicted_species'] = prediction
                # also normalized form
                try:
                    s = prediction
                    s2 = str(s).strip().lower()
                    s2 = re.sub(r"[\s\-]+", "_", s2)
                    s2 = s2.replace('.', '')
                    cached['predicted_species_norm'] = s2
                except Exception:
                    pass
    
        # Now reuse cached values for fallback decisions
        estimated_ndvi = float(np.clip(cached.get('ndvi', 0.5), 0.0, 1.0))
        estimated_month = int(cached.get('month', 6))
        estimated_doy = int(cached.get('doy', int(estimated_month * 30.4)))
        estimated_quarter = int(cached.get('quarter', max(1, (estimated_month - 1) // 3 + 1)))
        estimated_height = float(cached.get('estimated_height', 7.6))
        veg_frac = float(cached.get('veg_frac', 0.5))
        predicted_species = cached.get('predicted_species', None)
        predicted_species_norm = cached.get('predicted_species_norm', None)
    
        # If predictor returned a dict, extract the 'pred' field
        if isinstance(predicted_species, dict):
            predicted_species = predicted_species.get('pred', None)
    
        # convenience normalizer used by lookup matching later
        def _norm_species_name(s):
            if s is None:
                return None
            s2 = str(s).strip().lower()
            s2 = re.sub(r"[\s\-]+", "_", s2)
            s2 = s2.replace('.', '')
            return s2
    
        REVERSE_STATE_MAP = {'tas': 'Tas', 'nsw': 'NSW', 'wa': 'WA', 'vic': 'Vic'} 
        predicted_species_norm = predicted_species_norm or _norm_species_name(predicted_species)
        logger.debug(f"Cached image-level features used: ndvi={estimated_ndvi:.3f}, month={estimated_month}, height={estimated_height:.2f}, species_norm={predicted_species_norm}")
        cached_state = cached.get('state', None)
        # LEVEL 1: state + species
        try:
            level3 = TRAIN_BY_TARGET_STATE_SPECIES.get(target_name_norm, {})
            if not level3:
                logger.info(f"LEVEL 3 empty for target '{target_name_norm}' (no TRAIN_BY_TARGET_STATE_SPECIES entries)")
            else:
                if predicted_species_norm:
                    for state, species_dict in level3.items():
                        if predicted_species_norm in species_dict:
                            # use cached image-level estimates for ndvi/height/month
                            row = random.choice(species_dict[predicted_species_norm])
                            state_counts = {'tas': 690, 'vic': 560, 'nsw': 375, 'wa': 160}
                            state0 = state  
                            states = np.array(list(state_counts.keys()), dtype=object)
                            counts = np.array(list(state_counts.values()), dtype=float)
                            total = counts.sum()
                            emp_probs = counts / total
                            state_frequency = state_counts[state0] / total
                            alpha = 0.3 if state_frequency < 0.25 else 0.5
                            prior = np.zeros_like(emp_probs)
                            prior[states == state0] = 1.0
                            final_probs = alpha * prior + (1.0 - alpha) * emp_probs
                            final_probs = final_probs / final_probs.sum() 
                            state_new = np.random.choice(states, p=final_probs)
                            state_final = cached_state.lower() if cached_state is not None else state_new 
                            fetched = {
                                'ndvi': estimated_ndvi,
                                'height': estimated_height,
                                'month': estimated_month,
                                'doy': estimated_doy,
                                'quarter': estimated_quarter,
                                'state': REVERSE_STATE_MAP.get(state_final, state),
                                'species': predicted_species,
                                'level': 1,
                                'source': f"State+Species: {state_new}-{predicted_species_norm}",
                                'species_source': 'predicted'
                            }
                            logger.debug("Level 1a Fallback Used (The Best)")
                            # cache the state for future rows of the same image
                            try:
                                state_lower = str(fetched['state']).strip().lower()
                                self.test_features_cache[cache_key]['state'] = state_lower
                            except:
                                pass
                            logger.debug(
                                    f"Prediction complete | state={fetched['state']} | "
                                    f"species={fetched['species']} | ndvi={fetched['ndvi']:.3f} | "
                                    f"height={fetched['height']:.2f} | source={fetched['source']}"
                                    )
                            return fetched
    
                    # try normalized key matching inside species_dict
                    for state, species_dict in level3.items():
                        for sp_k in species_dict.keys():
                            if _norm_species_name(sp_k) == predicted_species_norm:
                                row = random.choice(species_dict[sp_k])
                                fetched = {
                                    'ndvi': row['Pre_GSHH_NDVI'],
                                    'height': row['Height_Ave_cm'],
                                    'month': row['Month'],
                                    'doy': row['DayOfYear'],
                                    'quarter': row['Quarter'],
                                    'state': state,
                                    'species': row.get('Species', sp_k),
                                    'level': 1,
                                    'source': f"State+Species (norm-match): {state}-{sp_k}",
                                    'species_source': 'predicted'
                                }
                                logger.info("Level 1a v2 Fallback Used")
                                return fetched
    
                # fallback random pick from level1 if any entries exist
                if level3:
                    state = random.choice(list(level3.keys()))
                    species = random.choice(list(level3[state].keys()))
                    row = random.choice(level3[state][species])
                    fetched = {
                        'ndvi': row['Pre_GSHH_NDVI'],
                        'height': row['Height_Ave_cm'],
                        'month': row['Month'],
                        'doy': row['DayOfYear'],
                        'quarter': row['Quarter'],
                        'state': state,
                        'species': species,
                        'level': 1,
                        'source': f"State+Species: {state}-{species}",
                        'species_source': 'training_data'
                    }
                    logger.info("Level 1b Fallback Used")
                    return fetched
        except Exception as e:
            logger.debug(f"LEVEL 1 failed: {e}")
    
        # LEVEL 2: sample_id + target
        try:
            level1 = TRAIN_BY_SAMPLE_ID_TARGET.get(target_name_norm, {})
            if sample_id_norm and sample_id_norm in level1:
                row = random.choice(level1[sample_id_norm])
                fetched = {
                    'ndvi': row['Pre_GSHH_NDVI'],
                    'height': row['Height_Ave_cm'],
                    'month': row['Month'],
                    'doy': row['DayOfYear'],
                    'quarter': row['Quarter'],
                    'state': row['State'],
                    'species': row['Species'],
                    'level': 2,
                    'source': f"Exact: {sample_id_norm}",
                    'species_source': 'training_data'
                }
                logger.info("Level 2 Fallback Used")
                return fetched
        except Exception as e:
            logger.debug(f"LEVEL 2 failed: {e}")
    
        # LEVEL 3: image_id + species
        try:
            level2_species_map = TRAIN_BY_IMAGE_SPECIES_TARGET.get(target_name_norm, {})
            level2_image_map = TRAIN_BY_IMAGE_ID_TARGET.get(target_name_norm, {})
    
            # 2a: image + species
            if image_id and predicted_species_norm:
                # try tuple-key first
                key_candidates = [
                    (image_id, predicted_species_norm),
                    (image_id, predicted_species_norm.replace('_', ' ')),
                    (image_id, predicted_species_norm.replace('_', ''))
                ]
                rows = []
                for key in key_candidates:
                    rows = level2_species_map.get(key, [])
                    if rows:
                        break
    
                # if not found, try scanning level2 map for matching image_id and normalized species
                if not rows:
                    for kk, vals in list(level2_species_map.items()):
                        try:
                            ik, sk = kk
                            if str(ik) == image_id and _norm_species_name(sk) == predicted_species_norm:
                                rows = vals
                                break
                        except Exception:
                            # could be string key; try to match roughly
                            if isinstance(kk, str) and image_id in kk and predicted_species_norm in kk:
                                rows = vals
                                break
    
                if rows:
                    row = random.choice(rows)
                    fetched = {
                        'ndvi': row['Pre_GSHH_NDVI'],
                        'height': row['Height_Ave_cm'],
                        'month': row['Month'],
                        'doy': row['DayOfYear'],
                        'quarter': row['Quarter'],
                        'state': row['State'],
                        'species': row['Species'],
                        'level': 3,
                        'source': f"Image {image_id} + Predicted Species",
                        'species_source': 'predicted'
                    }
                    logger.info("Level 3a Fallback Used")
                    return fetched
    
            # 3b: image only
            if image_id and image_id in level2_image_map:
                row = random.choice(level2_image_map[image_id])
                fetched = {
                    'ndvi': row['Pre_GSHH_NDVI'],
                    'height': row['Height_Ave_cm'],
                    'month': row['Month'],
                    'doy': row['DayOfYear'],
                    'quarter': row['Quarter'],
                    'state': row['State'],
                    'species': row['Species'],
                    'level': 3,
                    'source': f"Image {image_id}",
                    'species_source': 'training_data'
                }
                logger.info("Level 3b Fallback Used")
                return fetched
        except Exception as e:
            logger.debug(f"LEVEL 3 failed: {e}")
    
        # LEVEL 4: target statistics
        try:
            stats = self.lookups_dict.get('TABULAR_STATS_BY_TARGET', {}).get(target_name_norm,
                                                                            self.lookups_dict.get('TABULAR_STATS', {}))
            ndvi = np.clip(np.random.normal(stats.get('ndvi_mean', estimated_ndvi if 'estimated_ndvi' in locals() else 0.65),
                                           stats.get('ndvi_std', 0.15)), 0.0, 1.0)
            HEIGHT_MAX = getattr(self, "HEIGHT_MAX", self.lookups_dict.get("HEIGHT_MAX", 80.0))
            height = np.clip(np.random.normal(stats.get('height_mean', estimated_height if 'estimated_height' in locals() else 7.6),
                                              stats.get('height_std', 10.3)), 0.0, HEIGHT_MAX)
            doy = int(stats.get('sampling_date_mean', estimated_doy if 'estimated_doy' in locals() else 197))
            month = max(1, (doy - 1) // 30 + 1)
            quarter = max(1, (month - 1) // 3 + 1)
            unique_states = self.lookups_dict.get('UNIQUE_STATES', ['NSW'])
            state = np.random.choice(unique_states)
            species = predicted_species_norm or np.random.choice(self.lookups_dict.get('UNIQUE_SPECIES', ['clover']))
            species_source = 'predicted' if predicted_species_norm else 'training_data'
            logger.info("Level 4 Fallback Used")
            return {
                'ndvi': float(ndvi), 'height': float(height),
                'month': int(month), 'doy': int(doy), 'quarter': int(quarter),
                'state': state, 'species': species,
                'level': 4, 'source': f"Target stats {target_name_norm}",
                'species_source': species_source
            }
        except Exception as e:
            logger.debug(f"LEVEL 4 failed: {e}")
    
        # default fallback
        logger.info("Default Fallback is Being Used.")
        return self._get_default_features(test_row)



    def _load_image(self, image_path):
        try:
            p = self._resolve_image_path(image_path)

            # Missing file → sentinel image
            if not p.exists():
                self._missing_count = getattr(self, '_missing_count', 0) + 1
                if self._missing_count <= 10:
                    logger.warning(f"Image missing {p} – using SENTINEL MAGENTA (#{self._missing_count})")
                return Image.new('RGB', (self.image_width, self.image_height), color=self.SENTINEL_COLOR)

            # Load image with OpenCV
            img = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
            if img is None:
                self._missing_count = getattr(self, '_missing_count', 0) + 1
                if self._missing_count <= 10:
                    logger.warning(f"Image unreadable {p} – using SENTINEL MAGENTA (#{self._missing_count})")
                return Image.new('RGB', (self.image_width, self.image_height), color=self.SENTINEL_COLOR)

            # Convert various channel formats
            if img.ndim == 2:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            elif img.shape[2] == 3:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            elif img.shape[2] == 4:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
            else:
                try:
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                except Exception:
                    return Image.new('RGB', (self.image_width, self.image_height), color=self.SENTINEL_COLOR)

            return Image.fromarray(img_rgb)

        except Exception as e:
            logger.warning(f"Error loading {image_path}: {e}")
            return Image.new('RGB', (self.image_width, self.image_height), color=self.SENTINEL_COLOR)



    def _crop_patches(self, image):
        patches = []
        w, h = image.size
        target_w, target_h = 2000, 1000
        if (w,h) != (target_w, target_h):
            image = image.resize((target_w, target_h), Image.BILINEAR)
        crop_coords = [
            (0, 0, 500, 500), (500, 0, 1000, 500),
            (1000, 0, 1500, 500), (1500, 0, 2000, 500),
            (0, 500, 500, 1000), (500, 500, 1000, 1000),
            (1000, 500, 1500, 1000), (1500, 500, 2000, 1000),
        ]
        for left, top, right, bottom in crop_coords:
            patch = F.crop(image, top=top, left=left,
                          height=bottom-top, width=right-left)
            patch = patch.resize((self.model_input_size[0], self.model_input_size[1]), Image.BILINEAR)
            patches.append(patch)
        return patches

    def _get_default_features(self, test_row):
        return {
            'month': 6,
            'doy': 180,
            'quarter': 2,
            'state': 'NSW',
            'species': 'Clover',
            'ndvi': 0.6,
            'height': 15.0,
            'level': -1,
            'source': "Hardcoded defaults (no lookups)",
        }

    def _build_features_from_fetch(self, test_row, lookups_dict):
        """
        Build complete tabular feature vector for test row.
        
        Column order MUST match training:
        [numeric_cols | state_cols | species_cols | target_type_cols]
        
        Returns: (features_tensor, target_idx)
        """
        numeric_cols = lookups_dict.get('TAB_NUMERIC_COLS', [])
        state_cols = lookups_dict.get('STATE_COLS', [])
        species_cols = lookups_dict.get('SPECIES_COLS', [])
        target_type_cols = lookups_dict.get('TARGET_TYPE_COLS', [])
        scaler = lookups_dict.get('TAB_SCALER', None)
        
        # ===== NUMERIC FEATURES =====
        raw_numeric = []
        for col in numeric_cols:
            if col == 'Pre_GSHH_NDVI':
                val = float(test_row.get('Pre_GSHH_NDVI', 0.5))
                raw_numeric.append(np.clip(val, 0.0, 1.0))
            elif col == 'Height_Ave_cm':
                val = float(test_row.get('Height_Ave_cm', 10.0))
                raw_numeric.append(np.clip(val, 0.0, 100.0))
            elif col == 'Month':
                val = float(test_row.get('Month', 6))
                raw_numeric.append(np.clip(val, 1.0, 12.0))
            elif col == 'DayOfYear':
                val = float(test_row.get('DayOfYear', 1))
                raw_numeric.append(np.clip(val, 1.0, 365.0))
            elif col == 'Quarter':
                val = float(test_row.get('Quarter', 1))
                raw_numeric.append(np.clip(val, 1.0, 4.0))
            else:
                raw_numeric.append(float(test_row.get(col, 0.0)))
        
        # Scale numeric features if scaler available
        if scaler is not None and len(raw_numeric) > 0:
            try:
                scaled_numeric = scaler.transform([raw_numeric])[0].tolist()
            except Exception as e:
                logger.warning(f"Scaler transform failed: {e}, using raw values")
                scaled_numeric = raw_numeric
        else:
            scaled_numeric = raw_numeric
        
        features = list(scaled_numeric)
        
        # ===== STATE ONE-HOT =====
        def _safe_to_float(x):
            if isinstance(x, (bool, np.bool_)):
                return 1.0 if x else 0.0
            try:
                return float(x)
            except Exception:
                return 0.0
        
        for col in state_cols:
            features.append(_safe_to_float(test_row.get(col, 0)))
        
        # ===== SPECIES ONE-HOT =====
        for col in species_cols:
            features.append(_safe_to_float(test_row.get(col, 0)))
        
        # ===== TARGET TYPE ONE-HOT =====
        target_onehot = []
        for col in target_type_cols:
            target_onehot.append(_safe_to_float(test_row.get(col, 0)))
        
        for val in target_onehot:
            features.append(val)
        
        # ===== EXTRACT TARGET_IDX (CRITICAL FOR META-LEARNER) =====
        # target_idx is the argmax of the one-hot encoding at the END
        target_idx = 0
        if len(target_onehot) > 0:
            try:
                target_idx = int(np.argmax(target_onehot))
            except Exception:
                target_idx = 0
        
        # ===== RETURN =====
        features_tensor = torch.tensor(features, dtype=torch.float32)
        
        logger.debug(f"Built features: {len(features)} dims, target_idx={target_idx}")
        logger.debug(f"  Numeric: {len(scaled_numeric)} | State: {len(state_cols)} | Species: {len(species_cols)} | Target: {len(target_onehot)}")
        
        return features_tensor, target_idx

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self._load_image(row['image_path'])
        patches = self._crop_patches(img)
    
        # build image tensor batch (8 patches)
        if self.transform:
            images = torch.stack([self.transform(p) for p in patches])
        else:
            images = torch.stack([transforms.ToTensor()(p) for p in patches])
    
        # features: test uses intelligent fetch, train/val uses _extract_train_tabular
        target_idx = 0  # NEW: Initialize target_idx
        
        if self.mode == 'test':
            prediction = self.predicting_species(row, species_model=self.species_model, device=self.device)
            fetched = self._intelligent_feature_fetch(
                row, species_model=self.species_model, prediction=prediction, device=self.device)
            if fetched is None:
                fetched = self._get_default_features(row)
            
            # NEW: Get both features AND target_idx
            features_tensor, target_idx = self._build_features_from_fetch(fetched, self.lookups_dict)
            features_1d = features_tensor.numpy()
        else:
            features_1d = self._extract_train_tabular(row).numpy()
            
            # EXTRACT target_idx from training row (from one-hot encoding)
            target_type_cols = self.lookups_dict.get('TARGET_TYPE_COLS', [])
            numeric_cols = self.lookups_dict.get('TAB_NUMERIC_COLS', [])
            state_cols = self.lookups_dict.get('STATE_COLS', [])
            species_cols = self.lookups_dict.get('SPECIES_COLS', [])
            
            # Position where target one-hot starts
            target_start = len(numeric_cols) + len(state_cols) + len(species_cols)
            target_end = target_start + len(target_type_cols)
            
            if target_end <= len(features_1d):
                target_onehot = features_1d[target_start:target_end]
                try:
                    target_idx = int(np.argmax(target_onehot))
                except Exception:
                    target_idx = 0
            else:
                target_idx = 0
    
        # repeat per patch
        features = np.repeat(features_1d[np.newaxis, :], self.num_patches, axis=0).astype(np.float32)
        features = torch.from_numpy(features).float()
    
        sample_id = row.get('sample_id', None)
        target_name = row.get('target_name', None)
    
        species_idx = None
    
        def _norm_species_name_val(s):
            if s is None:
                return "mixed"
            s2 = str(s).strip().lower()
            s2 = re.sub(r"[\s\-]+", "_", s2)
            s2 = s2.replace(".", "")
            # treat empty-like values as "nothing"
            if s2 == "" or s2 in {"nan", "none", "null"}:
                return "mixed"
            return s2
    
        fixed_species_norm = None
        try:
            scols = self.lookups_dict.get("SPECIES_COLS", None)
            if scols:
                fixed_species_norm = [
                    _norm_species_name_val(c.replace("Species_", "")) if isinstance(c, str) else _norm_species_name_val(c)
                    for c in scols
                ]
        except Exception:
            fixed_species_norm = None
    
        if (not fixed_species_norm) and ('FIXED_SPECIES_LIST' in globals()):
            fixed_species_norm = [
                _norm_species_name_val(s) for s in globals().get('FIXED_SPECIES_LIST', [])
            ]
        if not fixed_species_norm:
            try:
                if getattr(self, "train_df", None) is not None and 'Species' in self.train_df.columns:
                    tmp_list = self.train_df['Species'].fillna('').astype(str).apply(_norm_species_name_val).unique().tolist()
                    fixed_species_norm = list(dict.fromkeys(tmp_list))
                else:
                    fixed_species_norm = ["nothing"]
            except Exception:
                fixed_species_norm = ["nothing"]
        if "nothing" not in fixed_species_norm:
            fixed_species_norm = fixed_species_norm + ["nothing"]
    
        if not hasattr(self, "species_to_idx") or not isinstance(self.species_to_idx, dict) or len(self.species_to_idx) != len(fixed_species_norm):
            self.species_to_idx = {s: i for i, s in enumerate(fixed_species_norm)}
            # also store reverse for convenience
            self.idx_to_species = {i: s for s, i in self.species_to_idx.items()}
    
        species_raw = None
        if 'Species_canon' in row.index and pd.notna(row.get('Species_canon', None)) and str(row.get('Species_canon')).strip() != "":
            species_raw = row.get('Species_canon')
        else:
            species_raw = row.get('Species', None)
    
        species_norm = _norm_species_name_val(species_raw)
    
        if species_norm not in self.species_to_idx:
            logger.debug(f"Species '{species_raw}' normalized -> '{species_norm}' not in fixed vocab; mapping to 'nothing'")
            species_norm = "mixed"
        species_idx = int(self.species_to_idx.get(species_norm, self.species_to_idx.get("mixed", 0)))
    
        # FINAL RETURNS - NOW INCLUDES target_idx
        if self.mode == 'test':
            return images, features, species_idx, target_idx, None, sample_id, target_name
        else:
            target = float(row['target'])
            return images, features, species_idx, target_idx, target, sample_id, target_name


    def calibrate_height_model(
        self,
        image_col='image_path',
        height_col='Height_Ave_cm',
        test_size=0.2,
        random_state=42,
        min_positive_samples=5,
        fit_ridge=True,
        ridge_alphas=(0.01, 0.1, 1.0, 10.0),
        verbose=True
    ):
        """
        Calibrate height estimator from training images:
          - compute features and combined_score using self.compute_features_from_image(image)
          - fit power-law (log-linear): log(h) = log(a) + b * log(score) -> saves self.height_a, self.height_b
          - optionally fit  to predict log(height) from features -> saves self.height_model and self.height_model_type='log'
          - evaluate on a holdout set and return metrics & trained models.
    
        Returns:
            dict with:
              - 'n_samples', 'n_positive', 'powerlaw': {'a','b','rmse','mae','r2'}, 
                'ridge': {model, 'rmse','mae','r2'} (if fit_ridge True), 
              - 'df_features' (pandas DataFrame used for fitting)
        """
    
        # defensive checks
        if not hasattr(self, 'train_df') or self.train_df is None:
            raise ValueError("train_df not found on self. Populate self.train_df first.")
    
        df = self.train_df.copy()
    
        # find image column
        if image_col not in df.columns:
            # try common alternatives
            if 'image_id' in df.columns and 'image_path' in df.columns:
                image_col = 'image_path' if 'image_path' in df.columns else 'image_id'
            elif 'image_path' in df.columns:
                image_col = 'image_path'
            else:
                raise ValueError(f"Image column '{image_col}' not found in train_df")

        # collect features
        features_rows = []
        total_rows = len(df)
        iterator = range(total_rows)
        if verbose:
            iterator = tqdm(iterator, desc="Extract features", unit="rows")
    
        for i in iterator:
            r = df.iloc[i]
            img_identifier = r.get(image_col, None)
            img = None
    
            if pd.isna(img_identifier) or img_identifier is None:
                img_identifier = None
    
            try:
              
                if isinstance(img_identifier, str) and img_identifier.strip() != "":
                    img_id = img_identifier.strip()
                    # Remove leading 'train/' or 'test/'
                    if img_id.startswith("train/"):
                        img_id = img_id[len("train/"):]
                    elif img_id.startswith("test/"):
                        img_id = img_id[len("test/"):]
                    
                    # Build correct absolute path
                    p = Path(self.image_dir) / img_id
    
                    try:
                        with Image.open(str(p)) as _img:
                            img = _img.convert("RGB").copy()
                    except Exception:
                        try:
                            arr = cv2.imread(str(p))
                            if arr is not None:
                                arr_rgb = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
                                img = Image.fromarray(arr_rgb)
                            else:
                                img = None
                        except Exception:
                            img = None
    
                elif isinstance(img_identifier, (np.ndarray,)):
                    try:
                        img = Image.fromarray(np.uint8(img_identifier)).convert("RGB")
                    except Exception:
                        img = None
    

                elif isinstance(img_identifier, Image.Image):
                    img = img_identifier.convert("RGB")
    
            except Exception:
                img = None
    
            if img is None:
                try:
                    img = Image.new("RGB", (self.image_width, self.image_height), color=(255, 0, 255))
                except Exception:
                    img = Image.new("RGB", (200, 200), color=(255, 0, 255))

            #=========================================
            #Remove Later to use Full size Image
            #========================================
            # Resize image for consistent calibration 
            MAX_SIZE = 512
            w, h = img.size
            scale = MAX_SIZE / max(w, h)
            new_w, new_h = int(w * scale), int(h * scale)
            img = img.resize((new_w, new_h), Image.BILINEAR)
            #====================================================
            try:
                feats = self.compute_features_from_image(img)
            except Exception:
                feats = {'ndvi': 0.5, 'texture': 0.02, 'green': 0.4, 'brightness': 0.5, 'height_proxy': 0.4, 'veg_frac': 0.5}
    
            combined_score = (
                0.35 * feats['ndvi'] +
                0.25 * feats['texture'] +
                0.20 * feats['green'] +
                0.10 * feats['brightness'] +
                0.10 * feats['height_proxy']
            )
            MIN_SCORE = 1e-2   
            combined_score = float(np.clip(combined_score * (0.5 + 0.5 * feats.get('veg_frac', 1.0)), MIN_SCORE, 1.0))
    
            features_rows.append({
                'index': i,
                'image_id': img_identifier if isinstance(img_identifier, str) else f"img_{i}",
                'ndvi': float(feats['ndvi']),
                'texture': float(feats['texture']),
                'green': float(feats['green']),
                'brightness': float(feats['brightness']),
                'height_proxy': float(feats['height_proxy']),
                'veg_frac': float(feats.get('veg_frac', 0.5)),
                'combined_score': combined_score,
                'height_cm_true': float(r[height_col]) if (height_col in df.columns and not pd.isna(r[height_col])) else np.nan
            })
    
    
        df_features = pd.DataFrame(features_rows)
        cs = df_features['combined_score'].astype(float)
        cs_norm = (cs - cs.min()) / (cs.max() - cs.min() + 1e-6)
        cs_norm = cs_norm.clip(1e-3, 1.0)
        
        df_features['combined_score_norm'] = cs_norm
        feat_cols = ['ndvi','texture','green','brightness','height_proxy','veg_frac']
        for c in feat_cols:
            df_features[c] = pd.to_numeric(df_features[c], errors='coerce')
        medians = df_features[feat_cols].median()
        # if many rows have all NaNs, they should be dropped for calibrationX_log
        all_missing_mask = df_features[feat_cols].isna().all(axis=1)
        if all_missing_mask.sum() > 0:
            logger.info(f"Dropping {all_missing_mask.sum()} rows with all features missing for height calibration.")
        df_features.loc[:, feat_cols] = df_features.loc[:, feat_cols].fillna(medians)
        # only rows with ground truth
        mask_gt = df_features['height_cm_true'].notnull()
        n_total = len(df_features)
        n_gt = int(mask_gt.sum())
    
        results = {'n_samples': n_total, 'n_positive': n_gt, 'powerlaw': None, 'ridge': None, 'df_features': df_features}
    
        if n_gt < min_positive_samples:
            # not enough GT to fit
            logger.warning(f"Not enough ground-truth samples to fit calibration (found {n_gt}, need >= {min_positive_samples}).")
            return results
    
        # prepare arrays for fitting
        df_gt = df_features.loc[mask_gt].copy()
        # ensure positive combined_score & height
        pos_mask = (df_gt['combined_score'] > 0) & (df_gt['height_cm_true'] > 0)
        df_gt = df_gt.loc[pos_mask].copy()
        if len(df_gt) < min_positive_samples:
            logger.warning(f"After removing nonpositive samples, only {len(df_gt)} samples remain. Aborting fit.")
            return results
    
        # Fit power-law: log(height) = log(a) + b * log(score)
        X_log = np.log1p(df_gt['combined_score_norm'].values).reshape(-1, 1)
        y_log = np.log1p(df_gt['height_cm_true'].values).reshape(-1, 1)
        lr = LinearRegression().fit(X_log, y_log)
        b = float(lr.coef_[0][0])
        loga = float(lr.intercept_[0])
        a = float(np.exp(loga))
        # Save to self
        self.height_a = a
        self.height_b = b
    
        # Evaluate on holdout split for power-law
        bins = pd.qcut(df_gt['height_cm_true'], q=10, duplicates='drop')
        X_train, X_test, y_train, y_test = train_test_split(
            df_gt['combined_score_norm'].values,
            df_gt['height_cm_true'].values,
            test_size=test_size,
            random_state=random_state,
            stratify=bins
        )
        
        mask_train = (X_train > 0) & (y_train > 0)
        if mask_train.sum() >= 3:
            lr2 = LinearRegression().fit(
                np.log1p(X_train[mask_train]).reshape(-1, 1),
                np.log1p(y_train[mask_train]).reshape(-1, 1))
        
            loga_train = float(np.ravel(lr2.intercept_)[0])
            b_train = float(np.ravel(lr2.coef_)[0])
        
            X_test_pos = np.maximum(X_test, 1e-6)
            preds_power = np.expm1(loga_train + b_train * np.log1p(X_test_pos))
        
            # data-driven cap (robust fallback)
            q99 = df_gt['height_cm_true'].dropna().quantile(0.99)
        
            HEIGHT_MAX = getattr(self, "HEIGHT_MAX", self.lookups_dict.get("HEIGHT_MAX", 80.0))
            preds_power = np.clip(preds_power, 0.0, HEIGHT_MAX * 1.3)

            rmse_p = float(np.sqrt(mean_squared_error(y_test, preds_power)))
            mae_p = float(mean_absolute_error(y_test, preds_power))
            if np.allclose(y_test, y_test[0]):
                r2_p = float('nan')
            else:
                r2_p = r2_score(np.log1p(y_test), np.log1p(preds_power))
        else:
            rmse_p = mae_p = r2_p = float('nan')

        results['powerlaw'] = {'a': a, 'b': b, 'rmse': rmse_p, 'mae': mae_p, 'r2': r2_p}
    
        if fit_ridge:
            # Prepare features & log1p target
            X_feats = df_gt[['ndvi','texture','green','brightness','height_proxy','veg_frac']].values.astype(float)
            y_log_all = np.log1p(df_gt['height_cm_true'].values)
        
            # Holdout split
            X_tr, X_te, ytr, yte = train_test_split(
                X_feats, y_log_all,
                test_size=test_size,
                random_state=random_state
            )
        
            try:
                # Train tree-based model predicting log1p(height)
                ridge = RandomForestRegressor(
                    n_estimators=200,
                    max_depth=15,
                    min_samples_leaf=5,
                    min_samples_split=10,
                    random_state=42,
                    n_jobs=-1
                ).fit(X_tr, ytr)
        
                # Predict on holdout
                logpred = ridge.predict(X_te)
                pred_cm = np.expm1(logpred)
                yte_cm = np.expm1(yte)
        
                # Compute metrics
                rmse_r = float(np.sqrt(mean_squared_error(yte_cm, pred_cm)))
                mae_r  = float(mean_absolute_error(yte_cm, pred_cm))
        
                # Weighted R² requires a target-type → use dummy GDM_g
                target_names_test = np.array(['GDM_g'] * len(yte_cm))
                r2_r = compute_weighted_r2_metric(pred_cm, yte_cm, target_names_test)
        
                # Save model into class
                self.height_model = ridge
                self.height_model_type = 'log1p'
        
                # Store metrics and the model object (so propagation can find it)
                results['ridge'] = {
                    'model': ridge,
                    'rmse': rmse_r,
                    'mae': mae_r,
                    'r2':  r2_r,
                    'height_model_type': 'log1p'
                }
                results['_ridge_model_obj'] = ridge
        
                # Persist artifacts to disk
                try:
                    import joblib, json, os
                    out_dir = OUTPUTS_DIR if isinstance(OUTPUTS_DIR, Path) else Path(OUTPUTS_DIR)
                    out_dir.mkdir(parents=True, exist_ok=True)
                    # powerlaw
                    pl_path = out_dir / "height_powerlaw.json"
                    json.dump({'a': float(a), 'b': float(b)}, pl_path.open("w"))
                    # ridge model
                    ridge_path = out_dir / "height_ridge.joblib"
                    joblib.dump(ridge, ridge_path)
                    logger.info(f"Saved height calibration artifacts to {out_dir}")
                except Exception as e:
                    logger.debug(f"Failed to save height calibration artifacts: {e}")
        
            except Exception as e:
                logger.warning(f"Ridge model fitting failed: {e}")
                results['ridge'] = None
                # ensure we don't leave partial model refs
                results.pop('_ridge_model_obj', None)
        
        # Final logging (powerlaw + ridge summary)
        powerlaw_rmse = results.get('powerlaw', {}).get('rmse', float('nan'))
        logger.info(
            f"Calibrated powerlaw: a={a:.4f}, b={b:.4f} "
            f"(trained on {len(df_gt)} samples). Powerlaw test RMSE={powerlaw_rmse:.3f}"
        )
        
        if results.get('ridge'):
            logger.info(
                f"Trained Ridge (log-target) test RMSE={results['ridge']['rmse']:.3f}, "
                f"R²={results['ridge']['r2']:.4f}"
            )
        
        results['_source_dataset'] = self
        return results


def create_data_loaders(train_df, test_df, lookups_dict, TOTAL_TABULAR_DIM, model=None):
    """
    Create training, validation, and test data loaders with patch-based processing.
    Integrates intelligent TARGET-SPECIFIC test feature fetching.
    
    Returns:
        tuple: (train_loader, val_loader, test_loader, train_split_df, val_split_df)
    """
    logger.info("="*80)
    logger.info("CREATING DATA LOADERS WITH INTELLIGENT TEST FEATURE FETCHING")
    logger.info("="*80)

    # STEP 1: SPLIT DATA
    logger.info("\nStep 1: Train/Val Split")
    
    train_df['image_id'] = train_df['sample_id'].fillna('').astype(str).str.split('__').str[0]
    unique_images = train_df[['image_id', 'image_path']].drop_duplicates()
    n_unique = len(unique_images)
    
    # Split by image (not by row!)
    train_images, val_images = train_test_split(
        unique_images,
        test_size=CONFIG['val_split'],
        random_state=CONFIG['random_seed']
    )
    
    train_split_df = train_df[
        train_df['image_id'].isin(train_images['image_id'])
    ].reset_index(drop=True)
    
    val_split_df = (
        train_df[train_df['image_id'].isin(val_images['image_id'])]   
        .reset_index(drop=True)
    )

    logger.info(f"  Total unique images: {n_unique}")
    logger.info(f"  Training: {len(train_images)} images → {len(train_split_df)} rows")
    logger.info(f"    After cropping: {len(train_split_df) * 8} patches")
    logger.info(f"  Validation: {len(val_images)} images → {len(val_split_df)} rows")
    logger.info(f"    After cropping: {len(val_split_df) * 8} patches")
    
    # STEP 2: DEFINE TRANSFORMS
    logger.info("\nStep 2: Defining Transforms")

    
    train_transform = transforms.Compose([
        # PIL Image transforms (BEFORE ToTensor)
        transforms.RandomHorizontalFlip(p=0.7),
        transforms.RandomVerticalFlip(p=0.7),
        transforms.RandomRotation(60),
        transforms.RandomAffine(
            degrees=45,
            translate=(0.25, 0.25),
            scale=(0.7, 1.3),
            shear=20
        ),
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
        
        # Lighting conditions
        transforms.ColorJitter(
            brightness=0.6,
            contrast=0.6,
            saturation=0.5,
            hue=0.15
        ),
        
        transforms.ToTensor(),
        
        # Normalize (tensor transform)
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 3.0)),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.2)),
    ])

    
    val_test_transform = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # STEP 3: CREATE DATASETS
    logger.info("\nStep 3: Creating Datasets with Intelligent Test Feature Fetching")
    
    train_dataset = BiomassDataset(
        train_split_df,
        RAW_DATA_DIR / 'train',
        transform=train_transform,
        mode='train',
        train_df=train_split_df,
        lookups_dict=lookups_dict
    )
    
    val_dataset = BiomassDataset(
        val_split_df,
        RAW_DATA_DIR / 'train',
        transform=val_test_transform,
        mode='val',
        train_df=train_df,
        lookups_dict=lookups_dict
    )
    
    test_dataset = BiomassDataset(
        test_df,
        RAW_DATA_DIR / 'test',
        transform=val_test_transform,
        mode='test',
        train_df=train_df, 
        lookups_dict=lookups_dict,
        species_model = model,
        device = device
    )

    # --- QUICK SANITY CHECK: total tabular dim consistent with lookups ---
    try:
        numeric_cols = lookups_dict.get("TAB_NUMERIC_COLS", [])
        STATE_COLS = lookups_dict.get("STATE_COLS", [])
        SPECIES_COLS = lookups_dict.get("SPECIES_COLS", [])
        TARGET_TYPE_COLS = lookups_dict.get("TARGET_TYPE_COLS", [])
        calc = len(numeric_cols) + len(STATE_COLS) + len(SPECIES_COLS) + len(TARGET_TYPE_COLS)
        if calc != TOTAL_TABULAR_DIM:
            logger.warning(f"TOTAL_TABULAR_DIM mismatch: computed={calc}, declared={TOTAL_TABULAR_DIM}")
        else:
            logger.info("TOTAL_TABULAR_DIM consistent")
    except Exception as e:
        logger.debug(f"TOTAL_TABULAR_DIM sanity check failed: {e}")

    logger.info(f"  ✓ Train dataset: {len(train_dataset)} rows")
    logger.info(f"  ✓ Val dataset: {len(val_dataset)} rows")
    
    # --- Calibrate height model on training dataset and propagate to val/test ---
    logger.info("Calibrating height estimator using train dataset features...")
    
    # Create a static flag on the function to ensure ONE-TIME calibration
    if not hasattr(create_data_loaders, "_height_calibrated_once"):
        create_data_loaders._height_calibrated_once = False
    
    calib_res = None
    def _propagate_calibration(dst_dataset, calib_res):
        """Copy fitted artifacts onto dataset instance (powerlaw a,b and ridge model)."""
        if calib_res is None:
            return
        # powerlaw
        pl = calib_res.get('powerlaw')
        if pl and 'a' in pl and 'b' in pl:
            setattr(dst_dataset, 'height_a', float(pl['a']))
            setattr(dst_dataset, 'height_b', float(pl['b']))
    
        # ridge/model: check several possible locations
        ridge_info = calib_res.get('ridge', {}) or {}
        model_obj = ridge_info.get('model', None) or calib_res.get('_ridge_model_obj', None)
    
        # if still None, try to read from train_dataset attr (if calib_res came from that instance)
        if model_obj is None:
            try:
                src = calib_res.get('_source_dataset', None)
                if src is not None and hasattr(src, 'height_model') and getattr(src, 'height_model', None) is not None:
                    model_obj = getattr(src, 'height_model')
            except Exception:
                pass
    
        if model_obj is not None:
            try:
                setattr(dst_dataset, 'height_model', model_obj)
                # prefer explicit type if present
                ht = ridge_info.get('height_model_type') or calib_res.get('height_model_type') or getattr(model_obj, 'height_model_type', None)
                if ht:
                    setattr(dst_dataset, 'height_model_type', ht)
                else:
                    # fallback default
                    setattr(dst_dataset, 'height_model_type', 'log1p')
            except Exception as e:
                logger.debug(f"_propagate_calibration: failed to set height_model: {e}")

    
    # ====== CASE A: First call → run calibration ======
    if not create_data_loaders._height_calibrated_once:
    
        try:
            calib_res = train_dataset.calibrate_height_model(
                image_col='image_path',
                height_col='Height_Ave_cm',
                test_size=0.2,
                random_state=CONFIG.get('random_seed', 42),
                min_positive_samples=5,
                fit_ridge=True,
                ridge_alphas=(0.01, 0.1, 1.0, 10.0)
            )
            logger.info(f"Calibration results: {calib_res.get('powerlaw')}")
            if calib_res.get('ridge') is not None:
                logger.info(f"  Ridge test RMSE: {calib_res['ridge']['rmse']:.3f}")
    
            # mark as done so we NEVER run this again
            create_data_loaders._height_calibrated_once = True
    
        except Exception as e:
            logger.exception(f"Height calibration failed: {e}")
            calib_res = None
    
    # ====== CASE B: Later calls → load stored calibration only ======
    else:
        logger.info("✓ Skipping height calibration (already performed once).")
        # Attempt to load saved calibration if available
        try:
            import json, joblib
            calib_res = {}
            out_dir = OUTPUTS_DIR if isinstance(OUTPUTS_DIR, Path) else Path(OUTPUTS_DIR)
            pl_path = out_dir / "height_powerlaw.json"
            ridge_path = out_dir / "height_ridge.joblib"
    
            if pl_path.exists():
                try:
                    calib_res['powerlaw'] = json.loads(pl_path.read_text())
                    logger.info("Loaded saved powerlaw calibration from disk.")
                except Exception as e:
                    logger.debug(f"Failed to read powerlaw json: {e}")
    
            if ridge_path.exists():
                try:
                    ridge_model = joblib.load(ridge_path)
                    calib_res['ridge'] = {'model': ridge_model, 'height_model_type': 'log1p'}
                    logger.info("Loaded saved ridge calibration from disk.")
                except Exception as e:
                    logger.debug(f"Failed to load ridge model: {e}")
    
            # If loaded nothing, leave calib_res None to maintain previous behavior
            if not calib_res:
                calib_res = None
    
        except Exception as e:
            logger.debug(f"Failed loading saved calibration: {e}")
            calib_res = None

    # Always propagate whatever calibration we have
    _propagate_calibration(train_dataset, calib_res)
    _propagate_calibration(val_dataset, calib_res)
    _propagate_calibration(test_dataset, calib_res)
    
    
    # Quick sanity checks
    try:
        logger.info("Sanity check - sample calibrated heights:")
        sample_img_path = train_dataset.df.iloc[0]['image_path']
        img = train_dataset._load_image(sample_img_path)
        h_train = train_dataset.estimate_height_final(np.array(img))
        logger.info(f"  sample train estimated height: {h_train} cm")
    except Exception as e:
        logger.debug(f"Sanity check train failed: {e}")
    
    try:
        sample_img_path = test_dataset.df.iloc[0]['image_path']
        img = test_dataset._load_image(sample_img_path)
        h_test = test_dataset.estimate_height_final(np.array(img))
        logger.info(f"  sample test estimated height: {h_test} cm")
    except Exception as e:
        logger.debug(f"Sanity check test failed: {e}")


    def collate_patches(batch):
        """
        Collate function for patch-based dataset.
        
        Input batch items (7 elements):
            (images, tabular, species_idx, target_idx, target, sample_id, target_name)
        
        Returns:
            (images_batch, tabular_batch, species_batch, target_idx_batch, targets_batch, 
             sample_ids, target_names)
        """
        PATCHES_PER_IMAGE = 8
        C, H, W = 3, 224, 224
    
        # detect TAB_DIM
        TAB_DIM = None
        for item in batch:
            try:
                tab = item[1]
                TAB_DIM = tab.shape[-1]
                break
            except Exception:
                continue
        if TAB_DIM is None:
            TAB_DIM = 30  # default for meta-learner
    
        sentinel_images = torch.zeros((PATCHES_PER_IMAGE, C, H, W))
        sentinel_tab = torch.zeros((PATCHES_PER_IMAGE, TAB_DIM))
        sentinel_target = torch.tensor(0.0)
        sentinel_target_idx = torch.tensor(0, dtype=torch.long)
    
        per_sample_images = []
        per_sample_tabular = []
        per_sample_species = []
        per_sample_target_idx = []  # NEW: store target_idx per sample
        per_sample_targets = []
        per_sample_sample_ids = []
        per_sample_target_names = []
    
        # parse batch items safely and collect per-sample scalars
        for item in batch:
            try:
                # Handle 7-item return (NEW FORMAT with target_idx)
                if len(item) == 7:
                    images, tabular, species, target_idx, target, sample_id, target_name = item
                # Handle 6-item return (OLD FORMAT - backward compat)
                elif len(item) == 6:
                    images, tabular, species, target, sample_id, target_name = item
                    target_idx = 0  # default fallback
                # Handle 5-item return (even older format - backward compat)
                elif len(item) == 5:
                    images, tabular, target, sample_id, target_name = item
                    species = None
                    target_idx = 0
                else:
                    # flexible fallback
                    images = item[0] if len(item) > 0 else sentinel_images.clone()
                    tabular = item[1] if len(item) > 1 else sentinel_tab.clone()
                    species = item[2] if len(item) > 2 else None
                    target_idx = item[3] if len(item) > 3 else 0  # NEW: try to get from position 3
                    target = item[4] if len(item) > 4 else None
                    sample_id = item[-2] if len(item) >= 2 else "None"
                    target_name = item[-1] if len(item) >= 1 else "None"
    
                sample_id = "None" if sample_id is None else sample_id
                target_name = "None" if target_name is None else target_name
    
                # ensure images shape = (PATCHES_PER_IMAGE, C, H, W)
                if not (isinstance(images, torch.Tensor) and images.ndim == 4 and images.size(0) == PATCHES_PER_IMAGE):
                    logger.warning(f"[collate] BAD IMAGES for sample {sample_id} -> shape={getattr(images, 'shape', None)}; using sentinel")
                    images = sentinel_images.clone()
    
                # ensure tabular shape = (PATCHES_PER_IMAGE, TAB_DIM) or (TAB_DIM,) (convert)
                if isinstance(tabular, torch.Tensor) and tabular.ndim == 1:
                    tabular = tabular.unsqueeze(0).repeat(PATCHES_PER_IMAGE, 1)
                if not (isinstance(tabular, torch.Tensor) and tabular.ndim == 2 and tabular.size(0) == PATCHES_PER_IMAGE):
                    logger.warning(f"[collate] BAD TABULAR for sample {sample_id} -> shape={getattr(tabular, 'shape', None)}; using sentinel")
                    tabular = sentinel_tab.clone()
    
                per_sample_images.append(images)
                per_sample_tabular.append(tabular)
                per_sample_species.append(species)
                
                # NEW: Store target_idx (convert to tensor if needed)
                if isinstance(target_idx, torch.Tensor):
                    per_sample_target_idx.append(target_idx.long())
                else:
                    per_sample_target_idx.append(torch.tensor(int(target_idx), dtype=torch.long))
    
                # store one scalar or None
                if target is None:
                    per_sample_targets.append(None)
                else:
                    # ensure scalar float tensor
                    t = torch.tensor(float(target), dtype=torch.float32) if not isinstance(target, torch.Tensor) else target.float().reshape(())
                    per_sample_targets.append(t)
    
                per_sample_sample_ids.append(str(sample_id))
                per_sample_target_names.append(str(target_name))
    
            except Exception as e:
                logger.exception(f"[collate] EXCEPTION parsing sample -> {e}")
                per_sample_images.append(sentinel_images.clone())
                per_sample_tabular.append(sentinel_tab.clone())
                per_sample_species.append(None)
                per_sample_target_idx.append(sentinel_target_idx.clone())  # NEW: add sentinel
                per_sample_targets.append(sentinel_target)
                per_sample_sample_ids.append("None")
                per_sample_target_names.append("None")
    
        # Now expand per-sample -> per-patch deterministically
        images_batch = torch.cat(per_sample_images, dim=0)          # shape (B*P, C, H, W)
        tabular_batch = torch.cat(per_sample_tabular, dim=0)       # shape (B*P, TAB_DIM)
    
        # species batch: convert to tensor if all ints else keep list
        species_is_int = all((x is None) or isinstance(x, (int, np.integer, torch.Tensor)) for x in per_sample_species)
        if species_is_int:
            clean_species = []
            for x in per_sample_species:
                if x is None:
                    clean_species.append(0)
                elif isinstance(x, torch.Tensor):
                    clean_species.append(int(x.item()))
                else:
                    clean_species.append(int(x))
            species_batch = torch.tensor(clean_species, dtype=torch.long)
        else:
            species_batch = [None if x is None else x for x in per_sample_species]
    
        # NEW: target_idx_batch (repeat per patch, per sample)
        target_idx_list = []
        for tidx in per_sample_target_idx:
            # Each sample's target_idx repeated PATCHES_PER_IMAGE times
            tidx_val = tidx.item() if isinstance(tidx, torch.Tensor) else int(tidx)
            target_idx_list.extend([tidx_val] * PATCHES_PER_IMAGE)
        target_idx_batch = torch.tensor(target_idx_list, dtype=torch.long)
    
        # Build targets_list (per patch) from per_sample_targets to avoid accidental length mismatch
        targets_list = []
        for t in per_sample_targets:
            if t is None:
                # keep no-target marker (we use None > caller expects None for test)
                targets_list.extend([None] * PATCHES_PER_IMAGE)
            else:
                targets_list.extend([t] * PATCHES_PER_IMAGE)
    
        # sample_ids and target_names repeated per patch
        sample_ids = []
        target_names = []
        for sid, tn in zip(per_sample_sample_ids, per_sample_target_names):
            sample_ids.extend([sid] * PATCHES_PER_IMAGE)
            target_names.extend([tn] * PATCHES_PER_IMAGE)
    
        # Final alignment checks & conversions
        expected_len = len(batch) * PATCHES_PER_IMAGE
        assert images_batch.size(0) == expected_len, f"images length {images_batch.size(0)} != expected {expected_len}"
        assert tabular_batch.size(0) == expected_len, f"tabular length {tabular_batch.size(0)} != expected {expected_len}"
        assert target_idx_batch.size(0) == expected_len, f"target_idx length {target_idx_batch.size(0)} != expected {expected_len}"  # NEW
        assert len(sample_ids) == expected_len, f"sample_ids length {len(sample_ids)} != expected {expected_len}"
        assert len(target_names) == expected_len, f"target_names length {len(target_names)} != expected {expected_len}"
        assert len(targets_list) == expected_len, f"targets_list length {len(targets_list)} != expected {expected_len}"
    
        # If we have no targets (test mode -> all targets None) return None target batch
        if all(t is None for t in targets_list):
            # NEW: return 7 items including target_idx_batch
            return images_batch, tabular_batch, species_batch, target_idx_batch, None, sample_ids, target_names
    
        # else convert targets_list (some elements may be torch tensors already)
        targ_tensors = []
        for t in targets_list:
            if t is None:
                targ_tensors.append(torch.tensor(0.0))   # sentinel for safety (shouldn't happen in train/val)
            elif isinstance(t, torch.Tensor) and t.numel() == 1:
                targ_tensors.append(t.reshape(()))
            else:
                targ_tensors.append(torch.tensor(float(t), dtype=torch.float32))
        targets_batch = torch.stack(targ_tensors, dim=0).float()
    
        # NEW: return 7 items including target_idx_batch
        return images_batch, tabular_batch, species_batch, target_idx_batch, targets_batch, sample_ids, target_names


    # STEP 5: CREATE DATALOADERS
    logger.info("\nStep 5: Creating DataLoaders")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=CONFIG['num_workers'],
        pin_memory=CONFIG['pin_memory'],
        collate_fn=collate_patches,
        drop_last=True 
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=CONFIG['num_workers'],
        pin_memory=CONFIG['pin_memory'],
        collate_fn=collate_patches,
        drop_last=True 
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=CONFIG['num_workers'],
        pin_memory=CONFIG['pin_memory'],
        collate_fn=collate_patches,
        drop_last=False 
    )
    
    logger.info(f"  ✓ Train loader: {len(train_loader)} batches")
    logger.info(f"  ✓ Val loader: {len(val_loader)} batches")
    logger.info(f"  ✓ Test loader: {len(test_loader)} batches")
    logger.info(f"\n  Batch size: {CONFIG['batch_size']} images")
    logger.info(f"  Per batch: {CONFIG['batch_size']} images × 8 patches = {CONFIG['batch_size']*8} patches")
    logger.info(f"  Batch shapes:")
    logger.info(f"    - images: [{CONFIG['batch_size']*8}, 3, 224, 224]")
    logger.info(f"    - tabular: [{CONFIG['batch_size']*8}, {TOTAL_TABULAR_DIM}]")
    logger.info(f"    - targets: [{CONFIG['batch_size']*8}]")
    logger.info("\n✓ Data loaders created successfully!")
    logger.info("  ✓ Test feature fetching: TARGET-SPECIFIC at ALL LEVELS")
    logger.info("  ✓ Patches per image: 8 (4×2 grid from 2000×1000 images)")
    logger.info("="*80 + "\n")
    
    return train_loader, val_loader, test_loader, train_split_df, val_split_df

logger.info("✓ Data Augmentation and Preparation")

## 5: LOSS FUNCTION - WEIGHTED R²

**Purpose**: Implement competition metric as neural network loss function.

**Weighted R² Formula**:
```
R² = 1 - (SS_residual / SS_total)

Where:
  SS_residual = Σ(w_i × (ŷ_i - y_i)²)   [weighted squared errors]
  SS_total    = Σ(w_i × (y_i - ȳ_weighted)²)  [weighted variance]
  w_i         = weight for target type i
  ȳ_weighted  = Σ(w_i × y_i) / Σ(w_i)  [weighted mean]
```

**Weighting Scheme** (Competition Metric):

| Target | Weight | Interpretation |
|--------|--------|-----------------|
| Dry_Total_g | 0.50 | Primary focus (50% of loss) |
| GDM_g | 0.20 | Secondary focus (20%) |
| Dry_Green_g | 0.10 | Supporting (10%) |
| Dry_Dead_g | 0.10 | Supporting (10%) |
| Dry_Clover_g | 0.10 | Supporting (10%) |

**Implementation Details**:

1. **Prediction Clamping**: Ensure predictions ≥ 0 (non-negative biomass)
2. **Weight Normalization**: Normalize weights to sum to 1.0
3. **Numerical Stability**: Check SS_total > 1e-8 before division
4. **Gradient Direction**: Return -R² for minimization (loss to minimize)

**Training Usage**:
```python
loss = weighted_r2_loss(predictions, targets, target_names)
loss.backward()  # Backpropagation
```

**Validation Usage**:
```python
r2_metric = compute_weighted_r2_metric(preds_np, targets_np, names_np)
# Display positive R² (higher is better)
```

**Key Properties**:
- R² ∈ [-100, 1]: Clamped to prevent numerical instability
- Perfect prediction: R² = 1 (loss = 0)
- Mean prediction: R² = 0 (loss = 0)
- Worse than mean: R² < 0 (loss > 0)

**Output**: 
- Weighted R² loss function defined
- Compatible with PyTorch backpropagation
- Metric computation for evaluation

---

In [2]:
def weighted_r2_loss(
    predictions, 
    targets, 
    target_names, 
    device=None, 
    weights_config=None,
    eps=1e-8,
    strict_length_check=False
):
    """
    Production-ready weighted R² loss (differentiable).
    
    Args:
        predictions (torch.Tensor): Model predictions, shape [N] or [N, 1]
        targets (torch.Tensor): Ground truth values, same shape as predictions
        target_names (torch.Tensor | np.ndarray | list): Target type names for weighting
        weights_config (dict): Mapping of {target_name: weight}, e.g. {'dry_total_g': 2.0}
                              If None, uses uniform weights
        device (torch.device | str): Device for computation
        eps (float): Small constant for numerical stability
        strict_length_check (bool): If True, raise error on length mismatch instead of truncating
    
    Returns:
        torch.Tensor: Scalar loss = 1.0 - R²_weighted (0 = perfect, higher = worse)
    
    Raises:
        TypeError: If inputs are not torch tensors
        ValueError: If shapes are incompatible or tensors are empty
    """
    # 1. INPUT VALIDATION
    # ═══════════════════════════════════════════════════════════════
    if not torch.is_tensor(predictions) or not torch.is_tensor(targets):
        raise TypeError("predictions and targets must be torch tensors")
    
    if predictions.numel() == 0 or targets.numel() == 0:
        raise ValueError("Empty tensors not supported")
    
    preds = predictions.view(-1).float()
    targs = targets.view(-1).float()
    
    if preds.shape != targs.shape:
        raise ValueError(
            f"Shape mismatch after flattening: predictions={preds.shape}, targets={targs.shape}"
        )
    
    # 2. DEVICE HANDLING
    # ═══════════════════════════════════════════════════════════════
    dev = device if device is not None else preds.device
    if not isinstance(dev, torch.device):
        dev = torch.device(dev)
    
    preds = preds.to(dev)
    targs = targs.to(dev)
    
    # 3. PROCESS TARGET NAMES
    # ═══════════════════════════════════════════════════════════════
    if target_names is None:
        names_list = ["__uniform__"] * preds.numel()
    elif isinstance(target_names, torch.Tensor):
        # Detach to avoid gradient issues
        target_names = target_names.detach().view(-1)
        names_list = [str(x).strip().lower() for x in target_names.cpu().tolist()]
    else:
        names_arr = np.asarray(target_names).flatten()
        names_list = [str(x).strip().lower() for x in names_arr]
    
    # 4. LENGTH ALIGNMENT
    # ═══════════════════════════════════════════════════════════════
    if len(names_list) != preds.numel():
        if strict_length_check:
            raise ValueError(
                f"Length mismatch: predictions={preds.numel()}, "
                f"targets={targs.numel()}, target_names={len(names_list)}"
            )
        else:
            # Lenient mode: truncate to shortest
            m = min(len(names_list), preds.numel())
            logger.warning(
                f"weighted_r2_loss: length mismatch, truncating to {m} samples "
                f"(preds={preds.numel()}, targets={targs.numel()}, names={len(names_list)})"
            )
            names_list = names_list[:m]
            preds = preds[:m]
            targs = targs[:m]
    
    # 5. BUILD WEIGHT MAP
    # ═══════════════════════════════════════════════════════════════
    weights_map = {}
    if weights_config is not None:
        weights_map = {str(k).strip().lower(): float(v) for k, v in weights_config.items()}
    elif 'CONFIG' in globals():
        # Fallback to global CONFIG if available
        weights_map = {str(k).strip().lower(): float(v) 
                      for k, v in globals()['CONFIG'].get('r2_weights', {}).items()}
    
    # 6. CREATE WEIGHT TENSOR 
    # ═══════════════════════════════════════════════════════════════
    if weights_map:
        # Vectorized lookup via numpy (faster than list comprehension for large batches)
        row_weights_np = np.array([weights_map.get(name, 0.0) for name in names_list], 
                                   dtype=np.float32)
        row_weights = torch.from_numpy(row_weights_np).to(dev)
    else:
        row_weights = torch.ones(preds.numel(), dtype=torch.float32, device=dev)
    
    # Fallback to uniform weights if all zeros
    if row_weights.sum() <= eps:
        logger.debug("All weights are zero; using uniform weights")
        row_weights = torch.ones_like(row_weights)
    
    # 7. COMPUTE WEIGHTED R²
    # ═══════════════════════════════════════════════════════════════
    # Weighted mean: ȳ_w = Σ(w_i * y_i) / Σ(w_i)
    wsum = row_weights.sum() + eps
    y_wbar = (row_weights * targs).sum() / wsum
    
    # Sum of squares
    ss_res = ((targs - preds).pow(2) * row_weights).sum()  # Residual
    ss_tot = ((targs - y_wbar).pow(2) * row_weights).sum()  # Total
    
    # R² = 1 - (SS_res / SS_tot)
    r2 = 1.0 - ss_res / (ss_tot + eps)
    
    # Clamp to reasonable bounds (R² ∈ [-100, 1])
    r2 = r2.clamp(min=-1.0, max=1.0)
    
    # Return loss (1 - R²): perfect model → loss = 0
    return 1.0 - r2


def compute_weighted_r2_metric(
    predictions, 
    targets, 
    target_names, 
    weights_config=None,
    eps=1e-8
):
    """
    NumPy implementation of weighted R² (evaluation only).
    
    Args:
        predictions (np.ndarray | list): Model predictions
        targets (np.ndarray | list): Ground truth values
        target_names (np.ndarray | list): Target type names for weighting
        weights_config (dict): Mapping of {target_name: weight}
        eps (float): Small constant for numerical stability
    
    Returns:
        float: R²_weighted ∈ [-100, 1] (1 = perfect, 0 = baseline, negative = worse than mean)
    """
    # 1. CONVERT TO NUMPY
    # ═══════════════════════════════════════════════════════════════
    preds = np.asarray(predictions).flatten()
    targs = np.asarray(targets).flatten()
    
    if target_names is None:
        names = np.array(["__uniform__"] * len(preds))
    else:
        names = np.asarray(target_names).flatten()
    
    # 2. ALIGN LENGTHS
    # ═══════════════════════════════════════════════════════════════
    min_len = min(len(preds), len(targs), len(names))
    if not (len(preds) == len(targs) == len(names)):
        logger.warning(
            f"compute_weighted_r2_metric: length mismatch, truncating to {min_len} "
            f"(preds={len(preds)}, targets={len(targs)}, names={len(names)})"
        )
        preds = preds[:min_len]
        targs = targs[:min_len]
        names = names[:min_len]
    
    if min_len == 0:
        logger.error("No valid samples after alignment")
        return 0.0
    
    # 3. BUILD WEIGHT MAP
    # ═══════════════════════════════════════════════════════════════
    names_norm = np.array([str(x).strip().lower() for x in names], dtype=object)
    
    weights_map = {}
    if weights_config is not None:
        weights_map = {str(k).strip().lower(): float(v) for k, v in weights_config.items()}
    elif 'CONFIG' in globals():
        weights_map = {str(k).strip().lower(): float(v) 
                      for k, v in globals()['CONFIG'].get('r2_weights', {}).items()}
    
    # 4. CREATE WEIGHTS ARRAY
    # ═══════════════════════════════════════════════════════════════
    if weights_map:
        row_w = np.array([weights_map.get(name, 0.0) for name in names_norm], dtype=np.float32)
    else:
        row_w = np.ones(len(preds), dtype=np.float32)
    
    # Fallback to uniform weights
    if row_w.sum() == 0:
        logger.warning("compute_weighted_r2_metric: no matching weights; using uniform")
        row_w[:] = 1.0
    
    # 5. COMPUTE WEIGHTED R²
    # ═══════════════════════════════════════════════════════════════
    wsum = row_w.sum() + eps
    y_wbar = (row_w * targs).sum() / wsum
    
    ss_res = ((targs - preds) ** 2 * row_w).sum()
    ss_tot = ((targs - y_wbar) ** 2 * row_w).sum()
    
    if ss_tot < eps:
        logger.debug("SS_tot ≈ 0; returning R² = 0")
        r2 = 0.0
    else:
        r2 = 1.0 - ss_res / ss_tot
    
    return float(np.clip(r2, -100.0, 1.0))


logger.info('✓ Weighted R² loss functions defined')

NameError: name 'logger' is not defined

##  6: MODEL ARCHITECTURES - 6 MODELS

**Purpose**: Define 6 models for ensemble learning system.

### Model 1: Vision Transformer (ViT-Base)

**Architecture**:
```
Input: [batch*8, 3, 224, 224] patches
  ↓
ViT-Base Encoder
- Patch embedding: 16×16 patches
- 12 transformer layers
- 12 attention heads
- Hidden dim: 768
  ↓
Global average pooling → [batch*8, 768]
  ↓
Regression Head
- FC: 768 → 256 (GELU + Dropout 0.3)
- FC: 256 → 128 (GELU + Dropout 0.3)
- FC: 128 → 1
  ↓
Output: [batch*8] predictions
```

**Configuration**:
- Pretrained: ImageNet weights (timm)
- Learning rate: 1e-4
- Epochs: 50
- Early stopping: patience=20

### Model 2: ResNet-50

**Architecture**:
```
Input: [batch*8, 3, 224, 224]
  ↓
ResNet-50 Backbone
- Conv blocks with residual connections
- 50 layers total
- Output: [batch*8, 2048]
  ↓
Global Average Pooling → [batch*8, 2048]
  ↓
Regression Head
- FC: 2048 → 512 (ReLU + Dropout 0.3)
- FC: 512 → 256 (ReLU + Dropout 0.2)
- FC: 256 → 1
  ↓
Output: [batch*8] predictions
```

**Configuration**:
- Pretrained: ImageNet weights
- Learning rate: 5e-5
- Epochs: 50

### Model 3: DenseNet-121

**Architecture**:
```
Input: [batch*8, 3, 224, 224]
  ↓
DenseNet-121 Backbone
- Dense connections between layers
- 121 layers total
- Output: [batch*8, 1024]
  ↓
Global Average Pooling → [batch*8, 1024]
  ↓
Regression Head
- FC: 1024 → 512 (ReLU + Dropout 0.3)
- FC: 512 → 256 (ReLU + Dropout 0.2)
- FC: 256 → 1
  ↓
Output: [batch*8] predictions
```

**Configuration**:
- Pretrained: ImageNet weights
- Learning rate: 5e-5
- Epochs: 50

### Model 4: XGBoost (Tabular Model)

**Purpose**: Learn from 29 tabular features alone (without image patches)

**Hyperparameters**:
```python
XGBRegressor(
    n_estimators=1000,     # Boosting rounds
    max_depth=6,           # Tree depth
    learning_rate=0.03,    # Step size
    subsample=0.85,        # Row subsampling
    colsample_bytree=0.85, # Feature subsampling
    objective='reg:squarederror'
)
```

**Features**: 29 tabular features (NDVI, height, month, state, species, target type)

**Training**:
- Image-level (not patch-level)
- One prediction per training row
- Optional: Separate models per target type

### Model 5: Meta-Learner (Stacking)

**Purpose**: Learn optimal combination of 5 base model predictions

**Input Fusion**:
```
5 base predictions + 29 tabular features = 34 dimensions
```

**Architecture**:
```
Input: [batch, 34] (5 predictions + 29 features)
  ↓
Layer 1: FC(34→64) + BatchNorm + ReLU + Dropout(0.2)
  ↓
Layer 2: FC(64→32) + BatchNorm + ReLU + Dropout(0.15)
  ↓
Layer 3: FC(32→16) + ReLU + Dropout(0.1)
  ↓
Output: FC(16→1)
  ↓
Final prediction
```

**Training**:
- Input: Predictions from 5 base models on training data
- Target: Ground truth biomass values
- Loss: Weighted R²
- Epochs: 100
- Early stopping: patience=20

### Model 6: Species Classification ViT

**Purpose**: Predict species from  image patches (improves test feature fetching)

**Architecture**:
```
Input: [batch*8, 3, 224, 224] patches
  ↓
ViT-Base backbone → [batch*8, 768]
  ↓
Classification Head
- FC(768→256) + GELU + Dropout(0.3)
- FC(256→128) + GELU + Dropout(0.2)
- FC(128→15 logits)
  ↓
Output: [batch*8, 15] logits (15 species classes)
```

**Training**:
- Loss: CrossEntropyLoss (classification)
- Optimizer: AdamW

### Ensemble Strategy Summary

| Component | Type | Input | Output | Purpose |
|-----------|------|-------|--------|---------|
| ViT-Base | Image | Patches | Prediction | Transformer attention |
| ResNet-50 | Image | Patches | Prediction | Residual connections |
| DenseNet-121 | Image | Patches | Prediction | Dense connections |
| XGBoost | Tabular | 29 features | Prediction | Gradient boosting |
| Meta-Learner | Fusion | 5+29 | Final | Learned combination |
| Species ViT | Image | Patches | Species class | Improve fallback |

**Output**: 
- 6 model classes defined

In [ ]:
class MILAggregator(nn.Module):
    """
    Multiple Instance Learning Aggregator for patch-based predictions.

    Learns attention weights for each patch to perform weighted aggregation.
    Designed to work seamlessly with your 8-patch strategy.
    """
    def __init__(self, feature_dim=512, num_patches=8, dropout=0.2):
        """
        Args:
            feature_dim (int): Feature dimension from models (ViT: 768, ResNet: 2048, DenseNet: 1024)
            num_patches (int): Number of patches per image (default: 8)
            dropout (float): Dropout rate for regularization
        """
        super().__init__()
        self.feature_dim = feature_dim
        self.num_patches = num_patches

        # Attention network: learns importance of each patch
        self.attention_net = nn.Sequential(
            nn.Linear(feature_dim, max(128, feature_dim // 4)),
            nn.LayerNorm(max(128, feature_dim // 4)),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(max(128, feature_dim // 4), 64),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, patch_features, return_weights=False):
        """
        Aggregate patch-level features using learned attention weights.

        Args:
            patch_features: [batch_size*num_patches, feature_dim]
                           OR [batch_size, num_patches, feature_dim]
            return_weights: If True, return attention weights for visualization

        Returns:
            aggregated: [batch_size, feature_dim] - image-level aggregated features
            weights (optional): [batch_size, num_patches] - attention weights
        """
        # Handle different input shapes
        if patch_features.ndim == 2:
            # Reshape from [B*P, D] to [B, P, D]
            batch_size = patch_features.shape[0] // self.num_patches
            patch_features = patch_features.view(batch_size, self.num_patches, self.feature_dim)
        else:
            batch_size = patch_features.shape[0]

        # Compute attention scores for each patch
        scores = self.attention_net(patch_features.view(-1, self.feature_dim))  # [B*P, 1]
        scores = scores.view(batch_size, self.num_patches)  # [B, P]

        # Convert scores to weights via softmax
        weights = torch.nn.functional.softmax(scores, dim=1)  # [B, P]  

        # Weighted aggregation of patches
        weighted_features = patch_features * weights.unsqueeze(-1)  # [B, P, D] * [B, P, 1]
        aggregated = weighted_features.sum(dim=1)  # [B, D]

        if return_weights:
            return aggregated, weights
        return aggregated


class TaskEmbedding(nn.Module):
    def __init__(self, num_targets=5, embed_dim=64):
        super().__init__()
        self.task_embeddings = nn.Embedding(num_targets, embed_dim)
    
    def forward(self, target_idx):
        return self.task_embeddings(target_idx)


class VisionTransformerModel(nn.Module):
    """
    Vision Transformer regression model with optional MIL aggregation and task conditioning.

    - If use_mil=False: expects x shaped [B, 3, H, W] and returns [B] regression outputs.
    - If use_mil=True: expects x shaped [B*P, 3, H, W] (patches grouped per-image) and returns [B] outputs.
    - target_idx may be a tensor/array/list of length B (image-level targets) or length N (patch-level) but will be normalized.
    - return_features: when True:
        * if use_mil=False -> returns backbone features [B, D]
        * if use_mil=True  -> returns per-patch features [B*P, D] (useful for MIL training)
    """
    def __init__(self, pretrained=False, use_mil=False, num_patches=8):
        super().__init__()
        self.use_mil = bool(use_mil)
        self.num_patches = int(num_patches)
        self.task_embed = TaskEmbedding()  # expects to exist in scope

        # Create ViT backbone (try with pretrained flag, fallback to pretrained=False)
        try:
            self.backbone = create_model('vit_base_patch16_224', pretrained=pretrained)
            logger.info(f"✓ ViT backbone created (pretrained={pretrained})")
        except Exception as e:
            logger.warning(f"ViT create_model(pretrained={pretrained}) failed ({e}) -> retrying with pretrained=False")
            self.backbone = create_model('vit_base_patch16_224', pretrained=False)

        # Remove final classification head (support common naming)
        if hasattr(self.backbone, "head"):
            self.backbone.head = nn.Identity()
        elif hasattr(self.backbone, "fc"):
            self.backbone.fc = nn.Identity()
        elif hasattr(self.backbone, "classifier"):
            self.backbone.classifier = nn.Identity()

        # Infer ViT embedding dim robustly
        vit_feat_dim = getattr(self.backbone, 'embed_dim', None) \
                       or getattr(self.backbone, 'num_features', None) \
                       or 768
        self.vit_feat_dim = int(vit_feat_dim)

        # MIL aggregator (if requested) — uses runtime vit_feat_dim
        if self.use_mil:
            self.mil_aggregator = MILAggregator(
                feature_dim=self.vit_feat_dim,
                num_patches=self.num_patches,
                dropout=0.2
            )

        # Regression head: use runtime embedding dim + task embedding dim (64)
        self.regression_head = nn.Sequential(
            nn.Linear(self.vit_feat_dim + 64, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

        logger.info(f"VisionTransformerModel initialized (use_mil={self.use_mil}, num_patches={self.num_patches}, vit_feat_dim={self.vit_feat_dim})")

    def forward(self, x, target_idx=None, return_features=False):
        """
        Args:
            x: torch.Tensor
               - if use_mil=False: shape [B, 3, H, W]
               - if use_mil=True:  shape [B*P, 3, H, W] where P = self.num_patches
            target_idx: tensor/list/ndarray of target indices (image-level), length B (preferred)
            return_features: bool - if True returns features (no regression head applied)
        Returns:
            - if return_features True:
                * use_mil=False -> (B, D)
                * use_mil=True  -> (B*P, D)
            - else:
                * use_mil=False -> (B,) regression outputs
                * use_mil=True  -> (B,) regression outputs
        """
        # Backbone forward
        features = self.backbone(x)  # many vit backbones return shape [N, 1, D] or [N, D]

        # Normalize feature shape: if backbone returns sequence-like [N, seq, D], pick CLS token position 0
        if features.ndim == 3:
            # e.g. [N, 1, D] or [N, S, D] -> take first token
            features = features[:, 0, :]  # [N, D]
        elif features.ndim == 4:
            # unlikely for ViT, but flatten safely
            features = features.view(features.size(0), -1)

        # If return_features requested and not using MIL: return per-batch features
        if return_features and not self.use_mil:
            # features is [B, D] expected
            return features

        # Ensure features is 2D [N, D]
        assert features.ndim == 2, f"Backbone produced unexpected feature dims: {features.shape}"

        # Ensure target_idx is a LongTensor on same device when used for embedding
        if target_idx is not None:
            if not torch.is_tensor(target_idx):
                target_idx = torch.tensor(target_idx, dtype=torch.long, device=features.device)
            else:
                target_idx = target_idx.long().to(features.device)

        # Non-MIL mode: treat features as per-image features (batch size = features.size(0))
        if not self.use_mil:
            B = features.size(0)
            # prepare task embedding: expects one index per image in batch
            if target_idx is not None:
                # If user passed per-patch indices accidentally, try to reduce to per-image
                if target_idx.dim() > 1:
                    target_idx = target_idx.view(-1)[:B]
                # if length mismatches, try to broadcast or slice
                if target_idx.numel() != B:
                    if target_idx.numel() < B:
                        # extend with zeros (warning)
                        pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                        target_idx = torch.cat([target_idx, pad], dim=0)
                    else:
                        target_idx = target_idx[:B]
                task_emb = self.task_embed(target_idx)
            else:
                task_emb = torch.zeros(B, 64, device=features.device)

            combined = torch.cat([features, task_emb], dim=-1)  # [B, D+64]
            out = self.regression_head(combined).squeeze(-1)     # [B]
            return out

        # MIL mode: aggregate patches
        # If return_features True: return per-patch features (useful to build MIL losses)
        if return_features and self.use_mil:
            return features  # [B*P, D]

        # EXPECT features is [N, D] where N = B * P
        N = features.size(0)
        assert N % self.num_patches == 0, f"Number of feature rows ({N}) is not divisible by num_patches ({self.num_patches})"
        B = N // self.num_patches
        C = features.size(-1)
        # Reshape into [B, P, D]
        patches = features.view(B, self.num_patches, C).contiguous()  # [B, P, D]

        # Aggregate patches to image-level embedding
        aggregated = self.mil_aggregator(patches)  # [B, D]

        # Prepare task embedding for B images; ensure device/dtype correctness
        if target_idx is not None:
            # If target_idx was per-patch, convert to per-image by taking every num_patches-th element
            if target_idx.numel() == N:
                target_idx_img = target_idx.view(B, self.num_patches)[:, 0]
            elif target_idx.numel() == B:
                target_idx_img = target_idx
            elif target_idx.numel() < B:
                # pad with zeros
                pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                target_idx_img = torch.cat([target_idx, pad], dim=0)
            else:
                target_idx_img = target_idx[:B]
            target_idx_img = target_idx_img.long().to(aggregated.device)
            task_emb = self.task_embed(target_idx_img)
        else:
            task_emb = torch.zeros(B, 64, device=aggregated.device)

        combined = torch.cat([aggregated, task_emb], dim=-1)  # [B, D+64]
        predictions = self.regression_head(combined).squeeze(-1)  # [B]

        return predictions



class ResNet50Model(nn.Module):
    """
    ResNet50 regression model with optional MIL aggregation and task conditioning.

    - If use_mil=False: expects x shaped [B, 3, H, W] and returns [B] regression outputs.
    - If use_mil=True: expects x shaped [B*P, 3, H, W] (patches grouped per-image) and returns [B] outputs.
    - return_features: when True:
        * if use_mil=False -> returns backbone features [B, D]
        * if use_mil=True  -> returns per-patch features [B*P, D]
    """
    def __init__(self, pretrained=False, use_mil=False, num_patches=8):
        super().__init__()
        self.use_mil = bool(use_mil)
        self.num_patches = int(num_patches)
        self.task_embed = TaskEmbedding()

        # Load ResNet backbone (try pretrained flag, fallback to pretrained=False)
        try:
            self.backbone = resnet50(pretrained=pretrained)
            logger.info(f"✓ ResNet50 backbone created (pretrained={pretrained})")
        except Exception as e:
            logger.warning(f"ResNet50 create failed ({e}) -> retrying with pretrained=False")
            self.backbone = resnet50(pretrained=False)

        # Remove final fc/classifier so backbone returns embeddings
        # Many resnet implementations have `.fc` as final; replace with Identity
        if hasattr(self.backbone, "fc"):
            self.backbone.fc = nn.Identity()

        # Attempt to infer output feature dim robustly
        out_dim = getattr(self.backbone, "num_features", None) \
                  or getattr(self.backbone, "out_features", None) \
                  or 2048
        self.out_dim = int(out_dim)

        # MIL aggregator uses runtime out_dim
        if self.use_mil:
            self.mil_aggregator = MILAggregator(feature_dim=self.out_dim, num_patches=self.num_patches, dropout=0.2)

        # Regression head uses runtime out_dim + task embedding (64)
        self.regression_head = nn.Sequential(
            nn.Linear(self.out_dim + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

        logger.info(f"ResNet50Model initialized (use_mil={self.use_mil}, num_patches={self.num_patches}, out_dim={self.out_dim})")

    def _normalize_target_idx(self, target_idx, features_rows):
        """
        Ensure target_idx is LongTensor on same device and aligns to image-level B.
        features_rows: number of rows from backbone (N), equals B when non-MIL, or B*P when MIL.
        """
        if target_idx is None:
            return None

        if not torch.is_tensor(target_idx):
            target_idx = torch.tensor(target_idx, dtype=torch.long, device=next(self.parameters()).device)
        else:
            target_idx = target_idx.long().to(next(self.parameters()).device)

        return target_idx

    def forward(self, x, target_idx=None, return_features=False):
        """
        x: [B,3,H,W] (non-MIL) OR [B*P,3,H,W] (MIL)
        target_idx: optional indices (per-image preferred)
        """
        features = self.backbone(x)
        # backbone output may be: [N, C, 1, 1] (if conv output), [N, C] or [N, S, C]
        if features.ndim == 4:
            # [N, C, 1, 1] -> flatten to [N, C]
            features = torch.nn.functional.adaptive_avg_pool2d(features, 1)
            features = features.view(features.size(0), -1)
        elif features.ndim == 3:
            # e.g. [N, 1, C] or [N, S, C] -> take first token (safe)
            features = features[:, 0, :]

        assert features.ndim == 2, f"ResNet backbone returned unexpected feature tensor shape {features.shape}"

        # Normalize target_idx to tensor on the right device
        target_idx = self._normalize_target_idx(target_idx, features.size(0))

        # Non-MIL mode: per-image features
        if not self.use_mil:
            B = features.size(0)
            if target_idx is not None:
                # attempt to align length
                if target_idx.numel() != B:
                    if target_idx.numel() < B:
                        pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                        target_idx = torch.cat([target_idx, pad], dim=0)
                    else:
                        target_idx = target_idx[:B]
                target_idx_img = target_idx
                task_emb = self.task_embed(target_idx_img)
            else:
                task_emb = torch.zeros(B, 64, device=features.device)

            combined = torch.cat([features, task_emb], dim=-1)
            out = self.regression_head(combined).squeeze(-1)
            if return_features:
                return features
            return out

        # MIL mode
        if return_features and self.use_mil:
            return features  # per-patch features [B*P, D]

        N = features.size(0)
        assert N % self.num_patches == 0, f"Number of rows ({N}) not divisible by num_patches ({self.num_patches})"
        B = N // self.num_patches
        C = features.size(-1)
        patches = features.view(B, self.num_patches, C).contiguous()  # [B, P, D]
        aggregated = self.mil_aggregator(patches)  # [B, D]

        # prepare image-level target_idx
        if target_idx is not None:
            if target_idx.numel() == N:
                target_idx_img = target_idx.view(B, self.num_patches)[:, 0]
            elif target_idx.numel() == B:
                target_idx_img = target_idx
            elif target_idx.numel() < B:
                pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                target_idx_img = torch.cat([target_idx, pad], dim=0)
            else:
                target_idx_img = target_idx[:B]
            target_idx_img = target_idx_img.long().to(aggregated.device)
            task_emb = self.task_embed(target_idx_img)
        else:
            task_emb = torch.zeros(B, 64, device=aggregated.device)

        combined = torch.cat([aggregated, task_emb], dim=-1)
        preds = self.regression_head(combined).squeeze(-1)
        return preds


class DenseNet121Model(nn.Module):
    """
    DenseNet121 regression model with optional MIL aggregation and task conditioning.

    Same interface as ResNet50Model.
    """
    def __init__(self, pretrained=False, use_mil=False, num_patches=8):
        super().__init__()
        self.use_mil = bool(use_mil)
        self.num_patches = int(num_patches)
        self.task_embed = TaskEmbedding()

        # Load DenseNet backbone
        try:
            self.backbone = densenet121(pretrained=pretrained)
            logger.info(f"✓ DenseNet121 backbone created (pretrained={pretrained})")
        except Exception as e:
            logger.warning(f"DenseNet121 create failed ({e}) -> retrying with pretrained=False")
            self.backbone = densenet121(pretrained=False)

        # Remove classifier head if present
        if hasattr(self.backbone, "classifier"):
            self.backbone.classifier = nn.Identity()

        # infer out_dim robustly
        out_dim = getattr(self.backbone, "num_features", None) \
                  or getattr(self.backbone, "out_features", None) \
                  or 1024
        self.out_dim = int(out_dim)

        if self.use_mil:
            self.mil_aggregator = MILAggregator(feature_dim=self.out_dim, num_patches=self.num_patches, dropout=0.2)

        self.regression_head = nn.Sequential(
            nn.Linear(self.out_dim + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

        logger.info(f"DenseNet121Model initialized (use_mil={self.use_mil}, num_patches={self.num_patches}, out_dim={self.out_dim})")

    def _normalize_target_idx(self, target_idx):
        if target_idx is None:
            return None
        if not torch.is_tensor(target_idx):
            target_idx = torch.tensor(target_idx, dtype=torch.long, device=next(self.parameters()).device)
        else:
            target_idx = target_idx.long().to(next(self.parameters()).device)
        return target_idx

    def forward(self, x, target_idx=None, return_features=False):
        features = self.backbone(x)

        # DenseNet may return [N, C, 1, 1] or [N, C] or [N, S, C], normalize
        if features.ndim == 4:
            features = torch.nn.functional.adaptive_avg_pool2d(features, 1)
            features = features.view(features.size(0), -1)
        elif features.ndim == 3:
            features = features[:, 0, :]

        assert features.ndim == 2, f"DenseNet backbone returned unexpected features shape: {features.shape}"

        target_idx = self._normalize_target_idx(target_idx)

        if not self.use_mil:
            B = features.size(0)
            if target_idx is not None:
                if target_idx.numel() != B:
                    if target_idx.numel() < B:
                        pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                        target_idx = torch.cat([target_idx, pad], dim=0)
                    else:
                        target_idx = target_idx[:B]
                task_emb = self.task_embed(target_idx)
            else:
                task_emb = torch.zeros(B, 64, device=features.device)

            combined = torch.cat([features, task_emb], dim=-1)
            out = self.regression_head(combined).squeeze(-1)
            if return_features:
                return features
            return out

        # MIL mode
        if return_features and self.use_mil:
            return features

        N = features.size(0)
        assert N % self.num_patches == 0, f"Rows ({N}) not divisible by num_patches ({self.num_patches})"
        B = N // self.num_patches
        C = features.size(-1)
        patches = features.view(B, self.num_patches, C).contiguous()
        aggregated = self.mil_aggregator(patches)

        if target_idx is not None:
            if target_idx.numel() == N:
                target_idx_img = target_idx.view(B, self.num_patches)[:, 0]
            elif target_idx.numel() == B:
                target_idx_img = target_idx
            elif target_idx.numel() < B:
                pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=features.device)
                target_idx_img = torch.cat([target_idx, pad], dim=0)
            else:
                target_idx_img = target_idx[:B]
            target_idx_img = target_idx_img.long().to(aggregated.device)
            task_emb = self.task_embed(target_idx_img)
        else:
            task_emb = torch.zeros(B, 64, device=aggregated.device)

        combined = torch.cat([aggregated, task_emb], dim=-1)
        preds = self.regression_head(combined).squeeze(-1)
        return preds


class XGBoostTabularModel:
    """
    XGBoost model for tabular features.
    Trains separate XGBoost models for each biomass target type.
    
    FIXED IMPROVEMENTS:
    - Added regularization: reg_alpha, reg_lambda, min_child_weight, gamma
    - Added early stopping for validation-based training
    - FIXED: Proper train/val split to prevent data contamination
    - Better hyperparameters for noisy tabular features
    - More conservative settings to prevent overfitting
    """

    def __init__(self, lookups_dict, targets=None):
        """
        Initialize XGBoost models for each target.

        Args:
            lookups_dict: Dictionary with STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS
            targets: optional list of target names to initialize (defaults to known list)
        """
        STATE_COLS = lookups_dict.get('STATE_COLS', [])
        SPECIES_COLS = lookups_dict.get('SPECIES_COLS', [])
        TARGET_TYPE_COLS = lookups_dict.get('TARGET_TYPE_COLS', [])

        self.models = {}
        self.feature_names = (
            ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month', 'DayOfYear', 'Quarter']
            + STATE_COLS + SPECIES_COLS + TARGET_TYPE_COLS
        )

        logger.info("✓ XGBoost initialized (tree-based model)")
        logger.info(f"  - Features: {len(self.feature_names)} ({', '.join(self.feature_names[:3])}...)")
        logger.info("  - Separate model for each biomass target type")
        logger.info("  - WITH regularization and early stopping")
        logger.info("  - FIXED: Proper train/val split to prevent overfitting")

        if targets is None:
            targets = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']

        # Initialize one model per target
        for target_name in targets:
            self.models[target_name] = xgb.XGBRegressor(
                n_estimators=CONFIG.get('xgb_n_estimators', 1000),              # Reduced from 1500
                max_depth=CONFIG.get('xgb_max_depth', 4),                       # Reduced from 5
                learning_rate=CONFIG.get('xgb_learning_rate', 0.05),            # Increased from 0.015
                subsample=CONFIG.get('xgb_subsample', 0.75),                    # Reduced from 0.8
                colsample_bytree=CONFIG.get('xgb_colsample_bytree', 0.75),      # Reduced from 0.8
                min_child_weight=CONFIG.get('xgb_min_child_weight', 5),         # Increased from 4
                reg_alpha=CONFIG.get('xgb_reg_alpha', 0.5),                     # Increased from 0.3
                reg_lambda=CONFIG.get('xgb_reg_lambda', 1.5),                   # Increased from 1.0
                gamma=CONFIG.get('xgb_gamma', 1.0),                             # Increased from 0.8
                random_state=CONFIG.get('random_seed', 42),
                tree_method='gpu_hist' if torch.cuda.is_available() else 'auto',
                verbosity=0
            )

    def fit(self, X, y, target_names):
        """
        Train XGBoost models with early stopping.
        
        FIXED: Implements proper train/eval split to prevent data contamination.
        The key fix is that we create eval sets from training data only,
        ensuring that validation data used for early stopping is completely
        separate from any external evaluation or meta-learner training.

        Args:
            X (np.array or pd.DataFrame): Tabular features (n_samples, len(feature_names))
            y (np.array): Target values (n_samples,)
            target_names (np.array): Target type names (n_samples,)
        """
        from sklearn.model_selection import train_test_split
        
        # accept DataFrame or numpy; ensure numpy for xgboost .fit
        if hasattr(X, "values"):
            X_arr = X.values
        else:
            X_arr = np.asarray(X)

        for target_type, model in self.models.items():
            mask = (np.array(target_names) == target_type)
            n = int(mask.sum())
            if n > 0:
                X_target = X_arr[mask]
                y_target = np.asarray(y)[mask]
                
                # Use early stopping if enough samples
                if n > 50:  # Increased threshold from 30 to 50
                    # FIXED: Create internal train/eval split for early stopping
                    # This is used ONLY for early stopping, not for external validation
                    X_train, X_eval, y_train, y_eval = train_test_split(
                        X_target, y_target,
                        test_size=0.15,  # Reduced from 0.2 to have larger training set
                        random_state=CONFIG.get('random_seed', 42)
                    )
                    
                    model.fit(
                        X_train, y_train,
                        eval_set=[(X_eval, y_eval)],
                        eval_metric='rmse',  # Explicit metric specification
                        early_stopping_rounds=CONFIG.get('xgb_early_stopping_rounds', 50),  # Reduced from 75
                        verbose=False
                    )
                    
                    # Log diagnostics to monitor overfitting
                    best_iter = model.best_iteration
                    best_score = model.best_score
                    logger.info(f"✓ XGBoost trained for {target_type} ({n} samples, early stopping)")
                    logger.info(f"  - Best iteration: {best_iter}, Best RMSE: {best_score:.4f}")
                    logger.info(f"  - Train samples: {len(X_train)}, Eval samples: {len(X_eval)}")
                    
                    # Alert if using too many trees (potential overfitting sign)
                    if best_iter > 800:
                        logger.warning(f"  ⚠️ WARNING: Used {best_iter} trees - monitor for overfitting!")
                else:
                    # Train without early stopping if not enough samples
                    model.fit(X_target, y_target)
                    logger.info(f"✓ XGBoost trained for {target_type} ({n} samples, no early stopping)")
            else:
                logger.info(f"→ No samples for {target_type}; skipping training")

    def predict(self, X, target_name):
        """
        Make predictions.

        Args:
            X (np.array or pd.DataFrame): Tabular features (n_samples, len(feature_names))
            target_name (str): Target type

        Returns:
            np.array: Predictions
        """
        if target_name not in self.models:
            raise ValueError(f"Model for target '{target_name}' not found. Available: {list(self.models.keys())}")

        if hasattr(X, "values"):
            X_arr = X.values
        else:
            X_arr = np.asarray(X)

        return self.models[target_name].predict(X_arr)

    def save(self, path):
        """Save models to a pickle file (only models dict)."""
        with open(path, 'wb') as f:
            pickle.dump({'models': self.models, 'feature_names': self.feature_names}, f)
        logger.info(f"✓ XGBoost models saved to {path}")

    def load(self, path):
        """Load models from a pickle file."""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        self.models = data.get('models', {})
        self.feature_names = data.get('feature_names', self.feature_names)
        logger.info(f"✓ XGBoost models loaded from {path}")

        
class MetaLearnerModel(nn.Module):
    """
    ✓ Refactored for small data (~1700 samples)
    ✓ Symmetric treatment of all 5 inputs: [ViT, ResNet, DenseNet, XGBoost, Smart_Ensemble]
    ✓ Task-conditioned gating based on research in Mixture-of-Experts
    ✓ Minimal overfitting risk through regularization
    ✓ Backward compatible with existing training loop
    """
    def __init__(self, dropout_rate=0.1, pretrained=False, num_base_preds=5):
        super().__init__()
        self.num_base_preds = int(num_base_preds)  # Keep for backward compat
        self.pretrained = pretrained
        
        # ============ TASK EMBEDDING ============
        # Reduced from 64 → 32 to prevent overfitting on small data
        self.task_embed = TaskEmbedding(num_targets=5, embed_dim=32)
        
        # ============ GATING NETWORK (Task-Conditioned Expert Weighting) ============
        # Takes task context (32D) and outputs weights for ALL 5 models
        # Research shows this is more stable than arbitrary slicing
        self.gate = nn.Sequential(
            nn.Linear(32, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 16),  # Bottleneck to regularize
            nn.GELU(),
            nn.Dropout(dropout_rate * 0.8),
            nn.Linear(16, self.num_base_preds)  # Output: 1 weight per model
        )
        
        # ============ TRUNK (Residual Correction) ============
        # Small correction network (not the main predictor)
        # Input: 5 (raw preds) + 1 (weighted ensemble) + 32 (task) = 38
        self.trunk = nn.Sequential(
            nn.Linear(5 + 1 + 32, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 16),
            nn.GELU(),
            nn.Dropout(dropout_rate * 0.8),
            nn.Linear(16, 1)  # Output: correction scalar
        )
        
        self.register_buffer('_eps', torch.tensor(1e-6))
    
    def forward(self, predictions, target_idx):
        """
        Args:
            predictions: [B, 5] - predictions from 5 base models
                        Index 0: ViT
                        Index 1: ResNet
                        Index 2: DenseNet
                        Index 3: XGBoost
                        Index 4: Smart_Ensemble (Inverse-Variance Weighted)
            target_idx: [B] - target type indices (0-4)
        
        Returns:
            [B] - final meta-learner predictions
        """
        preds = predictions.float()
        
        if preds.ndim != 2:
            raise ValueError(f"predictions must be 2D [B, num_preds], got {preds.shape}")
        
        B = preds.shape[0]
        
        # ========== DEVICE & SHAPE HANDLING (Backward Compatible) ==========
        if not torch.is_tensor(target_idx):
            target_idx = torch.tensor(target_idx, dtype=torch.long, device=preds.device)
        else:
            target_idx = target_idx.long().to(preds.device)
        
        if target_idx.numel() != B:
            if target_idx.numel() < B:
                pad = torch.zeros(B - target_idx.numel(), dtype=torch.long, device=preds.device)
                target_idx = torch.cat([target_idx, pad], dim=0)
            else:
                target_idx = target_idx[:B]
        
        # ========== CORE LOGIC (FIXED) ==========
        
        # 1. Get Task Context
        task_embedding = self.task_embed(target_idx)  # [B, 32]
        
        # 2. Compute Dynamic Weights for ALL 5 Models (Symmetric)
        # This is the KEY FIX: Use softmax so weights sum to 1.0 and all models compete equally
        gate_logits = self.gate(task_embedding)  # [B, 5]
        gate_weights = torch.softmax(gate_logits, dim=1)  # [B, 5] - sums to 1
        
        # 3. Weighted Ensemble Prediction (Symmetric)
        # Now XGBoost (index 3) and Smart_Ensemble (index 4) have equal opportunity to be weighted
        weighted_ensemble = torch.sum(gate_weights * preds, dim=1, keepdim=True)  # [B, 1]
        
        # 4. Residual Correction (Reduced Size)
        # The trunk sees the raw votes + the weighted ensemble + task context
        # It learns a small CORRECTION, not the full prediction
        trunk_input = torch.cat([preds, weighted_ensemble, task_embedding], dim=1)  # [B, 38]
        correction = self.trunk(trunk_input).squeeze(-1)  # [B]
        
        # 5. Final Output = Weighted Ensemble + Correction
        final = weighted_ensemble.squeeze(-1) + correction  # [B]
        
        return final



class SpeciesClassificationViT(nn.Module):
    """
    Vision Transformer for species classification with optional MIL aggregation.

    Inputs:
      - x: [B*P, 3, 224, 224] patches (P = num_patches, default 8)

    Modes:
      - use_mil = False -> output per-patch logits: [B*P, num_classes]
      - use_mil = True  -> aggregate patches -> output image-level logits: [B, num_classes]
    """
    def __init__(self, num_species_classes=16, pretrained=False, use_mil=False, num_patches=8):
        super().__init__()

        # create backbone (honour pretrained flag)
        self.backbone = create_model('vit_base_patch16_224', pretrained=pretrained)

        # remove default head (return embedding vectors)
        if hasattr(self.backbone, "head"):
            self.backbone.head = nn.Identity()
        elif hasattr(self.backbone, "fc"):
            self.backbone.fc = nn.Identity()
        elif hasattr(self.backbone, "classifier"):
            self.backbone.classifier = nn.Identity()

        # infer embedding dim robustly
        in_dim = getattr(self.backbone, 'num_features', None) \
                 or getattr(self.backbone, 'embed_dim', None) \
                 or 768
        self.in_dim = int(in_dim)

        self.num_classes = int(num_species_classes)
        self.use_mil = bool(use_mil)
        self.num_patches = int(num_patches)

        # classification head (works on either aggregated image embedding or per-patch embedding)
        self.classification_head = nn.Sequential(
            nn.Linear(self.in_dim, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, self.num_classes)
        )

        # optional MIL aggregator
        if self.use_mil:
            self.mil_aggregator = MILAggregator(feature_dim=self.in_dim, num_patches=self.num_patches, dropout=0.2)

        self.species_list = FIXED_SPECIES_LIST
        assert len(self.species_list) == self.num_classes, (
            f"species_list length ({len(self.species_list)}) must equal num_species_classes ({self.num_classes})"
        )

        logger.info("✓ Species Classification ViT initialized")
        logger.info(f" - use_mil={self.use_mil}, num_patches={self.num_patches}, in_dim={self.in_dim}, num_classes={self.num_classes}")

    def forward(self, x, return_features=False):
        """
        Args:
            x: [N, 3, H, W] where N = B*P (patches)
            return_features: if True (during training), return features before classification
        
        Returns:
            if use_mil and return_features: [B*P, in_dim] features (for training)
            if use_mil and not return_features: [B, num_classes] logits (for inference)
            else: [N, num_classes] logits
        """
        features = self.backbone(x)
        if features.ndim == 3:
            features = features[:, 0, :]
        
        if not self.use_mil:
            logits = self.classification_head(features)
            return logits
        
        if return_features:
            # Return raw features for training
            return features  # [B*P, in_dim]
        else:
            # Return fully aggregated logits for inference
            N = features.size(0)
            assert N % self.num_patches == 0
            aggregated = self.mil_aggregator(features)
            logits = self.classification_head(aggregated)
            return logits


    def predict_species(self, x, with_probs=False):
        """
        Convenience: run forward and return predicted index and name(s).

        If model.use_mil=True and x contains patches for multiple images, x must be arranged
        such that patches for each image are consecutive and len(x) % num_patches == 0.
        """
        self.eval()
        with torch.no_grad():
            logits = self.forward(x)  # either [N, C] or [B, C]
            if logits.ndim == 2:
                probs = F.softmax(logits, dim=1)
                preds = probs.argmax(dim=1).cpu().numpy().astype(int)
            else:
                # fallback in case of unexpected shape
                logits = logits.view(-1, self.num_classes)
                probs = FF.softmax(logits, dim=1)
                preds = probs.argmax(dim=1).cpu().numpy().astype(int)

        names = [self.species_list[p] if 0 <= p < len(self.species_list) else "Unknown" for p in preds]
        if with_probs:
            return preds, names, probs.cpu().numpy()
        return preds, names

    def save_backbone_weights(self, path):
        try:
            torch.save(self.backbone.state_dict(), path)
            logger.info(f"✓ Saved Species ViT backbone weights to {path}")
        except Exception as e:
            logger.warning(f"Could not save Species ViT backbone weights: {e}")

    def load_backbone_weights(self, path, strict=False, map_location=None):
        try:
            state = torch.load(path, map_location=map_location)
            self.backbone.load_state_dict(state, strict=strict)
            logger.info(f"✓ Loaded Species ViT backbone weights from {path}")
        except FileNotFoundError:
            logger.error(f"Backbone weights not found: {path}")
            raise
        except Exception as e:
            logger.warning(f"Failed to load Species ViT backbone weights from {path}: {e}")



def compute_and_save_ndvi_stats_with_sklearn(
    train_df,
    base_dir= RAW_DATA_DIR,
    image_path_col="image_path",
    species_col="Species",
    ndvi_col_candidates=('Pre_GSHH_NDVI','ndvi', 'NDVI', 'vndvi'),
    out_path= OUTPUTS_DIR / 'ndvidata.csv',
    compute_rgb_indices=True,
    resize_to=(512, 512),
    min_samples_for_ml=20,
    max_images=None,
    force_recompute=False,
    random_state=42,
    model_params=None
):
    """
    Compute per-image RGB indices and (optionally) train per-species sklearn regressors
    to predict the provided NDVI column. Aggregate per-species stats and save CSV.
    """
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    if out_path.exists() and not force_recompute:
        logger.info("Loading existing NDVI stats from %s", out_path)
        return pd.read_csv(out_path)

    # validate columns
    if image_path_col not in train_df.columns:
        raise KeyError(f"train_df missing column '{image_path_col}'")
    if species_col not in train_df.columns:
        raise KeyError(f"train_df missing column '{species_col}'")

    # find ndvi column if any
    ndvi_col = None
    for c in ndvi_col_candidates:
        if c in train_df.columns:
            ndvi_col = c
            break
    if ndvi_col is None:
        logger.info("No NDVI column found in train_df; ML regression will be skipped, we will use computed indices only.")

    # model default
    if model_params is None:
        model_params = {'n_estimators': 100, 'max_depth': 10, 'random_state': random_state, 'n_jobs': -1}

    # subset rows
    rows = train_df[[image_path_col, species_col]].dropna().reset_index(drop=True)
    if max_images:
        rows = rows.iloc[:max_images]
        logger.info("Limiting to first %d images", max_images)
    logger.info("Processing %d images...", len(rows))

    # storage per-image
    per_image = []
    # iterate
    for idx, row in tqdm(rows.iterrows(), total=len(rows), desc="Extract features"):
        rel = row[image_path_col]
        species = row[species_col]
        full_path = os.path.join(base_dir, rel)

        if not os.path.exists(full_path):
            logger.warning("Missing image: %s", full_path)
            continue

        try:
            img = Image.open(full_path).convert("RGB")
            if resize_to is not None:
                img = img.resize(resize_to, Image.BILINEAR)
            img_np = np.asarray(img, dtype=np.float32)
            if img_np.max() > 1.0:
                img_np /= 255.0

            R = img_np[:, :, 0]
            G = img_np[:, :, 1]
            B = img_np[:, :, 2]

            # features
            # vNDVI
            denom = (G + R)
            denom_safe = np.where(np.isclose(denom, 0.0), 1e-8, denom)
            vndvi_arr = (G - R) / denom_safe
            vndvi_mean = float(np.nanmean(vndvi_arr))

            # rgbvi
            if compute_rgb_indices:
                denom_rgbvi = (G*G + R*B)
                denom_rgbvi_safe = np.where(np.isclose(denom_rgbvi, 0.0), 1e-8, denom_rgbvi)
                rgbvi_arr = (G*G - R*B) / denom_rgbvi_safe
                rgbvi_mean = float(np.nanmean(rgbvi_arr))
                denom_vari = (G + R - B)
                denom_vari_safe = np.where(np.isclose(denom_vari, 0.0), 1e-8, denom_vari)
                vari_arr = (G - R) / denom_vari_safe
                vari_mean = float(np.nanmean(vari_arr))
            else:
                rgbvi_mean = np.nan
                vari_mean = np.nan

            mean_r = float(np.nanmean(R))
            mean_g = float(np.nanmean(G))
            mean_b = float(np.nanmean(B))
            green_intensity = mean_g

            # assemble features
            features = {
                "image_path": full_path,
                "species": species,
                "vndvi_raw": vndvi_mean,
                "vndvi_clamped": float(np.clip(vndvi_mean, 0.0, 1.0)),
                "rgbvi": float(rgbvi_mean) if compute_rgb_indices else np.nan,
                "vari": float(vari_mean) if compute_rgb_indices else np.nan,
                "mean_r": mean_r,
                "mean_g": mean_g,
                "mean_b": mean_b,
                "green_intensity": green_intensity
            }

            # true ndvi if present in train_df
            if ndvi_col is not None:
                # attempt to locate matching row in train_df to get true ndvi (by image_path)
                # We assume train_df may have duplicate image paths; pick the first
                try:
                    # find exact row in the original train_df (iloc of matching relative path)
                    match = train_df[train_df[image_path_col] == rel]
                    if len(match) > 0 and ndvi_col in match.columns:
                        features["true_ndvi"] = float(match[ndvi_col].iloc[0])
                    else:
                        features["true_ndvi"] = np.nan
                except Exception:
                    features["true_ndvi"] = np.nan
            else:
                features["true_ndvi"] = np.nan

            per_image.append(features)

        except Exception as e:
            logger.warning("Error processing %s: %s", full_path, e)

    per_image_df = pd.DataFrame(per_image)
    if per_image_df.empty:
        logger.error("No valid images processed.")
        return pd.DataFrame()

    # Prepare output structure
    species_rows = []

    # group by species and optionally train a model per species
    grouped = per_image_df.groupby("species")
    for species, g in grouped:
        g = g.reset_index(drop=True)
        sample_count = len(g)
        logger.info("Species='%s': samples=%d", species, sample_count)

        # default per-image ndvi estimate (vndvi_clamped)
        g["pred_ndvi_baseline"] = g["vndvi_clamped"].values

        used_model = False
        model_score = None

        # train ML only if we have true_ndvi and enough samples
        if ndvi_col is not None and g["true_ndvi"].notna().sum() >= min_samples_for_ml:
            # features to use
            feat_cols = ["vndvi_raw", "rgbvi", "vari", "mean_r", "mean_g", "mean_b", "green_intensity"]
            # drop rows with NaN in features or target
            df_train = g.dropna(subset=["true_ndvi"] + feat_cols)
            if len(df_train) >= min_samples_for_ml:
                X = df_train[feat_cols].astype(float).values
                y = df_train["true_ndvi"].astype(float).values

                # quick train/val split to guard overfitting
                X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.15, random_state=random_state)

                try:
                    rf = RandomForestRegressor(**model_params)
                    rf.fit(X_tr, y_tr)
                    y_pred_val = rf.predict(X_val)
                    r2 = r2_score(y_val, y_pred_val)
                    mae = mean_absolute_error(y_val, y_pred_val)
                    model_score = {"r2_val": float(r2), "mae_val": float(mae)}
                    logger.info("  Trained RF (species=%s): r2=%.4f, mae=%.4f", species, r2, mae)

                    # predict for all rows of this species that have features
                    idx_has_feat = ~g[feat_cols].isnull().any(axis=1)
                    if idx_has_feat.sum() > 0:
                        X_all = g.loc[idx_has_feat, feat_cols].astype(float).values
                        preds_all = rf.predict(X_all)
                        g.loc[idx_has_feat, "pred_ndvi_ml"] = preds_all
                        # where ML preds exist prefer them
                        g["pred_ndvi_final"] = g["pred_ndvi_ml"].combine_first(g["pred_ndvi_baseline"])
                    else:
                        g["pred_ndvi_final"] = g["pred_ndvi_baseline"]

                    used_model = True
                except Exception as e:
                    logger.warning("  ML training failed for species %s: %s", species, e)
                    g["pred_ndvi_final"] = g["pred_ndvi_baseline"]
            else:
                logger.info("  Not enough clean rows for ML after dropna (%d < %d)", len(df_train), min_samples_for_ml)
                g["pred_ndvi_final"] = g["pred_ndvi_baseline"]
        else:
            # no NDVI column or not enough samples -> fallback to baseline
            g["pred_ndvi_final"] = g["pred_ndvi_baseline"]

        # enforce sane range
        g["pred_ndvi_final"] = g["pred_ndvi_final"].clip(0.0, 1.0)

        # compute aggregated stats
        nd_mean = float(g["pred_ndvi_final"].mean())
        nd_min = float(g["pred_ndvi_final"].min())
        nd_max = float(g["pred_ndvi_final"].max())

        species_rows.append({
            "species": species,
            "ndvi_mean": nd_mean,
            "ndvi_min": nd_min,
            "ndvi_max": nd_max,
            "sample_count": int(sample_count),
            "used_model": bool(used_model),
            "model_score": model_score
        })

    agg = pd.DataFrame(species_rows)
    if "model_score" in agg.columns:
        agg["model_score"] = agg["model_score"].apply(lambda x: "" if x is None else str(x))
    agg.to_csv(out_path, index=False, float_format="%.6f")
    logger.info("Saved NDVI stats to %s (rows=%d)", out_path, len(agg))
    return agg


            
logger.info("✓ Model Architectures-6 Models")

## 7: TRAINING FUNCTIONS

**Purpose**: Train all models with monitoring, checkpointing, and memory management.

### Deep Learning Training

**For each vision model** (ViT, ResNet, DenseNet):

```python
def train_deep_learning_model(
    model, train_loader, val_loader,
    model_name, learning_rate, num_epochs, device
)
```

**Training Loop**:
```
For each epoch:
  1. Set model to training mode
  2. Iterate through train_loader batches
     a. Forward pass: predictions = model(images, tabular)
     b. Compute loss: weighted_r2_loss(predictions, targets)
     c. Backward: loss.backward()
     d. Clip gradients: torch.nn.utils.clip_grad_norm_
     e. Update: optimizer.step()
     f. Clear gradients: optimizer.zero_grad()
  3. Validation pass on val_loader
     a. Set model to eval mode
     b. No gradient computation
     c. Compute R² metric
  4. Early stopping check: if no improvement for 20 epochs, stop
  5. Save checkpoint if validation R² improved
  6. Update learning rate (cosine schedule)
  7. Log metrics
```


**Memory Management**:
- Move model to device before training
- Clear gradients each step
- Periodic garbage collection
- Cleanup after training

### XGBoost Training

**Process**:
1. Create XGBRegressor with configuration
2. Train with eval_set for early stopping
3. Monitor validation R² each round
4. Stop if no improvement for 50 rounds
5. Return trained model + feature importance

**Output**: Binary pickle file with trained weights

### Meta-Learner Training

**Input Format**:
- train_pred: [n_train, 5] predictions from 5 base models
- train_tab: [n_train, 29] tabular features
- train_tgt: [n_train] ground truth targets

**Process**:
1. Concatenate predictions + tabular: [n, 34]
2. Create MetaLearner network
3. Train with weighted R² loss (100 epochs)
4. Early stopping if val R² doesn't improve
5. Save best checkpoint

**Output**: 
- All models trained and checkpointed
- Training/validation curves logged
- Best epoch per model documented

---

# Helper: compute per-image targets by averaging per-patch targets
def _image_targets_from_patch_targets(targets_per_patch, B, P, device):
    """
    targets_per_patch: tensor shape (N,) or (N,1) or numpy-like, with N = B*P
    returns: numpy array shape (B,) of per-image targets (mean across patches)
    """
    if not isinstance(targets_per_patch, torch.Tensor):
        targets_t = torch.tensor(np.asarray(targets_per_patch).ravel(), dtype=torch.float32, device=device)
    else:
        targets_t = targets_per_patch.view(-1).float().to(device)

    N = targets_t.numel()
    if N != B * P:
        # If it's already image-level (B), warn and coerce
        if N == B:
            return targets_t.view(B).detach().cpu().numpy()
        # As fallback, attempt safe truncation / padding
        if N > B * P:
            targets_t = targets_t[: B * P]
        else:
            # pad with mean value if fewer than expected
            pad_n = B * P - N
            pad_vals = targets_t.mean().repeat(pad_n).to(device) if N > 0 else torch.zeros(pad_n, device=device)
            targets_t = torch.cat([targets_t, pad_vals], dim=0)
    
    img_t = targets_t.view(B, P).mean(dim=1)
    return img_t.detach().cpu().numpy()


# Provided earlier but safe & identical implementation
def _sample_ids_to_image_ids(sample_ids, patches_per_image=8):
    """
    Robustly convert sample_ids (either image-level or patch-level) into
    an ordered list of image_ids (one per image in the batch).
    Uses run-length collapse of consecutive sample_ids so it works for:
      - image-level: ['IMG1__T','IMG2__T', ...] -> ['IMG1','IMG2']
      - patch-level:  ['IMG1__T', 'IMG1__T', ... (8x), 'IMG2__T', ...] -> ['IMG1','IMG2']
    """
    try:
        sids = [str(s) for s in sample_ids]
    except Exception:
        sids = list(sample_ids)

    if len(sids) == 0:
        return []

    img_ids = []
    prev = None
    for s in sids:
        img = str(s).split("__")[0]
        if img != prev:
            img_ids.append(img)
            prev = img
    return img_ids


def _collapse_target_names_by_sample_ids(sample_ids, target_names, patches_per_image):
    """
    Use sample_ids to deterministically collapse per-patch target_names to per-image names.
    Returns ordered list of image-level names (length inferred B).
    """
    # normalize
    try:
        sids = [str(s) for s in sample_ids]
    except Exception:
        sids = list(sample_ids)

    try:
        tns = [str(t) for t in target_names]
    except Exception:
        tns = list(target_names)

    if len(sids) != len(tns):
        # best-effort: if one is shorter/longer, fallback to simple truncation/tiling
        if len(tns) == 1:
            # single name -> repeat
            # infer B
            inferred_B = max(1, len(sids) // max(1, patches_per_image))
            return [tns[0]] * inferred_B
        # otherwise attempt to tile/truncate
        if len(sids) % patches_per_image == 0:
            # group by sids
            img_ids = _sample_ids_to_image_ids(sids, patches_per_image)
            img_to_name = {}
            for sid, tn in zip(sids, tns[:len(sids)]):
                img = sid.split("__")[0]
                if img not in img_to_name:
                    img_to_name[img] = tn
            return [img_to_name.get(img, "None") for img in img_ids]
        else:
            # fallback: flat unique over provided tns
            unique = []
            for t in tns:
                if t not in unique:
                    unique.append(t)
            return unique

    # normal case: len(sids) == len(tns)
    img_ids = _sample_ids_to_image_ids(sids, patches_per_image)
    # map first encountered patch -> name for each image (deterministic)
    img_to_name = {}
    for sid, tn in zip(sids, tns):
        img = sid.split("__")[0]
        if img not in img_to_name:
            img_to_name[img] = tn
    return [img_to_name.get(img, "None") for img in img_ids]


def _align_target_names(sample_ids, raw_target_names, pred_len, B, P):
    """
    Return aligned_target_names (list of strings) length == pred_len.
    Uses sample_ids to do deterministic grouping when possible.
    Cases:
     - pred_len == B (image-level predictions): return one name per image
     - pred_len == B*P (patch-level preds): return one name per patch
    """
    # Normalize raw_target_names into a list
    if isinstance(raw_target_names, torch.Tensor):
        try:
            tn_list = raw_target_names.tolist()
        except Exception:
            tn_list = [str(x) for x in raw_target_names.cpu().numpy().ravel()]
    elif isinstance(raw_target_names, (np.ndarray, tuple, list)):
        tn_list = list(raw_target_names)
    else:
        tn_list = [str(raw_target_names)]

    tn_list = [str(x) for x in tn_list]

    # If lengths already match desired, simply return truncated copy
    if len(tn_list) == pred_len:
        return tn_list[:pred_len]

    # If we have sample_ids, prefer deterministic collapse/expand
    if sample_ids is not None:
        try:
            collapsed = _collapse_target_names_by_sample_ids(sample_ids, tn_list, P)
            if pred_len == len(collapsed):  # image-level
                return collapsed
            elif pred_len == len(collapsed) * P:
                # expand image names to patch-level
                out = []
                for nm in collapsed:
                    out.extend([nm] * P)
                return out[:pred_len]
        except Exception:
            pass

    # Fallback heuristics:
    if pred_len == B and len(tn_list) == B * P:
        # collapse by taking first of each group
        return [tn_list[i * P] for i in range(B)]
    if pred_len == B * P and len(tn_list) == B:
        # expand by repeating each name P times
        out = []
        for nm in tn_list:
            out.extend([nm] * P)
        return out[:pred_len]
    # As last resort, tile/truncate
    if len(tn_list) == 0:
        return ["None"] * pred_len
    return (tn_list * ((pred_len // len(tn_list)) + 1))[:pred_len]



def train_deep_learning_model(model, train_loader, val_loader, model_name,
                               learning_rate, num_epochs, device):
    """
    Train a patch-based deep learning model.

    Keeps original variable names and overall logic, but:
      - Uses a safe builder for target_idx from tabular features
      - Fails fast on prediction/target length mismatches (logs and skips batch)
      - Ensures target_idx is LongTensor on same device as model
      - Preserves MIL / non-MIL modes and most original defensive behavior

    Args:
        model: torch.nn.Module (patch-based model). Expects signature model(images, target_idx, return_features=False)
        train_loader, val_loader: data loaders yielding batches in the same format used previously
        model_name: string (used for saving)
        learning_rate: float
        num_epochs: int
        device: torch.device or string
    Returns:
        history dict with train/val loss and R2 series
    """
    logger.info(f"\n{'='*80}")
    logger.info(f"TRAINING {model_name.upper()} (PATCH-ONLY, NO TABULAR)")
    logger.info(f"{'='*80}")

    use_mil = CONFIG.get("use_mil", False)
    patches_per_image = getattr(model, "num_patches", CONFIG.get("num_patches", 8))
    logger.info(f"MIL Mode: {'ENABLED' if use_mil else 'DISABLED'}")
    logger.info(f"Batch size: {CONFIG['batch_size']} images × {patches_per_image} patches")

    # normalize device
    try:
        device = device if isinstance(device, torch.device) else torch.device(device if device is not None else ("cuda" if torch.cuda.is_available() else "cpu"))
    except Exception:
        device = torch.device("cpu")

    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

    history = {'train_loss': [], 'val_loss': [], 'train_r2': [], 'val_r2': []}
    best_val_loss = float('inf')
    patience_counter = 0
    start_time = time.time()

    # Configurable target one-hot region (avoid hard-coded 25)
    TARGET_ONEHOT_START = CONFIG.get('target_onehot_start', 25)
    NUM_TARGET_ONEHOT = CONFIG.get('num_target_onehot', 5)

    def build_target_idx_from_tabular(tabular_tensor, N, B, P, want_image_level, device):
        """
        Robustly build target_idx (LongTensor) for model embedding lookup.
        - tabular_tensor: torch.Tensor shaped (N, D) or (B, D). May be on cpu.
        - N: number of rows in batch (patch-level)
        - B: number of images in batch
        - P: patches_per_image
        - want_image_level: True -> return length B. False -> return length N
        """
        if tabular_tensor is None:
            raw = torch.zeros(N, dtype=torch.long, device=device)
        else:
            try:
                tab = tabular_tensor
                if not torch.is_tensor(tab):
                    tab = torch.tensor(np.asarray(tab), device=device)
                else:
                    tab = tab.to(device)
                D = tab.size(1)
                if D >= (TARGET_ONEHOT_START + NUM_TARGET_ONEHOT):
                    raw = torch.argmax(tab[:, TARGET_ONEHOT_START: TARGET_ONEHOT_START + NUM_TARGET_ONEHOT].float(), dim=-1).long().to(device)
                else:
                    logger.debug(f"Tabular width {D} < required {TARGET_ONEHOT_START + NUM_TARGET_ONEHOT} for one-hot; using zeros for raw target idx")
                    raw = torch.zeros(tab.size(0), dtype=torch.long, device=device)
            except Exception as e:
                logger.warning(f"Failed to parse tabular for target_idx: {e}")
                raw = torch.zeros(N, dtype=torch.long, device=device)

        # Now unify length/shape
        cnt = raw.numel()
        if want_image_level:
            # target_idx length should be B
            if cnt == N:
                # assume patch-level -> take first patch per image
                try:
                    return raw.view(B, P)[:, 0].long().to(device)
                except Exception:
                    # fallback: trunc/pad
                    return raw[:B].long().to(device) if cnt >= B else raw.repeat((B // max(1, cnt)) + 1)[:B].long().to(device)
            elif cnt == B:
                return raw.long().to(device)
            else:
                if cnt > B:
                    return raw[:B].long().to(device)
                if cnt == 0:
                    return torch.zeros(B, dtype=torch.long, device=device)
                return raw.repeat((B // cnt) + 1)[:B].long().to(device)
        else:
            # want patch-level N
            if cnt == N:
                return raw.long().to(device)
            elif cnt == B:
                return raw.view(B, 1).repeat(1, P).view(-1).long().to(device)
            else:
                if cnt > N:
                    return raw[:N].long().to(device)
                if cnt == 0:
                    return torch.zeros(N, dtype=torch.long, device=device)
                return raw.repeat((N // cnt) + 1)[:N].long().to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_targets = []
        train_target_names = []

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [TRAIN]") as pbar:
            for batch_idx, batch_data in enumerate(pbar):
                # Unpack batch
                try:
                    if len(batch_data) == 7:
                        images, tabular, species_batch, target_idx_batch, targets_per_patch, sample_ids, target_names = batch_data
                    else:
                        raise ValueError(f"Unexpected batch length: {len(batch_data)}")
                except Exception as e:
                    logger.exception(f"Failed to unpack batch: {e}")
                    continue

                if targets_per_patch is None:
                    logger.debug("Skipping batch without targets.")
                    continue

                # Move tensors to device when possible
                try:
                    images = images.to(device)
                except Exception as e:
                    logger.warning(f"Failed to move images to device: {e}")

                try:
                    targets_per_patch = targets_per_patch.to(device).float()
                except Exception:
                    # allow numpy-like targets; coerce
                    try:
                        targets_per_patch = torch.tensor(np.asarray(targets_per_patch).ravel(), dtype=torch.float32, device=device)
                    except Exception:
                        logger.warning("Could not coerce targets_per_patch to tensor — skipping batch")
                        continue

                try:
                    tabular = tabular.to(device) if (tabular is not None and torch.is_tensor(tabular)) else tabular
                except Exception:
                    logger.debug("tabular could not be moved to device; will be handled in helper")

                # Determine sizes and safe-truncate
                N = images.size(0)
                P = patches_per_image
                if N % P != 0:
                    keep_sets = N // P
                    if keep_sets == 0:
                        logger.warning(f"Batch too small: N={N}, P={P}. Skipping.")
                        continue
                    keep_count = keep_sets * P
                    images = images[:keep_count]
                    targets_per_patch = targets_per_patch[:keep_count]
                    if sample_ids is not None:
                        try:
                            sample_ids = sample_ids[:keep_count]
                        except Exception:
                            pass
                    if target_names is not None:
                        try:
                            target_names = target_names[:keep_count]
                        except Exception:
                            pass
                    N = images.size(0)

                B = N // P

                # Build target_idx robustly
                try:
                    target_idx = build_target_idx_from_tabular(tabular, N, B, P, want_image_level=use_mil, device=device)
                except Exception as e:
                    logger.warning(f"Failed to build target_idx: {e}")
                    # fallback to zeros of appropriate length
                    target_idx = torch.zeros(B if use_mil else N, dtype=torch.long, device=device)

                optimizer.zero_grad()

                try:
                    # Forward
                    predictions = model(images, target_idx, return_features=False)
                    # Determine expected length
                    expected_len = B if use_mil else N
                    pred_len = predictions.view(-1).numel()
                    if pred_len != expected_len:
                        logger.error(f"Model returned unexpected length: {predictions.shape} (expected {expected_len}). Skipping batch {batch_idx}.")
                        continue

                    # Build image-level targets when needed
                    img_targets = _image_targets_from_patch_targets(targets_per_patch, B, P, device)
                    if not isinstance(img_targets, torch.Tensor):
                        img_targets = torch.from_numpy(img_targets).float().to(device)
                    if use_mil:
                        # predictions are image-level [B]
                        loss_preds = predictions.view(-1)
                        loss_targets = img_targets.view(-1)
                    else:
                        # predictions are patch-level [N]
                        loss_preds = predictions.view(-1)
                        loss_targets = targets_per_patch.view(-1)

                except Exception as e:
                    logger.exception(f"Forward pass failed: {e}")
                    continue

                # Ensure same length
                if loss_preds.shape[0] != loss_targets.shape[0]:
                    logger.error(f"Length mismatch preds {loss_preds.shape} vs targets {loss_targets.shape} — skipping batch")
                    continue

                # Compute aligned target names for weighted loss
                try:
                    if use_mil:
                        try:
                            unique_target_names = get_unique_sample_id(target_names, patches_per_image=P)
                            aligned_target_names = unique_target_names[:B]
                        except Exception:
                            aligned_target_names = _collapse_target_names_by_sample_ids(sample_ids or [], target_names or [], P)[:B]
                    else:
                        aligned_target_names = _align_target_names(sample_ids, target_names, pred_len=loss_targets.shape[0], B=B, P=P)
                except Exception as e:
                    logger.warning(f"Failed to align target names: {e}")
                    aligned_target_names = ["None"] * loss_targets.shape[0]

                # Compute loss
                try:
                    loss = weighted_r2_loss(loss_preds, loss_targets, aligned_target_names, device)
                except Exception as e:
                    logger.warning(f"Loss computation failed: {e}")
                    continue

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG.get('gradient_clip', 5.0))
                optimizer.step()

                train_loss += float(loss.item())
                train_preds.append(loss_preds.detach().cpu().numpy().ravel())
                train_targets.append(loss_targets.detach().cpu().numpy().ravel())

                # store aligned names (replicate/trim to match numeric length)
                try:
                    if use_mil:
                        names_to_extend = aligned_target_names if isinstance(aligned_target_names, (list, tuple, np.ndarray)) else [aligned_target_names]
                    else:
                        names_to_extend = _align_target_names(sample_ids, target_names, pred_len=loss_targets.shape[0], B=B, P=P)
                except Exception:
                    names_to_extend = (target_names[:len(loss_targets)]
                                       if (isinstance(target_names, (list, tuple, np.ndarray)) and len(target_names) >= len(loss_targets))
                                       else ["None"] * len(loss_targets))

                numeric_len = train_preds[-1].shape[0]
                if len(names_to_extend) != numeric_len:
                    names_to_extend = (list(names_to_extend) * ((numeric_len // max(1, len(names_to_extend))) + 1))[:numeric_len]

                train_target_names.extend(names_to_extend)

                pbar.set_postfix({'loss': float(loss.item())})

        # Scheduler step at epoch end
        try:
            scheduler.step()
        except Exception:
            pass

        # ===== VALIDATION =====
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        val_target_names = []

        with torch.no_grad():
            with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [VAL]") as pbar:
                for batch_idx, batch_data in enumerate(pbar):
                    try:
                        if len(batch_data) == 7:
                            images, tabular, species_batch, target_idx_batch, targets_per_patch, sample_ids, target_names = batch_data
                        else:
                            raise ValueError(f"Unexpected batch length: {len(batch_data)}")
                    except Exception as e:
                        logger.exception(f"Failed to unpack batch: {e}")
                        continue

                    if targets_per_patch is None:
                        continue

                    try:
                        images = images.to(device)
                    except Exception:
                        pass

                    try:
                        targets_per_patch = targets_per_patch.to(device).float()
                    except Exception:
                        try:
                            targets_per_patch = torch.tensor(np.asarray(targets_per_patch).ravel(), dtype=torch.float32, device=device)
                        except Exception:
                            logger.warning("Could not coerce val targets_per_patch to tensor; skipping")
                            continue

                    N = images.size(0)
                    P = patches_per_image
                    if N % P != 0:
                        keep_sets = N // P
                        if keep_sets == 0:
                            continue
                        keep_count = keep_sets * P
                        images = images[:keep_count]
                        targets_per_patch = targets_per_patch[:keep_count]
                        if sample_ids is not None:
                            try:
                                sample_ids = sample_ids[:keep_count]
                            except Exception:
                                pass
                        if target_names is not None:
                            try:
                                target_names = target_names[:keep_count]
                            except Exception:
                                pass
                        N = images.size(0)

                    B = N // P

                    # build target_idx
                    try:
                        target_idx = build_target_idx_from_tabular(tabular, N, B, P, want_image_level=use_mil, device=device)
                    except Exception:
                        target_idx = torch.zeros(B if use_mil else N, dtype=torch.long, device=device)

                    try:
                        predictions = model(images, target_idx, return_features=False)
                        expected_len = B if use_mil else N
                        if predictions.view(-1).numel() != expected_len:
                            logger.error(f"Val model returned unexpected length: {predictions.shape} (expected {expected_len}). Skipping val batch {batch_idx}.")
                            continue

                        img_targets = _image_targets_from_patch_targets(targets_per_patch, B, P, device)
                        if not isinstance(img_targets, torch.Tensor):
                            img_targets = torch.from_numpy(img_targets).float().to(device)

                        if use_mil:
                            loss_preds = predictions.view(-1)
                            loss_targets = img_targets.view(-1)
                        else:
                            loss_preds = predictions.view(-1)
                            loss_targets = targets_per_patch.view(-1)

                    except Exception as e:
                        logger.exception(f"Val forward pass failed: {e}")
                        continue

                    if loss_preds.shape[0] != loss_targets.shape[0]:
                        logger.error(f"Val length mismatch preds {loss_preds.shape} vs targets {loss_targets.shape} — skipping")
                        continue

                    try:
                        if use_mil:
                            try:
                                unique_target_names = get_unique_sample_id(target_names, patches_per_image=P)
                                aligned_target_names = unique_target_names[:B]
                            except Exception:
                                aligned_target_names = _collapse_target_names_by_sample_ids(sample_ids or [], target_names or [], P)[:B]
                        else:
                            aligned_target_names = _align_target_names(sample_ids, target_names, pred_len=loss_targets.shape[0], B=B, P=P)
                        loss = weighted_r2_loss(loss_preds, loss_targets, aligned_target_names, device)
                    except Exception as e:
                        logger.warning(f"Val loss computation failed: {e}")
                        continue

                    val_loss += float(loss.item())
                    val_preds.append(loss_preds.cpu().numpy().ravel())
                    val_targets.append(loss_targets.cpu().numpy().ravel())

                    # store aligned names for validation
                    try:
                        if use_mil:
                            names_to_extend = aligned_target_names if isinstance(aligned_target_names, (list, tuple, np.ndarray)) else [aligned_target_names]
                        else:
                            names_to_extend = _align_target_names(sample_ids, target_names, pred_len=loss_targets.shape[0], B=B, P=P)
                    except Exception:
                        names_to_extend = (target_names[:len(loss_targets)]
                                           if (isinstance(target_names, (list, tuple, np.ndarray)) and len(target_names) >= len(loss_targets))
                                           else ["None"] * len(loss_targets))

                    numeric_len = val_preds[-1].shape[0]
                    if len(names_to_extend) != numeric_len:
                        names_to_extend = (list(names_to_extend) * ((numeric_len // max(1, len(names_to_extend))) + 1))[:numeric_len]

                    val_target_names.extend(names_to_extend)

                    pbar.set_postfix({'loss': float(loss.item())})

        # Metrics
        train_preds_cat = np.concatenate(train_preds) if train_preds else np.array([])
        train_targets_cat = np.concatenate(train_targets) if train_targets else np.array([])
        val_preds_cat = np.concatenate(val_preds) if val_preds else np.array([])
        val_targets_cat = np.concatenate(val_targets) if val_targets else np.array([])

        train_r2 = compute_weighted_r2_metric(train_preds_cat, train_targets_cat, train_target_names) if len(train_preds) > 0 else 0.0
        val_r2 = compute_weighted_r2_metric(val_preds_cat, val_targets_cat, val_target_names) if len(val_preds) > 0 else 0.0

        avg_train_loss = train_loss / max(1, len(train_loader))
        avg_val_loss = val_loss / max(1, len(val_loader))

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_r2'].append(train_r2)
        history['val_r2'].append(val_r2)

        logger.info(
            f"Epoch {epoch+1}/{num_epochs} → "
            f"Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f} | "
            f"Train R²={train_r2:.4f}, Val R²={val_r2:.4f}"
        )

        # Save best
        if avg_val_loss < best_val_loss:
            patience_counter = 0
            best_val_loss = avg_val_loss
            save_path = MODELS_DIR / f"{model_name}_best.pth"
            try:
                torch.save(model.state_dict(), save_path)
                logger.info(f"✓ Saved best {model_name} model")
            except Exception as e:
                logger.warning(f"Failed to save model: {e}")
        else:
            patience_counter += 1
            if patience_counter >= CONFIG.get('early_stopping_patience', 10):
                logger.info(f"✓ Early stopping at epoch {epoch+1}")
                break

    elapsed = time.time() - start_time
    logger.info(f"\n✓ {model_name} training finished in {elapsed/60:.2f} minutes")
    logger.info(f"Best Val Loss = {best_val_loss:.4f}")

    return history



def train_xgboost_model(train_split_df, val_split_df, lookups_dict):
    """
    Train XGBoost on image-level tabular features.
    
    Args:
        train_split_df: Training dataframe
        val_split_df: Validation dataframe
        lookups_dict: Dictionary with STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS
    """
    logger.info(f"\n{'='*80}")
    logger.info("TRAINING XGBOOST (TABULAR - IMAGE LEVEL)")
    logger.info(f"{'='*80}")
    
    start_time = time.time()
    
    #  Get from lookups_dict
    STATE_COLS = lookups_dict['STATE_COLS']
    SPECIES_COLS = lookups_dict['SPECIES_COLS']
    TARGET_TYPE_COLS = lookups_dict['TARGET_TYPE_COLS']
    
    xgb_feature_cols = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month', 'DayOfYear', 'Quarter']
    xgb_feature_cols += STATE_COLS + SPECIES_COLS + TARGET_TYPE_COLS
    xgb_feature_cols = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month', 'DayOfYear', 'Quarter']
    xgb_feature_cols += STATE_COLS + SPECIES_COLS
    
    xgb_feature_cols += TARGET_TYPE_COLS  # 1st copy
    xgb_feature_cols += TARGET_TYPE_COLS  # 2nd copy
    xgb_feature_cols += TARGET_TYPE_COLS  # 3rd copy

    
    # Safeguard: ensure required columns exist
    missing_cols = [c for c in xgb_feature_cols if c not in train_split_df.columns]
    if missing_cols:
        logger.error(f"✗ Missing columns for XGBoost training: {missing_cols}")
        raise ValueError(f"Missing columns: {missing_cols}")
    
    X = train_split_df[xgb_feature_cols].values
    y = train_split_df['target'].values
    target_names = train_split_df['target_name'].values
    
    # Sanity check shapes
    assert X.shape[0] == y.shape[0] == target_names.shape[0], (
        "Training lengths mismatch: X={}, y={}, target_names={}".format(X.shape[0], len(y), len(target_names))
    )

    X = X.astype(np.float64)
    y = y.astype(np.float64)

    logger.info(f"XGBoost training samples: {len(X)} (image level)")
    logger.info(f"XGBoost features: {len(xgb_feature_cols)} ({', '.join(xgb_feature_cols[:3])}...)")
    

    # Tabular-noise augmentation
    # Use same style as meta-learner augmentation: multiplicative / scaled Gaussian noise.
    noise_scale = CONFIG.get('xgb_noise_scale', 0.05)  # default 5% if not set
    n_augment_rounds = int(CONFIG.get('xgb_augment_rounds', 1))  # how many augmented copies to create
    
    if n_augment_rounds > 0 and noise_scale > 0:
        logger.info(f"✓ Applying tabular noise augmentation to XGBoost training: rounds={n_augment_rounds}, noise_scale={noise_scale}")
        # create augmented copies and concatenate
        X_aug_list = [X]
        y_aug_list = [y]
        target_names_aug_list = [target_names]
        rng = np.random.RandomState(CONFIG.get('random_seed', 42))
        
        for r in range(n_augment_rounds):
            base_scale = noise_scale
            # Create noise shaped like X
            # We scale noise per-element by max(1.0, abs(X)) to allow both absolute and relative perturbation
            noise = rng.normal(loc=0.0, scale=base_scale * np.maximum(1.0, np.abs(X)), size=X.shape)
            X_noisy = X + noise
            # For one-hot columns (state/species/target-type) it's better to clip to [0,1]
            # identify one-hot offsets:
            numeric_dim = 5
            state_dim = len(STATE_COLS)
            species_dim = len(SPECIES_COLS)
            target_dim = len(TARGET_TYPE_COLS)
            # clip one-hot ranges to [0,1] for those columns
            start_state = numeric_dim
            end_state = start_state + state_dim
            start_species = end_state
            end_species = start_species + species_dim
            start_target = end_species
            end_target = start_target + target_dim
            if state_dim > 0:
                X_noisy[:, start_state:end_state] = np.clip(X_noisy[:, start_state:end_state], 0.0, 1.0)
            if species_dim > 0:
                X_noisy[:, start_species:end_species] = np.clip(X_noisy[:, start_species:end_species], 0.0, 1.0)
            if target_dim > 0:
                X_noisy[:, start_target:end_target] = np.clip(X_noisy[:, start_target:end_target], 0.0, 1.0)
            
            X_aug_list.append(X_noisy)
            y_aug_list.append(y.copy())
            target_names_aug_list.append(target_names.copy())
        
        # Concatenate original + augmented
        X_train_final = np.vstack(X_aug_list)
        y_train_final = np.concatenate(y_aug_list)
        target_names_final = np.concatenate(target_names_aug_list)
        
        logger.info(f"  Augmented training size: {X_train_final.shape[0]} (original {X.shape[0]})")
    else:
        X_train_final = X
        y_train_final = y
        target_names_final = target_names


    param_dist = {
        'n_estimators': randint(100, 1000),
        'max_depth': randint(3, 12),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.6, 1.0),
        'colsample_bytree': uniform(0.6, 1.0),
        'min_child_weight': randint(1, 10),
        'gamma': uniform(0, 5),
        'reg_alpha': uniform(0, 1),      
        'reg_lambda': uniform(0, 1),
    }
    
    # Initialize model
    xgb_base = xgb.XGBRegressor(random_state=42, n_jobs=-1, verbosity=0)

    search = RandomizedSearchCV(
        xgb_base,
        param_distributions=param_dist,
        n_iter=50,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        random_state=42,
        n_jobs=-1
    )

    logger.info(f"  Running RandomizedSearchCV with n_iter=50, cv=5...")

    with tqdm(total=20, desc="🔍 XGBoost Hyperparameter Search", unit="trial") as pbar:
        search.fit(X_train_final, y_train_final)
        pbar.update(2)


    logger.info(f"✓ Best CV score: {search.best_score_:.4f}")
    logger.info(f"✓ Best parameters:")
    
    # ===== UPDATE CONFIG WITH BEST PARAMETERS =====
    CONFIG['xgb_n_estimators'] = int(search.best_params_['n_estimators'])
    CONFIG['xgb_max_depth'] = int(search.best_params_['max_depth'])
    CONFIG['xgb_learning_rate'] = float(search.best_params_['learning_rate'])
    CONFIG['xgb_subsample'] = float(search.best_params_['subsample'])
    CONFIG['xgb_colsample_bytree'] = float(search.best_params_['colsample_bytree'])

    xgb_model = XGBoostTabularModel(lookups_dict)
    xgb_model.fit(X_train_final, y_train_final, target_names_final)
    logger.info("✓ XGBoost training complete")
    
    # Validation
    X_val = val_split_df[xgb_feature_cols].values
    y_val = val_split_df['target'].values
    target_names_val = val_split_df['target_name'].values

    X_val = X_val.astype(np.float64)
    y_val = y_val.astype(np.float64)

    
    # Sanity check
    assert X_val.shape[0] == y_val.shape[0] == target_names_val.shape[0], (
        "Validation lengths mismatch: X_val={}, y_val={}, target_names_val={}".format(X_val.shape[0], len(y_val), len(target_names_val))
    )
    
    logger.info(f"XGBoost validation samples: {len(X_val)} (image level)")
    
    val_preds_all = np.zeros(len(y_val), dtype=np.float32)
    
    # Make predictions for each target type 
    for target_type in np.unique(target_names_val):
        mask = target_names_val == target_type
        X_target = X_val[mask]
        try:
            pred = xgb_model.predict(X_target, target_type)
            val_preds_all[mask] = pred
        except Exception as e:
            logger.exception(f"XGBoost predict failed for target {target_type}: {e}")
            # fallback: use mean of train target for that target_type
            fallback = train_split_df[train_split_df['target_name'] == target_type]['target'].mean()
            logger.warning(f"Using fallback mean {fallback:.4f} for {target_type}")
            val_preds_all[mask] = fallback
    
    # Now compute R² with MATCHING order
    val_r2 = compute_weighted_r2_metric(
        val_preds_all,     
        y_val,              
        target_names_val   
    )
    logger.info(f"XGBoost Validation R²: {val_r2:.4f}")
    
    # Save model
    xgb_model.save(MODELS_DIR / 'xgboost_best.pkl')
    logger.info(f"✓ XGBoost model saved")
    
    elapsed_time = time.time() - start_time
    logger.info(f"✓ XGBoost training completed in {elapsed_time/60:.2f} minutes\n")
    
    return xgb_model

def train_meta_learner_model(
    base_model_preds_train, base_model_preds_val,
    num_epochs,
    device=device,
    pretrained_model=None,
    optimize_hparams=True
):
    """
    Train meta-learner on pre-computed base model predictions.
    
    UPDATED: Now uses target_idx (from TaskEmbedding) instead of tabular features.
    
    Args:
        base_model_preds_train: Dict with training predictions (vit, resnet, densenet, xgboost, ensemble)
        base_model_preds_val: Dict with validation predictions
        num_epochs: Number of epochs
        device: torch device
        pretrained_model: Optional pretrained meta-learner
        optimize_hparams: Whether to run hyperparameter search
    
    Returns:
        Trained meta-learner model
    """
    logger.info(f"\\n{'='*80}")
    logger.info("TRAINING META-LEARNER (IMAGE LEVEL ENSEMBLE + TASK EMBEDDING)")
    logger.info(f"{'='*80}")
     
    required_keys = ['vit', 'resnet', 'densenet', 'xgboost', 'ensemble', 'tabular', 'targets', 'target_names', 'target_idx']
    for k in required_keys:
        if k not in base_model_preds_train or k not in base_model_preds_val:
            logger.error(f"Missing key '{k}' in base_model_preds_train or base_model_preds_val")
            raise KeyError(k)
    
    n_samples_train = len(base_model_preds_train['targets'])
    n_samples_val = len(base_model_preds_val['targets'])
    
    logger.info(f"Training samples: {n_samples_train} (image level)")
    logger.info(f"Validation samples: {n_samples_val} (image level)")
    logger.info(f"Meta-learner input: 5 base predictions + 64D task embedding = 69 dims")
    
    # Normalize device
    try:
        device = device if isinstance(device, torch.device) else torch.device(
            device if device is not None else ("cuda" if torch.cuda.is_available() else "cpu")
        )
    except Exception:
        device = torch.device("cpu")
    
    logger.info(f"Meta-learner training device: {device}")
    
    # ✅ NEW: HYPERPARAMETER OPTIMIZATION
    if optimize_hparams:
        logger.info(f"\\n🔍 Starting Meta-Learner hyperparameter optimization...")
        
        param_grid = {
            'lr': [1e-4, 5e-4, 1e-3],
            'weight_decay': [1e-5, 1e-4],
            'dropout': [0.05, 0.1, 0.15],
        }
        
        best_meta_score = float('-inf')
        best_meta_params = None
        total_combinations = len(param_grid['lr']) * len(param_grid['weight_decay']) * len(param_grid['dropout'])
        
        with tqdm(total=total_combinations, desc="🔍 Meta-Learner Hyperparameter Search", unit="config") as pbar:
            for lr in param_grid['lr']:
                for wd in param_grid['weight_decay']:
                    for dropout in param_grid['dropout']:
                        trial_params = {
                            'learning_rate': lr,
                            'weight_decay': wd,
                            'dropout': dropout,
                        }
                        
                        logger.debug(f"\\n  Trial: lr={lr:.0e}, wd={wd:.0e}, dropout={dropout:.2f}")
                        
                        try:
                            # Create trial model with TaskEmbedding
                            trial_model = MetaLearnerModel(dropout_rate=dropout, pretrained=False).to(device)
                            trial_model.train()
                            
                            trial_optimizer = torch.optim.AdamW(
                                trial_model.parameters(),
                                lr=lr,
                                weight_decay=wd
                            )
                            
                            trial_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                                trial_optimizer, T_0=5, T_mult=2
                            )
                            
                            batch_size = CONFIG['batch_size']
                            best_trial_val_r2 = float('-inf')
                            patience = 5
                            patience_counter = 0
                            search_epochs = min(20, num_epochs)
                            
                            for search_epoch in range(search_epochs):
                                trial_model.train()
                                train_loss = 0.0
                                
                                indices = np.random.permutation(n_samples_train)
                                
                                for i in range(0, n_samples_train, batch_size):
                                    end_idx = min(i + batch_size, n_samples_train)
                                    batch_indices = indices[i:end_idx]
                                    
                                    # Build batch predictions (5 base models)
                                    base_preds = np.column_stack([
                                        base_model_preds_train['vit'][batch_indices],
                                        base_model_preds_train['resnet'][batch_indices],
                                        base_model_preds_train['densenet'][batch_indices],
                                        base_model_preds_train['xgboost'][batch_indices],
                                        base_model_preds_train['ensemble'][batch_indices],
                                    ])
                                    
                                    target_idx = torch.tensor(
                                        base_model_preds_train['target_idx'][batch_indices],
                                        dtype=torch.long).to(device)

                                    base_preds = torch.tensor(base_preds, dtype=torch.float32).to(device)
                                    targets = torch.tensor(
                                        base_model_preds_train['targets'][batch_indices],
                                        dtype=torch.float32
                                    ).to(device)
                                    target_names_batch = np.array(base_model_preds_train['target_names'])[batch_indices]
                                    
                                    # ✅ NEW: Pass target_idx instead of tabular
                                    predictions = trial_model(base_preds, target_idx)
                                    loss = weighted_r2_loss(predictions, targets, target_names_batch, device)
                                    
                                    trial_optimizer.zero_grad()
                                    loss.backward()
                                    torch.nn.utils.clip_grad_norm_(trial_model.parameters(), CONFIG['gradient_clip'])
                                    trial_optimizer.step()
                                    
                                    train_loss += float(loss.item())
                                
                                trial_scheduler.step()
                                
                                # Validate
                                trial_model.eval()
                                val_preds_all = []
                                val_targets_all = []
                                val_names_all = []
                                
                                with torch.no_grad():
                                    for i in range(0, n_samples_val, batch_size):
                                        end_idx = min(i + batch_size, n_samples_val)
                                        batch_indices = np.arange(i, end_idx)
                                        
                                        base_preds_val = np.column_stack([
                                            base_model_preds_val['vit'][batch_indices],
                                            base_model_preds_val['resnet'][batch_indices],
                                            base_model_preds_val['densenet'][batch_indices],
                                            base_model_preds_val['xgboost'][batch_indices],
                                            base_model_preds_val['ensemble'][batch_indices],
                                        ])
                                        target_idx_val = torch.tensor(
                                            base_model_preds_val['target_idx'][batch_indices],
                                            dtype=torch.long
                                        ).to(device)

                                        base_preds_val = torch.tensor(base_preds_val, dtype=torch.float32).to(device)
                                        targets_val = torch.tensor(
                                            base_model_preds_val['targets'][batch_indices],
                                            dtype=torch.float32
                                        ).to(device)
                                        target_names_val_batch = np.array(base_model_preds_val['target_names'])[batch_indices]
                                        
                                        # ✅ NEW: Pass target_idx instead of tabular
                                        predictions = trial_model(base_preds_val, target_idx_val)
                                        val_preds_all.append(predictions.cpu().numpy())
                                        val_targets_all.append(targets_val.cpu().numpy())
                                        val_names_all.append(target_names_val_batch)
                                
                                if not val_preds_all:
                                    logger.warning("No validation predictions collected during hyperopt trial")
                                    trial_val_r2 = float('-inf')
                                else:
                                    val_preds = np.concatenate(val_preds_all)
                                    val_targets = np.concatenate(val_targets_all)
                                    val_names = np.concatenate(val_names_all)
                                    trial_val_r2 = compute_weighted_r2_metric(val_preds, val_targets, val_names)
                                
                                if trial_val_r2 > best_trial_val_r2:
                                    best_trial_val_r2 = trial_val_r2
                                    patience_counter = 0
                                else:
                                    patience_counter += 1
                                    if patience_counter >= patience:
                                        break
                            
                            logger.info(f"    → Best validation R²: {best_trial_val_r2:.4f}")
                            
                            if best_trial_val_r2 > best_meta_score:
                                best_meta_score = best_trial_val_r2
                                best_meta_params = trial_params
                                logger.info(f"    ✓ NEW BEST!")
                            
                            del trial_model, trial_optimizer, trial_scheduler
                            try:
                                torch.cuda.empty_cache()
                            except Exception:
                                pass
                            gc.collect()
                        
                        except Exception as e:
                            logger.error(f"    ✗ Trial failed: {e}")
                        
                        pbar.update(1)
        
        logger.info("\\n✓ Meta-Learner optimization complete!")
        
        if best_meta_params is None:
            logger.warning("Meta-Learner hyperopt found no successful trials — using defaults.")
            best_meta_params = {
                'learning_rate': CONFIG.get('metalearner_lr', 1e-4),
                'weight_decay': CONFIG.get('meta_weight_decay', 1e-4),
                'dropout': CONFIG.get('meta_dropout_rate', 0.1)
            }
        
        logger.info(f"✓ Best validation R²: {best_meta_score:.4f}")
        logger.info("✓ Best parameters:")
        for param, value in best_meta_params.items():
            logger.info(f"  - {param}: {value}")
        
        CONFIG['metalearner_lr'] = best_meta_params['learning_rate']
        CONFIG['meta_weight_decay'] = best_meta_params['weight_decay']
        CONFIG['meta_dropout_rate'] = best_meta_params['dropout']
        logger.info("✓ CONFIG updated with best meta-learner parameters")
    
    # Instantiate model
    if pretrained_model is not None:
        try:
            model = pretrained_model.to(device)
            logger.info("Using provided meta_learner instance for training.")
        except Exception as e:
            logger.warning(f"Could not load pretrained meta-learner: {e}; training from scratch")
            model = MetaLearnerModel(
                dropout_rate=CONFIG.get('meta_dropout_rate', 0.1),
                pretrained=False
            ).to(device)
    else:
        model = MetaLearnerModel(
            dropout_rate=CONFIG.get('meta_dropout_rate', 0.1),
            pretrained=False
        ).to(device)
    
    model.train()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG['metalearner_lr'],
        weight_decay=CONFIG.get('meta_weight_decay', 1e-4)
    )
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    best_val_loss = float('inf')
    patience_counter = 0
    start_time = time.time()
    batch_size = CONFIG['batch_size']
    

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_batches = 0
        train_preds_all = []
        train_targets_all = []
        train_names_all = []
        
        indices = np.random.permutation(n_samples_train)
        
        for i in range(0, n_samples_train, batch_size):
            end_idx = min(i + batch_size, n_samples_train)
            batch_indices = indices[i:end_idx]
            
            # Build base predictions (5 base models)
            base_preds = np.column_stack([
                base_model_preds_train['vit'][batch_indices],
                base_model_preds_train['resnet'][batch_indices],
                base_model_preds_train['densenet'][batch_indices],
                base_model_preds_train['xgboost'][batch_indices],
                base_model_preds_train['ensemble'][batch_indices],
            ])
            target_idx = torch.tensor(
                base_model_preds_train['target_idx'][batch_indices],
                dtype=torch.long
            ).to(device)

            base_preds = torch.tensor(base_preds, dtype=torch.float32).to(device)
            targets = torch.tensor(
                base_model_preds_train['targets'][batch_indices],
                dtype=torch.float32
            ).to(device)
            target_names_batch = np.array(base_model_preds_train['target_names'])[batch_indices]
            
 
            predictions = model(base_preds, target_idx)
            loss = weighted_r2_loss(predictions, targets, target_names_batch, device)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['gradient_clip'])
            optimizer.step()
            
            train_loss += float(loss.item())
            train_batches += 1
            train_preds_all.append(predictions.detach().cpu().numpy())
            train_targets_all.append(targets.cpu().numpy())
            train_names_all.append(target_names_batch)
        
        scheduler.step()
        

        train_preds_concat = np.concatenate(train_preds_all) if train_preds_all else np.array([])
        train_targets_concat = np.concatenate(train_targets_all) if train_targets_all else np.array([])
        train_names_concat = np.concatenate(train_names_all) if train_names_all else np.array([])
        
        train_r2 = compute_weighted_r2_metric(
            train_preds_concat, train_targets_concat, train_names_concat
        ) if train_preds_concat.size else 0.0
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_batches = 0
        val_preds_all = []
        val_targets_all = []
        val_names_all = []
        
        with torch.no_grad():
            for i in range(0, n_samples_val, batch_size):
                end_idx = min(i + batch_size, n_samples_val)
                batch_indices = np.arange(i, end_idx)
                
                base_preds_val = np.column_stack([
                    base_model_preds_val['vit'][batch_indices],
                    base_model_preds_val['resnet'][batch_indices],
                    base_model_preds_val['densenet'][batch_indices],
                    base_model_preds_val['xgboost'][batch_indices],
                    base_model_preds_val['ensemble'][batch_indices],
                ])
                
                target_idx_val = torch.tensor(
                    base_model_preds_val['target_idx'][batch_indices],
                    dtype=torch.long
                ).to(device)

                base_preds_val = torch.tensor(base_preds_val, dtype=torch.float32).to(device)
                targets_val = torch.tensor(
                    base_model_preds_val['targets'][batch_indices],
                    dtype=torch.float32
                ).to(device)
                target_names_val_batch = np.array(base_model_preds_val['target_names'])[batch_indices]
                
                predictions = model(base_preds_val, target_idx_val)
                loss = weighted_r2_loss(predictions, targets_val, target_names_val_batch, device)
                
                val_loss += float(loss.item())
                val_batches += 1
                val_preds_all.append(predictions.cpu().numpy())
                val_targets_all.append(targets_val.cpu().numpy())
                val_names_all.append(target_names_val_batch)
        
        val_preds_concat = np.concatenate(val_preds_all) if val_preds_all else np.array([])
        val_targets_concat = np.concatenate(val_targets_all) if val_targets_all else np.array([])
        val_names_concat = np.concatenate(val_names_all) if val_names_all else np.array([])
        
        avg_train_loss = train_loss / max(1, train_batches)
        avg_val_loss = val_loss / max(1, val_batches)
        
        val_r2 = compute_weighted_r2_metric(
            val_preds_concat, val_targets_concat, val_names_concat
        ) if val_preds_concat.size else 0.0
        
        if (epoch + 1) % 10 == 0 or epoch == 0:
            logger.info(
                f"Epoch {epoch+1}/{num_epochs} | "
                f"Train R²={train_r2:.4f}, Val R²={val_r2:.4f} | "
                f"Loss={avg_val_loss:.6f}"
            )
        
        if epoch % 10 == 0:
            try:
                torch.cuda.empty_cache()
            except Exception:
                pass
            gc.collect()
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            
            torch.save(model.state_dict(), MODELS_DIR / 'metalearner_best.pth')
            logger.info(f"  ✓ Best meta-learner saved (Val R²={val_r2:.4f})")
        else:
            patience_counter += 1
            if patience_counter >= CONFIG['early_stopping_patience']:
                logger.info(f"✓ Early stopping at Epoch {epoch+1}")
                break
    

    best_path = MODELS_DIR / 'metalearner_best.pth'
    if best_path.exists():
        try:
            model.load_state_dict(torch.load(best_path, map_location=device))
        except Exception as e:
            logger.warning(f"Could not load best meta-learner weights: {e}")
    
    elapsed_time = time.time() - start_time
    logger.info(f"✓ Meta-learner training completed in {elapsed_time/60:.2f} minutes\\n")
    
    return model


def train_species_model(train_loader, val_loader, train_df, num_epochs=50, device=device, model=None):
    """
    Train SpeciesClassificationViT with optional MIL aggregation.
    Returns the trained model (and saves best checkpoint as 'species_best.pth' in MODELS_DIR).
    Notes:
      - Supports models that return per-patch logits (B*P, C) or per-image logits (B, C).
      - If CONFIG['use_mil'] is True but model returns per-patch logits, this function aggregates to image-level by mean.
      - Robust device handling and shapes checks; skips problematic batches rather than crashing.
    """
    logger.info("\n" + "="*80)
    logger.info("TRAINING SPECIES CLASSIFICATION MODEL")
    logger.info("="*80)

    if model is None:
        raise ValueError("model must be provided")
    species_model = model
    use_mil = CONFIG.get("use_mil", False)
    patches_per_image = getattr(species_model, "num_patches", None)
    
    if patches_per_image is None:
        patches_per_image = CONFIG.get("num_patches", 8)
        logger.warning(f"species_model.num_patches missing - falling back to CONFIG/assumed value {patches_per_image}")

    logger.info(f"✓ use_mil_species = {use_mil}")
    logger.info(f"✓ patches_per_image = {patches_per_image}")

    species_list = getattr(species_model, "species_list", None)
    if species_list is None:
        raise ValueError("species_model must expose species_list (ordered)")

    species_to_idx = {sp: i for i, sp in enumerate(species_list)}
    num_classes = len(species_list)

    optimizer = optim.AdamW(species_model.parameters(), lr=CONFIG.get("species_lr", 2e-4), weight_decay=CONFIG.get("weight_decay", 1e-2))
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=CONFIG.get("species_step", 10), gamma=0.5)

    best_val_loss = float("inf")
    patience_counter = 0
    patience = CONFIG.get("early_stopping_patience", 20)
    start_time = time.time()

    if "image_id" in train_df.columns and "Species" in train_df.columns:
        image_to_species = train_df.set_index("image_id")["Species"].to_dict()
    else:
        image_to_species = {}

    for epoch in range(num_epochs):
        species_model.train()
        running_loss = 0.0
        n_batches = 0
        for batch in tqdm(train_loader, desc=f"Species Train Epoch {epoch+1}/{num_epochs}", leave=False):
            try:    
                if len(batch) == 7:
                    images, tabular, species_targets,target_idx_batch, targets_per_patch, sample_ids, target_names = batch
                else:
                    raise ValueError(f"Unexpected batch length: {len(batch)}")
            except Exception as e:
                logger.warning(f"Skipping train batch due to unexpected structure: {e}")
                continue
            try:
                images = images.to(device)
            except Exception:
                images = images.cuda() if torch.cuda.is_available() else images

            # convert species_targets to indices if necessary
            if not torch.is_tensor(species_targets):
                try:
                    species_targets = torch.tensor(species_targets, dtype=torch.long)
                except Exception:
                    try:
                        species_targets = torch.tensor([species_to_idx.get(str(s), 0) for s in species_targets], dtype=torch.long)
                    except Exception:
                        logger.warning("Could not coerce species_targets to tensor - skipping batch")
                        continue
            species_targets = species_targets.to(device)

            optimizer.zero_grad()
            if use_mil:
                outputs = species_model(images, return_features=True)  
            else:
                outputs = species_model(images, return_features=False) 
            try:
                if outputs.dim() == 1:
                    logger.warning("Received 1D outputs from species model; skipping batch")
                    continue

                if use_mil:
                    N = outputs.size(0)
                    C = outputs.size(-1)
                    # if outputs are per-patch (N == B * P) and we have per-image targets (len==B), aggregate
                    if species_targets.dim() == 1 and N % species_targets.size(0) == 0 and N // species_targets.size(0) == patches_per_image:
                        B = species_targets.size(0)
                        P = patches_per_image
                        patches = outputs.view(B, P, C).contiguous()
                        aggregated = species_model.mil_aggregator(patches) 
                        outputs_img = species_model.classification_head(aggregated)
                        targets_img = species_targets.view(B)
                    elif outputs.size(0) == species_targets.size(0):
                        outputs_img = outputs
                        targets_img = species_targets.view(-1)
                    else:
                        if N % patches_per_image == 0:
                            B = N // patches_per_image
                            try:
                                outputs_img = outputs.view(B, patches_per_image, -1).mean(dim=1)
                                if species_targets.size(0) == N:
                                    targets_img = species_targets.view(B, patches_per_image)[:, 0]
                                elif species_targets.size(0) == B:
                                    targets_img = species_targets
                                else:
                                    logger.warning("Could not reconcile species target count with outputs under MIL; skipping batch")
                                    continue
                            except Exception:
                                logger.warning("Failed to reshape outputs for MIL aggregation; skipping batch")
                                continue
                        else:
                            logger.warning("MIL enabled but cannot deduce B and P; skipping batch")
                            continue

                    loss = criterion(outputs_img, targets_img.to(device))
                else:
                    if outputs.size(0) == species_targets.size(0):
                        loss = criterion(outputs, species_targets.to(device))
                    else:
                        if outputs.size(0) == species_targets.size(0) // patches_per_image:
                            B = outputs.size(0)
                            P = patches_per_image
                            expanded = outputs.view(B, 1, -1).repeat(1, P, 1).view(B * P, -1)
                            loss = criterion(expanded, species_targets.to(device))
                        else:
                            logger.warning("Mismatch between outputs and species_targets in non-MIL mode; skipping batch")
                            continue

                loss.backward()
                nn.utils.clip_grad_norm_(species_model.parameters(), max_norm=1.0)
                optimizer.step()
                running_loss += float(loss.item())
                n_batches += 1
            except Exception as e:
                logger.warning(f"Skipping problematic train batch: {e}")
                continue

        avg_train_loss = (running_loss / max(1, n_batches)) if n_batches else float("inf")

        species_model.eval()
        val_loss = 0.0
        val_batches = 0
        with torch.no_grad():
            for batch in val_loader:
                try:    
                    if len(batch) == 7:
                        images, tabular, species_targets, target_idx_batch, targets_per_patch, sample_ids, target_names = batch
                    else:
                        raise ValueError(f"Unexpected batch length: {len(batch)}")
                except Exception as e:
                    logger.warning(f"Skipping train batch due to unexpected structure: {e}")
                    continue
                try:
                    images = images.to(device)
                    if not torch.is_tensor(species_targets):
                        species_targets = torch.tensor([species_to_idx.get(str(s), 0) for s in species_targets], dtype=torch.long)
                    species_targets = species_targets.to(device)
                    if use_mil:
                        outputs = species_model(images, return_features=True)
                    else:
                        outputs = species_model(images, return_features=False)
                    if use_mil:
                        N = outputs.size(0)
                        if N % patches_per_image == 0 and species_targets.size(0) == N // patches_per_image:
                            B = species_targets.size(0)
                            C = outputs.size(-1)
                            P = patches_per_image
                            patches = outputs.view(B, P, C).contiguous()
                            aggregated = species_model.mil_aggregator(patches)
                            outputs_img = species_model.classification_head(aggregated)
                            targets_img = species_targets.view(B)
                            vloss = criterion(outputs_img, targets_img)
                        elif outputs.size(0) == species_targets.size(0):
                            vloss = criterion(outputs, species_targets)
                        else:
                            logger.warning("Validation MIL mismatch - trying best-effort reduction")
                            if outputs.dim() == 3:
                                try:
                                    outputs_img = outputs.mean(dim=1)
                                    vloss = criterion(outputs_img, species_targets[:outputs_img.size(0)])
                                except Exception:
                                    continue
                            else:
                                continue
                    else:
                        if outputs.size(0) == species_targets.size(0):
                            vloss = criterion(outputs, species_targets)
                        else:
                            if outputs.size(0) == (species_targets.size(0) // patches_per_image):
                                B = outputs.size(0)
                                P = patches_per_image 
                                expanded = outputs.view(B, 1, -1).repeat(1, patches_per_image, 1).view(B * patches_per_image, -1)
                                vloss = criterion(expanded, species_targets)
                            else:
                                logger.warning("Validation non-MIL mismatch; skipping batch")
                                continue

                    val_loss += float(vloss.item())
                    val_batches += 1
                except Exception:
                    continue

        avg_val_loss = (val_loss / max(1, val_batches)) if val_batches else float("inf")
        if scheduler is not None:
            try:
                scheduler.step()
            except Exception:
                pass

        logger.info(f"Epoch {epoch+1}/{num_epochs} | Train Loss={avg_train_loss:.6f} | Val Loss={avg_val_loss:.6f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            try:
                torch.save(species_model.state_dict(), MODELS_DIR / 'species_best.pth')
                logger.info(f"  ✓ Best species model saved (Val Loss={best_val_loss:.6f})")
            except Exception as e:
                logger.warning(f"Failed to save species_best.pth: {e}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                logger.info(f"✓ Early stopping at Epoch {epoch+1}")
                break

    elapsed_time = time.time() - start_time
    logger.info(f"✓ Species training completed in {elapsed_time/60:.2f} minutes")

    best_path = MODELS_DIR / 'species_best.pth'
    if best_path.exists():
        try:
            species_model.load_state_dict(torch.load(best_path, map_location=device))
        except Exception as e:
            logger.warning(f"Could not strictly load best species weights: {e}")

    return species_model


logger.info("✓ Training Functions")

## 8: ENSEMBLE TRAINING - COLLECT BASE PREDICTIONS

**Purpose**: Generate predictions on test set using all 5 models and meta-learner.

### Test Prediction Pipeline

**For each test sample**:

**Step 1: Image Loading**
```
Load  image (2000×1000)
↓
Check if file exists (missing = black image)
↓
Convert BGR (OpenCV) → RGB
↓
Prepare PIL Image
```

**Step 2: Patch Extraction**
```
Image → 8 patches (500×500 each)
├─ Patch 1-4: Top row [0,500] × [0,500]
└─ Patch 5-8: Bottom row [500,1000] × [0,500]
↓
Resize each to 224×224
↓
Normalize (ImageNet stats)
↓
Stack: [8, 3, 224, 224]
```

**Step 3: Vision Models** (Batch all 8 patches)
```
Patches → ViT-Base → [8] predictions
Patches → ResNet-50 → [8] predictions
Patches → DenseNet-121 → [8] predictions
↓
Average across 8 patches
↓
Result: 3 predictions (one per model)
```

**Step 4: Species Prediction** (Optional)
```
Patches → Species ViT → [8, 15] logits
↓
Argmax → [8] class indices
↓
Majority voting across 8 patches
↓
Result: predicted_species (string)
```

**Step 5: Test Feature Fetching**
```
Apply 4-level fallback:
Level 1: Exact sample_id match
   ↓ If not found:
Level 2: Image_id + predicted_species
   ↓ If not found:
Level 3: Random state + species combo
   ↓ If not found:
Level 4: Statistical generation
↓
Result: 29-dim feature vector
```

**Step 6: XGBoost Prediction**
```
29-dim features → XGBoost → prediction
↓
Result: 1 prediction
```

**Step 7: Meta-Learner Fusion**
```
Concatenate:
- 3 vision predictions (ViT, ResNet, DenseNet)
- 1 XGBoost prediction
- 1 dummy/mean prediction (for 5 total)
- 29 tabular features
↓
Total: 34 dimensions
↓
Meta-Learner neural network
↓
Final prediction
```

**Step 8: Prediction Aggregation**
```
Final prediction = Meta-Learner([pred_vit, pred_resnet, 
                                 pred_densenet, pred_xgb, 
                                 mean_pred, features_29])
```

### Per-Patch vs Per-Sample

**Vision Models**:
- Compute per patch: [8] predictions
- Average across patches: scalar prediction
- Rationale: Spatial robustness

**Tabular Model**:
- Constant features per image
- Single prediction: scalar

**Meta-Learner**:
- Single forward pass
- Input: averaged predictions + features
- Output: final prediction

**Output**: 
- 5 test predictions (one per target type)
- Predictions stored as sample_id → value pairs

---

In [ ]:
def get_unique_sample_id(sample_ids_batch, patches_per_image=8):
    """
    Safer: return one sample_id per image.

    - If input already appears to be image-level (all elements unique) we return the list unchanged.
    - If input appears patch-level with exact repetition pattern, return each first element of chunk.
    - Otherwise fall back to run-length collapse (preserves order).
    Always returns a list of strings.
    """
    # Normalize and convert tensors/ndarrays -> python list
    if sample_ids_batch is None:
        return []

    if isinstance(sample_ids_batch, torch.Tensor):
        try:
            sample_ids_batch = sample_ids_batch.cpu().tolist()
        except Exception:
            sample_ids_batch = list(sample_ids_batch.numpy())
    elif isinstance(sample_ids_batch, (np.ndarray, )):
        sample_ids_batch = sample_ids_batch.tolist()
    elif isinstance(sample_ids_batch, (str, int)):
        return [str(sample_ids_batch)]
    elif not isinstance(sample_ids_batch, (list, tuple)):
        try:
            sample_ids_batch = list(sample_ids_batch)
        except Exception:
            return []

    # now a python list
    try:
        sample_ids_batch = [str(x) for x in sample_ids_batch]
    except Exception:
        sample_ids_batch = list(sample_ids_batch)

    total = len(sample_ids_batch)
    if total == 0:
        return []

    # Quick: if it already looks one-per-image (all unique), return as-is
    if len(set(sample_ids_batch)) == total:
        return sample_ids_batch

    # If exact multiple of patches_per_image AND chunks are consistent -> return first element of each chunk
    if patches_per_image > 0 and total % patches_per_image == 0:
        consistent = True
        out = []
        for i in range(0, total, patches_per_image):
            chunk = sample_ids_batch[i:i+patches_per_image]
            # consider chunk consistent if majority identical to first
            if len(set(chunk)) > 1:
                # allow small noise: require at least half to equal the first entry
                cnt_first = sum(1 for s in chunk if s == chunk[0])
                if cnt_first < (len(chunk) // 2 + 1):
                    consistent = False
                    break
            out.append(chunk[0])
        if consistent:
            return out

    # Fallback: run-length collapse (preserves order)
    unique = []
    prev = None
    for s in sample_ids_batch:
        if s != prev:
            unique.append(s)
            prev = s
    return unique


# -----------------------------------------------------------------
# fallback trim_mean if scipy not available
try:
    from scipy.stats import trim_mean as _scipy_trim_mean
    def trim_mean(arr, proportion_to_cut):
        return float(_scipy_trim_mean(np.asarray(arr, dtype=float), proportion_to_cut))
except Exception:
    def trim_mean(arr, proportion_to_cut):
        a = sorted(float(x) for x in arr)
        n = len(a)
        if n == 0:
            return 0.0
        cut = int(round(n * proportion_to_cut))
        if 2 * cut >= n:
            return float(sum(a) / n)
        return float(sum(a[cut:n-cut]) / max(1, n - 2*cut))


# -----------------------------------------------------------------
def get_base_model_predictions(vit_model, resnet_model, densenet_model, xgb_model,
                               data_loader, species_model=None, mode='train', device=None):
    """
    Robust drop-in for original get_base_model_predictions.
    Preserves variable names & overall behavior but hardens:
      - per-patch -> per-image target aggregation (always deterministic)
      - robust tabular stacking / padding
      - stable trimmed-mean fallback
      - uses get_unique_sample_id to infer image count when possible
    """
    # ====== helpers (use existing global helpers if present) ======
    # stable trim_mean fallback
    try:
        from scipy.stats import trim_mean as _scipy_trim_mean
        def trim_mean(arr, proportion_to_cut):
            return float(_scipy_trim_mean(np.asarray(arr, dtype=float), proportion_to_cut))
    except Exception:
        def trim_mean(arr, proportion_to_cut):
            a = sorted(float(x) for x in arr)
            n = len(a)
            if n == 0:
                return 0.0
            # use floor to avoid over-cutting small arrays
            cut = int(np.floor(n * proportion_to_cut))
            if 2 * cut >= n:
                return float(sum(a) / n)
            return float(sum(a[cut:n-cut]) / max(1, n - 2*cut))

    # try to use existing _image_targets_from_patch_targets if defined, else fallback
    if '_image_targets_from_patch_targets' in globals():
        _img_targets_fn = globals()['_image_targets_from_patch_targets']
    else:
        def _img_targets_fn(targets_per_patch, B, P, device=None):
            # fallback mimic of your earlier implementation -> returns numpy 1D length B
            if targets_per_patch is None:
                return np.zeros((B,), dtype=float)
            if isinstance(targets_per_patch, torch.Tensor):
                t = targets_per_patch.detach().cpu().numpy().ravel()
            else:
                t = np.asarray(targets_per_patch).ravel()
            N = t.size
            exp = B * P
            if N != exp:
                if N > exp:
                    t = t[:exp]
                else:
                    pad_n = exp - N
                    pad_vals = np.full((pad_n,), t.mean() if N > 0 else 0.0)
                    t = np.concatenate([t, pad_vals], axis=0)
            return t.reshape(B, P).mean(axis=1)

    # ====== ensure device ======
    try:
        device = device if isinstance(device, torch.device) else torch.device(device if device is not None else ("cuda" if torch.cuda.is_available() else "cpu"))
    except Exception:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    all_vit_preds = []
    all_resnet_preds = []
    all_densenet_preds = []
    all_xgb_preds = []
    all_tabular = []
    all_targets = []
    all_sample_ids = []
    all_target_names = []
    all_target_idx = []

    use_mil = CONFIG.get("use_mil", False)
    patches_per_row_detected = None
    MAX_TARGET = globals().get('MAX_TARGET', None)
    if MAX_TARGET is None or MAX_TARGET < 0:
        MAX_TARGET = 200

    # ensure models on device & eval
    for m in (vit_model, resnet_model, densenet_model, species_model):
        if m is not None:
            try:
                m.to(device)
                m.eval()
            except Exception:
                pass

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(data_loader, desc=f"Base Predictions ({mode})", unit="batch")):
            # Unpack defensively (keep your original batch shapes logic)
            t_slice = None
            if len(batch) == 7:
                images, tabular, _spec,target_idx_batch, targets, sample_ids, target_names = batch
            elif len(batch) == 5:
                if mode in ['train', 'val']:
                    images, tabular, targets, sample_ids, target_names = batch
                else:
                    images, tabular, _spec, sample_ids, target_names = batch
                    targets = None
            elif len(batch) == 4:
                images, targets, sample_ids, target_names = batch
                tabular = None
            else:
                raise ValueError(f"Unexpected batch length: {len(batch)}")

            # normalize lists for sample_ids & target_names
            try:
                sample_ids_list = list(sample_ids) if not isinstance(sample_ids, str) else [sample_ids]
            except Exception:
                sample_ids_list = [sample_ids] if sample_ids is not None else []
            try:
                target_names_list = list(target_names) if target_names is not None and not isinstance(target_names, str) else ([target_names] if target_names is not None else [])
            except Exception:
                target_names_list = [target_names] if target_names is not None else []

            sample_ids_list = [str(s) for s in sample_ids_list] if len(sample_ids_list) else []
            target_names_list = [str(t) for t in target_names_list] if len(target_names_list) else []

            # move tensors to device where possible
            try:
                images = images.to(device)
            except Exception:
                pass
            try:
                if tabular is not None:
                    tabular = tabular.to(device)
                    target_start_idx = 25
                    try:
                        raw_target_idx = torch.argmax(tabular[:, target_start_idx:target_start_idx+5], dim=-1)
                        # FIX #1: Reconstruct raw_target_idx from target_names_list
                        # This ensures different target types get different indices
                        if raw_target_idx is not None and len(target_names_list) > 0:
                            TARGET_NAMES_ORDER = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
                            image_level_target_idx = []
                            patches_per_row_temp = CONFIG.get('num_patches', 8)
                            
                            for i in range(0, len(target_names_list), patches_per_row_temp):
                                target_name = target_names_list[i] if i < len(target_names_list) else 'Dry_Clover_g'
                                if target_name in TARGET_NAMES_ORDER:
                                    idx = TARGET_NAMES_ORDER.index(target_name)
                                else:
                                    idx = 0
                                image_level_target_idx.append(idx)
                            
                            raw_target_idx = torch.tensor(image_level_target_idx, dtype=torch.long, device=device)
                            logger.debug(f"FIX #1: Reconstructed raw_target_idx from target_names: {image_level_target_idx}")

                    except Exception:
                        raw_target_idx = None
                else:
                    raw_target_idx = None
            except Exception:
                raw_target_idx = None

            total_patches = int(images.shape[0])

            # infer patches_per_row safely (prefer sample_ids when possible)
            n_unique_ids = len(list(dict.fromkeys(sample_ids_list))) if len(sample_ids_list) > 0 else 0
            inferred_ppr = None
            if n_unique_ids > 0:
                inferred = total_patches // n_unique_ids if n_unique_ids > 0 else 0
                if inferred >= 1:
                    inferred_ppr = int(inferred)
            patches_per_row = int(inferred_ppr) if (inferred_ppr is not None and inferred_ppr > 0) else int(CONFIG.get('num_patches', 8))
            if patches_per_row <= 0:
                patches_per_row = int(CONFIG.get('num_patches', 8))
            if patches_per_row_detected is None:
                patches_per_row_detected = patches_per_row
            elif patches_per_row_detected != patches_per_row:
                logger.warning("Varying patches_per_row across batches: previous=%s now=%s", patches_per_row_detected, patches_per_row)

            # determine n_images using unique sample ids when possible
            unique_ids = []
            try:
                unique_ids = get_unique_sample_id(sample_ids_list, patches_per_image=patches_per_row)
            except Exception:
                unique_ids = []
            if len(unique_ids) > 0:
                n_images = len(unique_ids)
            else:
                n_images = int(total_patches // patches_per_row) if patches_per_row > 0 else 0

            # --- forward passes (defensive) ---
            try:
                # VIT
                try:
                    if use_mil:
                        # FIX #2: Use reconstructed raw_target_idx directly (no slicing)
                        t_slice = raw_target_idx if isinstance(raw_target_idx, torch.Tensor) else None
                        if t_slice is not None and t_slice.numel() < n_images:
                            pad_size = n_images - t_slice.numel()
                            t_slice = torch.cat([t_slice, torch.zeros(pad_size, dtype=torch.long, device=device)])
                            logger.debug(f"FIX #2: Padded t_slice to {t_slice}")
                        else:
                            logger.debug(f"FIX #2: Using reconstructed t_slice: {t_slice}")
                        
                        vit_out = vit_model(images, t_slice, return_features=False)

                    else:
                        vit_out = vit_model(images, raw_target_idx, return_features=False)
                    vit_preds = np.asarray(vit_out.detach().cpu().numpy()).ravel()
                except Exception as e:
                    logger.exception("vit forward failed in batch %d: %s", batch_idx, e)
                    n_expected = n_images if use_mil else total_patches
                    vit_preds = np.zeros((n_expected,), dtype=float)

                # RESNET
                try:
                    if use_mil:
                        t_slice = raw_target_idx if isinstance(raw_target_idx, torch.Tensor) else None
                        if t_slice is not None and t_slice.numel() < n_images:
                            pad_size = n_images - t_slice.numel()
                            t_slice = torch.cat([t_slice, torch.zeros(pad_size, dtype=torch.long, device=device)])
                            logger.debug(f"FIX #2: Padded t_slice to {t_slice}")
                        else:
                            logger.debug(f"FIX #2: Using reconstructed t_slice: {t_slice}")
                        resnet_out = resnet_model(images, t_slice, return_features=False)
                    else:
                        resnet_out = resnet_model(images, raw_target_idx, return_features=False)
                    resnet_preds = np.asarray(resnet_out.detach().cpu().numpy()).ravel()
                except Exception as e:
                    logger.exception("resnet forward failed in batch %d: %s", batch_idx, e)
                    n_expected = n_images if use_mil else total_patches
                    resnet_preds = np.zeros((n_expected,), dtype=float)

                # DENSENET
                try:
                    if use_mil:
                        t_slice = raw_target_idx if isinstance(raw_target_idx, torch.Tensor) else None
                        if t_slice is not None and t_slice.numel() < n_images:
                            pad_size = n_images - t_slice.numel()
                            t_slice = torch.cat([t_slice, torch.zeros(pad_size, dtype=torch.long, device=device)])
                            logger.debug(f"FIX #2: Padded t_slice to {t_slice}")
                        else:
                            logger.debug(f"FIX #2: Using reconstructed t_slice: {t_slice}")
                        densenet_out = densenet_model(images, t_slice, return_features=False)
                    else:
                        densenet_out = densenet_model(images, raw_target_idx, return_features=False)
                    densenet_preds = np.asarray(densenet_out.detach().cpu().numpy()).ravel()
                except Exception as e:
                    logger.exception("densenet forward failed in batch %d: %s", batch_idx, e)
                    n_expected = n_images if use_mil else total_patches
                    densenet_preds = np.zeros((n_expected,), dtype=float)

            except Exception as e:
                logger.exception("Unexpected forward pass error in batch %d: %s", batch_idx, e)
                vit_preds = np.zeros((0,), dtype=float)
                resnet_preds = np.zeros((0,), dtype=float)
                densenet_preds = np.zeros((0,), dtype=float)

            # clip preds
            vit_preds = np.clip(vit_preds, 0.0, MAX_TARGET)
            resnet_preds = np.clip(resnet_preds, 0.0, MAX_TARGET)
            densenet_preds = np.clip(densenet_preds, 0.0, MAX_TARGET)

            # --- aggregate to image-level if necessary ---
            if use_mil:
                vit_image_preds = vit_preds
                resnet_image_preds = resnet_preds
                densenet_image_preds = densenet_preds
                n_images_eff = len(vit_image_preds)
                logger.debug(f"MIL mode: received {n_images_eff} image-level predictions (shape={vit_preds.shape})")
            else:
                logger.debug(f"Non-MIL mode: received {len(vit_preds)} per-patch predictions (shape={vit_preds.shape})")

                vit_image_med = aggregate_blocks(vit_preds, patches_per_row, method='median')
                resnet_image_med = aggregate_blocks(resnet_preds, patches_per_row, method='median')
                densenet_image_med = aggregate_blocks(densenet_preds, patches_per_row, method='median')

                vit_image_trim = aggregate_blocks(vit_preds, patches_per_row, method='trimmed_mean')
                resnet_image_trim = aggregate_blocks(resnet_preds, patches_per_row, method='trimmed_mean')
                densenet_image_trim = aggregate_blocks(densenet_preds, patches_per_row, method='trimmed_mean')

                vit_image_preds = vit_image_trim if len(vit_image_trim) == len(vit_image_med) else vit_image_med
                resnet_image_preds = resnet_image_trim if len(resnet_image_trim) == len(resnet_image_med) else resnet_image_med
                densenet_image_preds = densenet_image_trim if len(densenet_image_trim) == len(densenet_image_med) else densenet_image_med

                n_images_eff = len(vit_image_preds)
                logger.debug(f"Non-MIL mode: aggregated to {n_images_eff} image-level predictions")

            # --- tabular -> per-image (take first row of each block) ---
            try:
                if tabular is not None:
                    tabular_np = tabular.cpu().numpy()
                else:
                    tabular_np = np.zeros((total_patches, TOTAL_TABULAR_DIM))
            except Exception:
                tabular_np = np.zeros((total_patches, TOTAL_TABULAR_DIM))

            TARGET_NAMES_ORDER = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
            tabular_image = []
            
            for i in range(0, total_patches, patches_per_row):
                idx = i
                if idx < tabular_np.shape[0]:
                    row = tabular_np[idx, :TOTAL_TABULAR_DIM].copy()  # COPY to avoid modifying original
                    if row.shape[0] < TOTAL_TABULAR_DIM:
                        row = np.pad(row, (0, max(0, TOTAL_TABULAR_DIM - row.shape[0])), 'constant')
                else:
                    row = np.zeros((TOTAL_TABULAR_DIM,), dtype=float)
                
                # FIX #3: Reconstruct target-name one-hot encoding (indices 25:30)
                patch_group_idx = i // patches_per_row  # Which image/target group is this?
                if patch_group_idx < len(target_names_list):
                    target_name = target_names_list[patch_group_idx]
                    if target_name in TARGET_NAMES_ORDER:
                        target_idx = TARGET_NAMES_ORDER.index(target_name)
                        # Create one-hot encoding for this target
                        one_hot = np.zeros(5, dtype=float)
                        one_hot[target_idx] = 1.0
                        # Replace the target encoding in the feature vector
                        row[25:30] = one_hot
                        logger.debug(f"FIX #3: Set target encoding for {target_name} (idx={target_idx}): {one_hot}")
                
                tabular_image.append(row)
            
            tabular_image = np.array(tabular_image, dtype=float)
            logger.debug(f"FIX #3: tabular_image shape={tabular_image.shape}, target encodings (col 25-30):\n{tabular_image[:, 25:30]}")


            # --- xgboost predictions per-image ---
            xgb_image_preds = np.zeros(n_images_eff, dtype=np.float32)
            for row_idx in range(n_images_eff):
                if use_mil:
                    tn_idx = row_idx
                else:
                    tn_idx = row_idx * patches_per_row
                if tn_idx < len(target_names_list):
                    target_name = target_names_list[tn_idx]
                elif row_idx < len(target_names_list):
                    target_name = target_names_list[row_idx]
                else:
                    target_name = None
                
                # ← NEW: Boost target type columns to match training (3x repeat)
                X_target_base = tabular_image[row_idx:row_idx+1, :25]        # First 25 cols
                target_type_boost = tabular_image[row_idx:row_idx+1, 25:30]   # Target type (5 cols)
                X_target = np.hstack([
                    X_target_base,
                    target_type_boost,
                    target_type_boost,
                    target_type_boost
                ])
                
                try:
                    xgb_pred = xgb_model.predict(X_target, target_name)
                except Exception:
                    try:
                        xgb_pred = xgb_model.predict(X_target)
                    except Exception:
                        xgb_pred = np.array([0.0])
                xgb_image_preds[row_idx] = float(np.clip(np.asarray(xgb_pred).ravel()[0], 0.0, MAX_TARGET))


            # canonical per-image sample ids: prefer get_unique_sample_id result if lengths match
            if len(unique_ids) == n_images_eff:
                batch_sample_ids = unique_ids
            else:
                # fallback: first id of each block
                batch_sample_ids = []
                for i in range(0, min(total_patches, patches_per_row * n_images_eff), patches_per_row):
                    if i < len(sample_ids_list):
                        batch_sample_ids.append(sample_ids_list[i])
                    else:
                        batch_sample_ids.append("None")
                batch_sample_ids = batch_sample_ids[:n_images_eff]

            # canonical per-image target names
            batch_target_names = []
            for i in range(n_images_eff):
                if use_mil:
                    tn_idx = i
                else:
                    tn_idx = i * patches_per_row
                if tn_idx < len(target_names_list):
                    batch_target_names.append(target_names_list[tn_idx])
                elif i < len(target_names_list):
                    batch_target_names.append(target_names_list[i])
                else:
                    batch_target_names.append(None)

            # collect
            all_vit_preds.append(vit_image_preds)
            all_resnet_preds.append(resnet_image_preds)
            all_densenet_preds.append(densenet_image_preds)
            all_xgb_preds.append(xgb_image_preds)
            all_tabular.append(tabular_image)
            all_sample_ids.extend(batch_sample_ids)
            all_target_names.extend(batch_target_names)
            
            if t_slice is not None:
                if isinstance(t_slice, torch.Tensor):
                    all_target_idx.append(t_slice.detach().cpu().numpy())
                else:
                    all_target_idx.append(np.asarray(t_slice))
            else:
                # If no target_idx, append placeholder
                all_target_idx.append(np.array([-1] * n_images_eff))
                
            # Targets -> robust image-level aggregation
            if mode in ['train', 'val'] and targets is not None:
                try:
                    # get numpy flat
                    try:
                        t_np = targets.cpu().numpy().ravel() if isinstance(targets, torch.Tensor) else np.asarray(targets).ravel()
                    except Exception:
                        t_np = np.asarray(targets).ravel()
                    if use_mil:
                        # if t_np already image-level length matches n_images_eff -> use it
                        if t_np.size == n_images_eff:
                            t_image = t_np[:n_images_eff]
                        else:
                            t_image = _img_targets_fn(t_np, n_images_eff, patches_per_row, device=device)
                    else:
                        # non-MIL: re-group patches -> image mean
                        if t_np.size == n_images_eff:
                            t_image = t_np
                        else:
                            t_image = _img_targets_fn(t_np, n_images_eff, patches_per_row, device=device)
                except Exception as e:
                    logger.warning(f"Failed to compute image-level targets for batch {batch_idx}: {e}")
                    t_image = np.zeros((n_images_eff,), dtype=float)
                if isinstance(t_image, torch.Tensor):
                    t_image = t_image.detach().cpu().numpy().ravel()
                else:
                    t_image = np.asarray(t_image).ravel()
            
                all_targets.append(t_image)


    # ===== concatenate safely =====
    vit_preds_all = np.concatenate(all_vit_preds) if len(all_vit_preds) else np.array([])
    resnet_preds_all = np.concatenate(all_resnet_preds) if len(all_resnet_preds) else np.array([])
    densenet_preds_all = np.concatenate(all_densenet_preds) if len(all_densenet_preds) else np.array([])
    xgb_preds_all = np.concatenate(all_xgb_preds) if len(all_xgb_preds) else np.array([])

    if len(all_target_idx) > 0:
        target_idx_all = np.concatenate(all_target_idx).ravel().astype(int)
    else:
        target_idx_all = np.array([], dtype=int)
        
    # STACK tabular robustly and pad/truncate columns to TOTAL_TABULAR_DIM
    if len(all_tabular) == 0:
        tabular_all = np.zeros((0, TOTAL_TABULAR_DIM), dtype=float)
    else:
        fixed_tab_list = []
        for arr in all_tabular:
            arr = np.asarray(arr, dtype=float)
            if arr.ndim == 1:
                arr = arr.reshape(-1, arr.shape[0])
            if arr.shape[1] < TOTAL_TABULAR_DIM:
                arr = np.pad(arr, ((0,0),(0, TOTAL_TABULAR_DIM - arr.shape[1])), 'constant', constant_values=0.0)
            elif arr.shape[1] > TOTAL_TABULAR_DIM:
                arr = arr[:, :TOTAL_TABULAR_DIM]
            fixed_tab_list.append(arr)
        tabular_all = np.vstack(fixed_tab_list) if fixed_tab_list else np.zeros((0, TOTAL_TABULAR_DIM), dtype=float)

    # ensemble 
    if vit_preds_all.size:
        stack = np.vstack([vit_preds_all, resnet_preds_all, densenet_preds_all]).astype(np.float64)
        stack = np.nan_to_num(stack, nan=0.0, posinf=1e8, neginf=0.0)
        med = np.median(stack, axis=0, keepdims=True)
        abs_dev = np.abs(stack - med)
        pseudo_var_model = (abs_dev + 1e-6) ** 2
        inv_var_model = 1.0 / (pseudo_var_model + 1e-12)

        temp = 1.0
        inv_var_adj = inv_var_model ** (1.0 / temp)
        max_w = 0.9
        weights = inv_var_adj / (inv_var_adj.sum(axis=0, keepdims=True) + 1e-12)
        weights = np.minimum(weights, max_w)
        weights = weights / (weights.sum(axis=0, keepdims=True) + 1e-12)

        ensemble_preds = np.sum(weights * stack, axis=0)
        overall_var = np.var(stack, axis=0)
        low_var_mask = overall_var < 1e-3
        if np.any(low_var_mask):
            ensemble_preds[low_var_mask] = np.median(stack[:, low_var_mask], axis=0)
        ensemble_preds = np.clip(ensemble_preds.astype(np.float32), 0.0, MAX_TARGET)
    else:
        ensemble_preds = np.array([])
        
    if vit_preds_all.size:
        # XGBoost Quality Check: Replace Values >10% Different from Ensemble
        # ═════════════════════════════════════════════════════════════════════════
    
        # Step 1: Calculate percentage difference from ensemble
        # percentage_diff = |xgb - ensemble| / ensemble * 100
        percentage_diff = np.abs(xgb_preds_all - ensemble_preds) / (np.abs(ensemble_preds) + 1e-6) * 100
    
        # Step 2: Define threshold
        PERCENT_THRESHOLD = 10.0  # ±10% difference
        # Tune: 5.0=strict (5%), 15.0=lenient (15%), 20.0=very lenient (20%)
    
        # Step 3: Identify outliers (difference > 10%)
        outlier_mask = percentage_diff > PERCENT_THRESHOLD
    
        # Step 4: Replace ONLY outlier values with DenseNet
        xgb_preds_all_fixed = xgb_preds_all.copy()
        xgb_preds_all_fixed[outlier_mask] = ensemble_preds[outlier_mask]
    
        # Step 5: Logging
        num_outliers = outlier_mask.sum()
        pct_outliers = (num_outliers / len(xgb_preds_all)) * 100 if len(xgb_preds_all) > 0 else 0
    
        logger.info(f"[XGBoost Quality Check]")
        logger.info(f"  Threshold: ±{PERCENT_THRESHOLD}% difference from ensemble")
        logger.info(f"  Percentage difference: mean={percentage_diff.mean():.2f}%, max={percentage_diff.max():.2f}%")
        logger.info(f"  Outliers detected: {num_outliers}/{len(xgb_preds_all)} ({pct_outliers:.1f}%)")
        if num_outliers > 0:
            logger.info(f"  Replaced {num_outliers} values with Ensemble Average")
    
        # Step 6: Use updated XGBoost
        xgb_preds_all = xgb_preds_all_fixed
    
    base_predictions = np.column_stack([vit_preds_all, resnet_preds_all, densenet_preds_all, xgb_preds_all, ensemble_preds]) if vit_preds_all.size else np.zeros((0,5), dtype=np.float32)

    # final sanity checks (preserve previous behavior)
    if base_predictions.shape[0] != len(all_sample_ids):
        logger.error("MISMATCH: %d predictions vs %d sample_ids", base_predictions.shape[0], len(all_sample_ids))
        raise ValueError("Predictions and sample IDs don't align!")

    if len(all_sample_ids) != len(all_target_names):
        logger.error("MISMATCH: %d sample_ids vs %d target_names", len(all_sample_ids), len(all_target_names))
        raise ValueError("Sample IDs and target names don't align!")

    # ensure tabular_all shape
    if tabular_all.size != 0:
        if tabular_all.ndim == 1:
            tabular_all = tabular_all.reshape(-1, TOTAL_TABULAR_DIM)
        if tabular_all.shape[1] != TOTAL_TABULAR_DIM:
            if tabular_all.shape[1] > TOTAL_TABULAR_DIM:
                tabular_all = tabular_all[:, :TOTAL_TABULAR_DIM]
            else:
                pad_width = TOTAL_TABULAR_DIM - tabular_all.shape[1]
                tabular_all = np.pad(tabular_all, ((0,0),(0,pad_width)), 'constant', constant_values=0.0)

    result = {
        'base_predictions': base_predictions,
        'vit': vit_preds_all,
        'resnet': resnet_preds_all,
        'densenet': densenet_preds_all,
        'xgboost': xgb_preds_all,
        'ensemble': ensemble_preds,
        'tabular': tabular_all,
        'sample_ids': all_sample_ids,
        'target_idx': target_idx_all,
        'target_names': np.array(all_target_names),
        'patches_per_row': patches_per_row_detected if patches_per_row_detected is not None else patches_per_row
    }

    if mode in ['train', 'val']:
        targets_all = np.concatenate(all_targets) if len(all_targets) > 0 else np.array([])
        result['targets'] = targets_all

    logger.info("BASE PREDICTIONS SHAPES: base=%s vit=%s resnet=%s densenet=%s xgb=%s ensemble=%s tabular=%s samples=%d (use_mil=%s)",
                result['base_predictions'].shape, result['vit'].shape, result['resnet'].shape,
                result['densenet'].shape, result['xgboost'].shape, result['ensemble'].shape if hasattr(result['ensemble'], 'shape') else (), 
                result['tabular'].shape if hasattr(result['tabular'], 'shape') else (), len(result['sample_ids']), use_mil)
    try:
        N = 5
        logger.info("\n--- First few base_predictions rows ---")
        logger.info(result['base_predictions'][:N])
    except Exception as e:
        logger.error(f"DEBUG LOGGING FAILED: {e}")
    return result



logger.info("✓ Ensemble Training")

## 9: SUBMISSION GENERATION

**Purpose**: Format predictions into competition submission CSV with robust error handling and flexible scaling.

---

### Overview

The updated submission generation now:
- ✅ Works with **any number of test rows** (1, 5, 50, 500, 4000+)
- ✅ Validates test data structure and integrity
- ✅ Handles GPU memory errors with automatic CPU fallback
- ✅ Automatically fixes NaN/Inf predictions
- ✅ Provides comprehensive error handling and logging

---

### Required Format

**CSV Structure** :
```
sample_id,target
test_1__Dry_Green_g,12345.6700
test_1__Dry_Dead_g,8901.2300
test_1__Dry_Clover_g,3456.7800
test_1__GDM_g,15678.9000
test_1__Dry_Total_g,24035.6800
```

**Key Features**:
- Column 1: `sample_id` (format: `{ImageID}__{TargetType}`)
- Column 2: `target` (biomass predictions, 4 decimal places)
- **Flexible rows**: Works with any number of rows


---

### DataFrame Creation

- Validation Checks

- 1. Structure Validation

- 2. Value Validation

- 3. Target Type Distribution

- 4. Component Consistency Check (If Dry_Total_g exists)

- Save to CSV

---

### Output Logging Example

```
================================================================================
GENERATING SUBMISSION FILE WITH VALIDATION
================================================================================

[STEP 1/5] Validating test data structure...
[STEP 2/5] Validating prediction data...
[STEP 3/5] Generating predictions...
[STEP 4/5] Cleaning predictions...
[STEP 5/5] Validating submission format...

────────────────────────────────────────────────────────────────────────────────
SUBMISSION FORMAT VALIDATION
────────────────────────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────────────────────────
FINAL SUBMISSION PREVIEW:
────────────────────────────────────────────────────────────────────────────────
  [0] ID1001187975__Dry_Clover_g              → 2.338747
  [1] ID1001187975__Dry_Dead_g                → 13.354437
  [2] ID1001187975__Dry_Green_g               → 14.033021
  [3] ID1001187975__Dry_Total_g               → 31.075960
  [4] ID1001187975__GDM_g                     → 22.634264

================================================================================
✓ SUBMISSION GENERATION COMPLETE
✓ File saved: /kaggle/working/submission.csv
✓ Total: 5 rows × 2 columns
================================================================================
```
---

In [1]:
def _extract_target_idx_from_tabular(tab_np, device):
    """
    Extract target indices from tabular one-hot encoding.
    
    Args:
        tab_np: numpy array (N, D) with one-hot at columns [25:30]
        device: torch device
    
    Returns:
        target_idx: LongTensor of shape (N,) on device
    """
    try:
        if tab_np is None:
            return torch.zeros(0, dtype=torch.long, device=device)
        
        if not isinstance(tab_np, np.ndarray):
            tab_np = np.asarray(tab_np)
        
        if tab_np.size == 0:
            return torch.zeros(0, dtype=torch.long, device=device)
        
        if tab_np.ndim == 1:
            tab_np = tab_np.reshape(1, -1)
        
        N = int(tab_np.shape[0])
        D = int(tab_np.shape[1])
        
        # Extract one-hot region (columns 25:30 = 5 target types)
        TARGET_ONEHOT_START = 25
        NUM_TARGETS = 5
        
        if D >= TARGET_ONEHOT_START + NUM_TARGETS:
            one_hot = tab_np[:, TARGET_ONEHOT_START:TARGET_ONEHOT_START + NUM_TARGETS]
            target_idx = np.argmax(one_hot, axis=1)  # shape (N,)
            return torch.from_numpy(target_idx.astype(np.int64)).to(device).long()
        else:
            logger.warning(
                f"Tabular width {D} < required {TARGET_ONEHOT_START + NUM_TARGETS}. "
                f"Returning zeros."
            )
            return torch.zeros(N, dtype=torch.long, device=device)
    
    except Exception as e:
        logger.warning(f"Failed to extract target_idx from tabular: {e}")

        try:
            N = int(tab_np.shape[0]) if (tab_np is not None and hasattr(tab_np, 'shape')) else 0
            return torch.zeros(N, dtype=torch.long, device=device)
        except:
            return torch.zeros(0, dtype=torch.long, device=device)


def get_default_device():
    """Return default device (prefers cuda if available)."""
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def run_inference_in_batches(model, X_np, tab_np, device=None, batch_size=1024):
    """
    Safe batched inference for meta-learner with TaskEmbedding.
    
    UPDATED: Now extracts target_idx from tabular and passes it to model.
    
    Args:
        model: MetaLearnerModel with TaskEmbedding
        X_np: Base model predictions (N, 5)
        tab_np: Tabular features with one-hot at [25:30]
        device: torch device
        batch_size: batch size for inference
    
    Returns:
        1-D numpy array of predictions
    """
    if device is None:
        device = get_default_device()
    if isinstance(device, str):
        device = torch.device(device)

    model = model.to(device)
    model.eval()

    preds = []
    N = int(X_np.shape[0])

    try:
        with torch.no_grad():
            for i in range(0, N, batch_size):
                end_idx = min(i + batch_size, N)
                xb = torch.from_numpy(X_np[i:end_idx]).float().to(device)
                

                target_idx_batch = _extract_target_idx_from_tabular(tab_np[i:end_idx], device)

                # Sanity checks
                if xb.ndim != 2:
                    logger.warning(f"Warning: xb has wrong shape: {xb.shape}, expected 2D")
                if target_idx_batch.ndim != 1:
                    logger.warning(f"Warning: target_idx has wrong shape: {target_idx_batch.shape}, expected 1D")
                
                if xb.shape[0] != target_idx_batch.shape[0]:
                    logger.warning(
                        f"Batch size mismatch: xb={xb.shape}, target_idx={target_idx_batch.shape}"
                    )
                    continue


                out = model(xb, target_idx_batch)  # [batch_size] predictions
                out = out.cpu().numpy().ravel()
                preds.append(out)

    except Exception as e:
        logger.error("\n===== FATAL ERROR DURING INFERENCE =====")
        logger.error(f"Error message: {e}")
        logger.error("=========================================\n")
        logger.debug(f"xb shape: {xb.shape if 'xb' in locals() else 'N/A'}")
        logger.debug(f"target_idx_batch shape: {target_idx_batch.shape if 'target_idx_batch' in locals() else 'N/A'}")
        raise

    if len(preds) == 0:
        return np.zeros((0,), dtype=np.float32)

    preds_arr = np.concatenate(preds, axis=0)

    logger.info(
        "DEBUG: run_inference_in_batches -> preds.shape=%s, min=%.4f, max=%.4f, nan_count=%d",
        preds_arr.shape,
        (np.nanmin(preds_arr) if preds_arr.size else -1),
        (np.nanmax(preds_arr) if preds_arr.size else -1),
        int(np.isnan(preds_arr).sum()) if preds_arr.size else 0
    )

    return preds_arr

def generate_submission(meta_learner, test_data, output_path, test_df=None, device=None,
                        batch_size_gpu=1024, batch_size_cpu=256, float_format="%.4f"):
    """
    Runs batched inference using updated meta-learner with TaskEmbedding.
    
    UPDATED: Now handles target_idx extraction internally.
    """
    if device is None:
        device = get_default_device()
    if isinstance(device, str):
        device = torch.device(device)

    logger.info("\n" + "=" * 70)
    logger.info("SUBMISSION GENERATION (META-LEARNER WITH TASK EMBEDDING)")
    logger.info("=" * 70)
    
    MAX_TARGET = globals().get('MAX_TARGET', None)
    if MAX_TARGET is None or MAX_TARGET < 0:
        MAX_TARGET = 200
        
    try:
        # -------- SANITY CHECK --------
        if test_data is None:
            logger.error("✗ test_data is None - returning empty DataFrame")
            return pd.DataFrame(columns=["sample_id", "target"])

        for key in ("base_predictions", "tabular", "sample_ids"):
            if key not in test_data:
                logger.error(f"✗ test_data missing key: {key}")
                return pd.DataFrame(columns=["sample_id", "target"])

        # -------- CONVERT INPUTS --------
        try:
            X_np = np.asarray(test_data["base_predictions"], dtype=np.float32)
            tab_np = np.asarray(test_data["tabular"], dtype=np.float32)
            sample_ids = [str(s).strip() for s in test_data["sample_ids"]]
        except Exception as e:
            logger.error(f"✗ Error converting test_data arrays: {e}")
            logger.debug(traceback.format_exc())
            return pd.DataFrame(columns=["sample_id", "target"])

        # -------- SHAPE CHECK --------
        if X_np.shape[0] != tab_np.shape[0]: 
            min_len = min(X_np.shape[0], tab_np.shape[0]) 
            logger.warning(
                "⚠ Shape mismatch between base_predictions and tabular. "
                "Truncating both to same length: %d", min_len  
            )
            X_np = X_np[:min_len]
            tab_np = tab_np[:min_len]
            sample_ids = sample_ids[:min_len]
        
        logger.info(
            "Test data sizes: predictions_rows=%d, tabular_rows=%d, sample_ids=%d",
            X_np.shape[0], tab_np.shape[0], len(sample_ids)  # ✅ Now integers
        )


        # -------- DEVICE INFERENCE --------
        try:
        
            final_predictions = run_inference_in_batches(
                meta_learner, X_np, tab_np, device=device, batch_size=batch_size_gpu
            )
            logger.info(f"✓ Inference completed on device: {device}")
        except Exception as e:
            logger.warning(f"⚠ Device inference raised exception: {e}")
            logger.debug(traceback.format_exc())
            logger.info("→ Trying CPU fallback...")

            try:
                final_predictions = run_inference_in_batches(
                    meta_learner, X_np, tab_np,
                    device=torch.device("cpu"),
                    batch_size=batch_size_cpu
                )
                logger.info("✓ CPU fallback successful")
            except Exception as e2:
                logger.error(f"✗ CPU fallback also failed: {e2}")
                logger.debug(traceback.format_exc())
                return pd.DataFrame(columns=["sample_id", "target"])

        final_predictions = np.asarray(final_predictions).ravel()


        # -------- LENGTH MISMATCH PROTECTION --------
        if final_predictions.shape[0] != len(sample_ids):  # ✅ Compare integers
            logger.warning(
                "⚠ Prediction mismatch: predictions=%d, sample_ids=%d. "
                "Aligning by truncation.",
                final_predictions.shape[0], len(sample_ids)  # ✅ Both integers
            )
            min_len = min(final_predictions.shape[0], len(sample_ids))  # ✅ Both integers
            final_predictions = final_predictions[:min_len]
            sample_ids = sample_ids[:min_len]


        # -------- CLEAN NANS --------
        final_predictions = np.nan_to_num(final_predictions, nan=np.nan)
        nan_mask = np.isnan(final_predictions)

        if nan_mask.any():
            fallback = float(np.nanmedian(final_predictions))
            if np.isnan(fallback):
                fallback = 0.0
                logger.warning("All predictions NaN → fallback = 0.0")
            else:
                logger.warning(f"Replacing {nan_mask.sum()} NaNs with fallback median = {fallback:.4f}")

            final_predictions[nan_mask] = fallback

        final_predictions = final_predictions.astype(float)
        final_predictions = np.clip(final_predictions, a_min=0, a_max=MAX_TARGET)
        
        logger.info(
            "Final predictions: min=%.4f, max=%.4f, mean=%.4f, std=%.4f",
            np.min(final_predictions), np.max(final_predictions),
            np.mean(final_predictions), np.std(final_predictions)
        )

        # -------- BUILD SUBMISSION DF --------
        submission_df = pd.DataFrame({
            "sample_id": sample_ids,
            "target": final_predictions
        })

        logger.info(f"✓ Submission DataFrame created: rows={len(submission_df)}")

        # -------- OPTIONAL CHECK AGAINST OFFICIAL test_df --------
        sorted_sample_ids = sorted(submission_df["sample_id"].tolist())

        if test_df is not None and "sample_id" in test_df.columns:
            official = sorted(test_df["sample_id"].astype(str).tolist())
            if sorted_sample_ids == official:
                logger.info("✓ Sorted sample_id exactly matches official ordering.")
            else:
                logger.warning("⚠ Sorted submission ids DO NOT match official test_df ordering.")
                logger.debug(f"  First 5 submission: {sorted_sample_ids[:5]}")
                logger.debug(f"  First 5 official:  {official[:5]}")
        else:
            logger.info("Official test_df not provided → skipping comparison.")

        # -------- SAVE --------
        submission_df = submission_df.sort_values("sample_id")
        submission_df.to_csv(output_path, index=False, float_format=float_format)
        logger.info(f"✓ Submission saved to {output_path}")
        logger.info(f"  Rows: {len(submission_df)}")
        logger.info(f"  File: {output_path}")

        return submission_df

    except Exception as e:
        logger.error(f"✗ Unexpected error in generate_submission: {e}")
        logger.debug(traceback.format_exc())
        return pd.DataFrame(columns=["sample_id", "target"])



logger.info("✓ Submission Function Defined")

NameError: name 'logger' is not defined

## 10: HELPER FUNCTIONS

**Purpose**: Utility functions for memory management and weights export.

### Memory Cleanup

```python
def cleanup_memory(model=None):
    '''Aggressive GPU memory cleanup between models'''
    
    if model is not None:
        model = model.cpu()  # Move to CPU
        del model            # Delete reference
    
    gc.collect()            # Garbage collection
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()          # Clear GPU cache
        torch.cuda.reset_peak_memory_stats()  # Reset counters
    
    # Log freed memory
    if torch.cuda.is_available():
        free_gb, total_gb = torch.cuda.mem_get_info()
        logger.info(f"GPU Memory: {free_gb/1e9:.2f}GB / {total_gb/1e9:.2f}GB")
```

**Usage**: Call between training different models to prevent OOM errors

### Weights Export

```python
def create_weights_zip_for_download(models_dir, output_dir):
    '''Create downloadable ZIP with trained model weights'''
    
    model_files = [
        'vit_best.pth',
        'resnet_best.pth',
        'densenet_best.pth',
        'xgboost_best.pkl',
        'species_best.pth',
        'metalearner_best.pth'
    ]
    
    # Find available files
    available = []
    for file_name in model_files:
        file_path = models_dir / file_name
        if file_path.exists():
            size_mb = file_path.stat().st_size / 1024**2
            available.append((file_name, size_mb))
    
    # Create ZIP
    with zipfile.ZipFile(output_path, 'w') as z:
        for file_name, _ in available:
            z.write(models_dir / file_name, arcname=file_name)
    
    logger.info(f"✓ ZIP created: {output_path}")
    logger.info(f"  - Files: {len(available)}")
    logger.info(f"  - Size: {total_size_mb:.1f}MB")
    
    return output_path
```

**Output Files** (Typical ~500-1000 MB total):
- vit_best.pth: ~200-250 MB
- resnet_best.pth: ~100-150 MB
- densenet_best.pth: ~80-120 MB
- xgboost_best.pkl: ~1-5 MB
- species_best.pth: ~200-250 MB
- metalearner_best.pth: ~1-2 MB

**Output**: 
- Memory cleanup utilities ready
- Weights ZIP prepared
- File ready for download/sharing

---

In [ ]:
def generate_synthetic_train_df_matching_schema(
    train_df, species_model, lookups_dict, raw_dir=RAW_DATA_DIR, device="cuda"
):
    """
    Generate synthetic train dataframe with EXACT SAME structure as train_df.
    
    PRIORITY:
      1) Use test_features_cache (dict with computed: ndvi, height, month, state, species)
      2) Decode species_batch (integer index) if species not in cache
      3) Fall back to original train_df values
      4) Use safe defaults
    """
    def _doy_to_date_str(doy, year=2015):
        try:
            doy_i = int(round(float(doy)))
        except Exception:
            doy_i = 180
        doy_i = max(1, min(365, doy_i))
        dt = datetime(year, 1, 1) + timedelta(days=doy_i - 1)
        return dt.strftime("%m/%d/%Y")

    def _month_to_doy(month):
        """Map month (1-12) to representative DOY (day of year)."""
        try:
            m = int(round(float(month)))
        except Exception:
            m = None
        
        if m is None or not (1 <= m <= 12):
            return None
        
        MONTH_TO_DOY = {
            1: 16, 2: 47, 3: 75, 4: 106, 5: 136, 6: 167,
            7: 197, 8: 228, 9: 259, 10: 289, 11: 320, 12: 350
        }
        return MONTH_TO_DOY[m]

    # Build dataset in 'test' mode
    dataset = BiomassDataset(
        dataframe=train_df,
        image_dir=raw_dir / 'train',
        transform=None,
        mode='test',
        train_df=train_df,
        lookups_dict=lookups_dict,
        species_model=species_model,
        device=device,
    )

    rows = []

    for i in tqdm(range(len(dataset)), desc="Synthetic train generation"):
        # Get dataloader output
        images, features, species_batch, target_idx_batch, targets_per_patch, sample_id, target_name = dataset[i]
        
        image_id = str(sample_id).split("__")[0].lower()
        original = train_df.iloc[i]

        #use the cached dict computed by _intelligent_feature_fetch        
        cached = dataset.test_features_cache.get(image_id, {}) if getattr(dataset, "test_features_cache", None) else {}
        
        logger.debug(f"Row {i} | image_id: {image_id} | cached keys: {list(cached.keys()) if cached else 'empty'}")


        # Extract values from cache 
        # ==========================
        ndvi_val = cached.get("ndvi", None)
        height_val = cached.get("height", cached.get("estimated_height", None))
        month_val = cached.get("month", None)
        doy_val = cached.get("doy", None)
        quarter_val = cached.get("quarter", None)
        state_val = cached.get("state", None)
        species_val = cached.get("predicted_species", None)

        # NDVI: cache -> original train_df -> None
        # =========================================
        if ndvi_val is None:
            ndvi_val = original.get("Pre_GSHH_NDVI", None)

        # HEIGHT: cache -> original train_df -> 7.6
        # ==========================================
        if height_val is None:
            height_val = original.get("Height_Ave_cm", None)
        try:
            if height_val is None or (isinstance(height_val, float) and np.isnan(height_val)):
                height_val = float(original.get("Height_Ave_cm", 7.6))
        except Exception:
            height_val = 7.6

        # SPECIES: cache -> species_batch index -> original train_df -> map to fixed
        # ============================================================================
        if species_val is None:
            # Try to decode species_batch (torch.LongTensor with integer index)
            try:
                if isinstance(species_batch, torch.Tensor):
                    species_idx = int(species_batch.item())
                else:
                    species_idx = int(species_batch)
                
                # Use dataset's idx_to_species mapping
                if hasattr(dataset, "idx_to_species") and dataset.idx_to_species:
                    species_val = dataset.idx_to_species.get(species_idx, None)
                elif hasattr(dataset, "species_to_idx") and dataset.species_to_idx:
                    idx_to_species = {v: k for k, v in dataset.species_to_idx.items()}
                    species_val = idx_to_species.get(species_idx, None)
                
                logger.debug(f"Row {i}: Mapped species_batch[{species_idx}] → {species_val}")
            except Exception as e:
                logger.debug(f"Row {i}: Failed to map species_batch: {e}")
                species_val = None
        
        # Fallback to original train_df
        if species_val is None:
            species_val = original.get("Species", None)
        
        # FINAL: Map to fixed species using map_to_fixed_species()
        species_val = map_to_fixed_species(species_val) if species_val else "Mixed"

        # STATE: cache -> original train_df -> lookups -> 'Tas'
        # =======================================================\
        if state_val is None:
            state_val = original.get("State", None)
        
        try:
            if isinstance(state_val, str):
                s = state_val.strip()
                REVERSE_STATE_MAP_INV = {'tas': 'Tas', 'nsw': 'NSW', 'wa': 'WA', 'vic': 'Vic'}
                key = s.lower()
                if key in REVERSE_STATE_MAP_INV:
                    state_val = REVERSE_STATE_MAP_INV[key]
                else:
                    unique_states = lookups_dict.get('UNIQUE_STATES', [])
                    if unique_states:
                        for u in unique_states:
                            if key == str(u).strip().lower() or key in str(u).strip().lower():
                                state_val = u
                                break
            if not state_val:
                state_val = lookups_dict.get('UNIQUE_STATES', ['Tas'])[0]
        except Exception:
            state_val = lookups_dict.get('UNIQUE_STATES', ['Tas'])[0]

        # DOY / MONTH: cache -> original Sampling_Date -> 180 (mid-year)
        # ============================================================================
        # Priority 1: Use cached DOY
        if doy_val is not None:
            try:
                doy_val = int(round(float(doy_val)))
                doy_val = max(1, min(365, doy_val))
            except Exception:
                doy_val = None

        # Priority 2: Use cached month and convert to DOY
        if doy_val is None and month_val is not None:
            try:
                month_val = int(round(float(month_val)))
                doy_val = _month_to_doy(month_val)
            except Exception:
                doy_val = None

        # Priority 3: Parse original Sampling_Date
        if doy_val is None:
            sd = original.get("Sampling_Date", None)
            if sd is not None:
                try:
                    dt = pd.to_datetime(sd, dayfirst=False, errors='coerce')
                    if pd.notna(dt):
                        doy_val = int(dt.dayofyear)
                except Exception:
                    doy_val = None

        # Compute final Sampling_Date
        if doy_val is not None:
            sampling_date = _doy_to_date_str(doy_val, year=2015)
        else:
            # Last resort: use original or mid-year default
            sampling_date = original.get("Sampling_Date", None)
            if sampling_date is None or (isinstance(sampling_date, float) and np.isnan(sampling_date)):
                sampling_date = _doy_to_date_str(180, year=2015)

        # Build final row
        row = {
            "sample_id": original["sample_id"],
            "image_path": original["image_path"],
            "Sampling_Date": sampling_date,
            "State": state_val,
            "Species": species_val,
            "Pre_GSHH_NDVI": ndvi_val,
            "Height_Ave_cm": height_val,
            "target_name": original["target_name"],
            "target": float(original["target"]),
        }

        rows.append(row)

    out_df = pd.DataFrame(rows)

    try:
        out_df = out_df[train_df.columns]
    except Exception:
        logger.warning("Generated synthetic DF columns differ from original train_df; returning generated column order")

    # Save to disk
    out_path = OUTPUTS_DIR / 'train_synthetic_matching.csv'
    out_df.to_csv(out_path, index=False)
    logger.info(f"Saved → {out_path}")
    return out_df

def _normalize_state_for_onehot(raw_state):
    """
    Return canonical State string exactly as used by the one-hot columns.
    E.g. 'Tas','NSW','Vic','WA' or '' for missing. Adjust to match training data.
    """
    if raw_state is None:
        return ''
    s = str(raw_state).strip()
    mappings = {
        'tasmania': 'Tas',
        'tas': 'Tas',
        'new south wales': 'NSW',
        'nsw': 'NSW',
        'western australia': 'WA',
        'wa': 'WA',
        'victoria': 'Vic',
        'vic': 'Vic'
    }
    low = s.lower()
    for k,v in mappings.items():
        if low == k:
            return v
    if len(s) <= 3:
        return s.upper()
    return s.title()

def diagnose_submission(test_data, test_df=None, meta_learner=None, device=None,
                        run_inference=False, batch_size_gpu=1024, batch_size_cpu=256,
                        float_format="%.4f"):
    """
    Diagnostic checks for submission creation. Uses logger.info to print checks.
    Returns a dict of diagnostics.
    
    Args:
        test_data: dict produced by get_base_model_predictions (must contain
                   'base_predictions', 'tabular', 'sample_ids' at minimum)
        test_df: optional official test DataFrame (with 'sample_id' column) for comparison
        meta_learner: optional trained model (will run inference if run_inference=True)
        device: torch device or None
        run_inference: bool - whether to run meta_learner inference to validate predictions
        batch_size_gpu / batch_size_cpu: inference batch sizes for fallback
    """
    diag = {
        "ok": True,
        "errors": [],
        "warnings": [],
        "notes": {},
    }

    # Basic presence checks
    required_keys = ("base_predictions", "tabular", "sample_ids")
    for k in required_keys:
        if (test_data is None) or (k not in test_data):
            msg = f"Missing required key in test_data: {k}"
            logger.error(msg)
            diag["errors"].append(msg)
            diag["ok"] = False
    if not diag["ok"]:
        return diag

    # Convert arrays/lists safely
    try:
        base_preds = np.asarray(test_data["base_predictions"])
        tabular = np.asarray(test_data["tabular"])
        sample_ids = [str(s).strip() for s in test_data["sample_ids"]]
    except Exception as e:
        msg = f"Error converting test_data arrays -> {e}"
        logger.error(msg)
        diag["errors"].append(msg)
        diag["ok"] = False
        return diag

    logger.info("DIAG: shapes: base_predictions=%s, tabular=%s, sample_ids=%d",
                getattr(base_preds, "shape", None),
                getattr(tabular, "shape", None),
                len(sample_ids))

    # Row-count consistency
    n_preds = base_preds.shape[0] if hasattr(base_preds, "shape") else -1
    n_tab = tabular.shape[0] if hasattr(tabular, "shape") else -1
    n_ids = len(sample_ids)
    logger.info("DIAG: rows => preds=%d, tabular=%d, sample_ids=%d", n_preds, n_tab, n_ids)

    if n_preds != n_tab:
        w = f"Row mismatch: base_predictions rows ({n_preds}) != tabular rows ({n_tab})"
        logger.warning("DIAG WARNING: %s", w)
        diag["warnings"].append(w)

    if n_preds != n_ids:
        e = f"Row mismatch: base_predictions rows ({n_preds}) != sample_ids ({n_ids})"
        logger.error("DIAG ERROR: %s", e)
        diag["errors"].append(e)
        diag["ok"] = False

    # Check for NaNs / inf / negative values in base preds
    try:
        preds_flat = base_preds.astype(np.float64).ravel()
        n_nan = int(np.isnan(preds_flat).sum())
        n_inf = int(np.isinf(preds_flat).sum())
        n_neg = int((preds_flat < 0.0).sum())
        logger.info("DIAG: base_predictions NaN=%d, Inf=%d, negative=%d", n_nan, n_inf, n_neg)
        diag["notes"]["pred_nan"] = n_nan
        diag["notes"]["pred_inf"] = n_inf
        diag["notes"]["pred_neg"] = n_neg
        if n_nan > 0 or n_inf > 0:
            diag["warnings"].append("base_predictions contains NaN or Inf values.")
        if n_neg > 0:
            diag["warnings"].append("base_predictions contains negative values (will be clipped in submission).")
    except Exception as e:
        logger.warning("DIAG: could not inspect base_predictions numerics: %s", e)
        diag["warnings"].append(f"could not inspect base_predictions numerics: {e}")

    # Check tabular NaNs / shape
    try:
        n_nan_tab = int(np.isnan(tabular).sum())
        logger.info("DIAG: tabular total NaNs=%d", n_nan_tab)
        diag["notes"]["tab_nan"] = n_nan_tab
        if n_nan_tab > 0:
            diag["warnings"].append("tabular contains NaNs")
    except Exception:
        pass

    # Check sample_id uniqueness and ordering
    try:
        unique_ids = set(sample_ids)
        n_unique = len(unique_ids)
        logger.info("DIAG: sample_ids unique=%d / total=%d", n_unique, n_ids)
        if n_unique != n_ids:
            logger.warning("DIAG WARNING: sample_ids are not unique (duplicates present).")
            # show example duplicates (up to 10)
            from collections import Counter
            c = Counter(sample_ids)
            dups = [k for k, v in c.items() if v > 1][:10]
            logger.info("DIAG: example duplicate ids (up to 10): %s", dups)
            diag["notes"]["sample_dup_examples"] = dups
    except Exception:
        pass

    # Lexicographic sort check and compare to official test_df if provided
    try:
        sorted_sample_ids = sorted(sample_ids)
        logger.info("DIAG: first 5 sample_ids (original): %s", sample_ids[:5])
        logger.info("DIAG: first 5 sample_ids (sorted):   %s", sorted_sample_ids[:5])
        if test_df is not None and "sample_id" in test_df.columns:
            official_ids = test_df["sample_id"].astype(str).tolist()
            logger.info("DIAG: official test_df sample_id count=%d", len(official_ids))
            if len(official_ids) != len(sorted_sample_ids):
                logger.warning("DIAG WARNING: official test_df length differs from submission candidate.")
                diag["warnings"].append("official test_df length differs")
            set_sub = set(sorted_sample_ids)
            set_off = set(official_ids)
            missing = sorted(list(set_off - set_sub))[:10]
            extra = sorted(list(set_sub - set_off))[:10]
            if missing:
                logger.error("DIAG ERROR: missing ids (up to 10): %s", missing)
                diag["errors"].append(f"missing_ids_example={missing}")
                diag["ok"] = False
            if extra:
                logger.error("DIAG ERROR: extra ids (up to 10): %s", extra)
                diag["errors"].append(f"extra_ids_example={extra}")
                diag["ok"] = False
            if not missing and not extra:
                logger.info("DIAG: sorted submission sample_ids match official test_df (set-wise).")
    except Exception as e:
        logger.warning("DIAG: lexicographic checks failed: %s", e)

    # If requested run_inference = True -> run meta_learner to produce final_predictions and validate length/NaN
    if run_inference:
        if meta_learner is None:
            msg = "run_inference=True but no meta_learner provided — skipping inference."
            logger.warning("DIAG: %s", msg)
            diag["warnings"].append(msg)
        else:
            try:
                # assume run_inference_in_batches available in scope
                preds = run_inference_in_batches(meta_learner, base_preds, tabular, device=device, batch_size=batch_size_gpu)
                preds = np.asarray(preds).ravel()
                logger.info("DIAG: meta_learner produced %d predictions", preds.shape[0])
                if preds.shape[0] != n_ids:
                    msg = f"Prediction length ({preds.shape[0]}) != sample_ids ({n_ids})"
                    logger.error("DIAG ERROR: %s", msg)
                    diag["errors"].append(msg)
                    diag["ok"] = False
                # numeric checks
                n_nan_p = int(np.isnan(preds).sum())
                n_neg_p = int((preds < 0.0).sum())
                logger.info("DIAG: final preds NaN=%d, negative=%d", n_nan_p, n_neg_p)
                diag["notes"]["final_pred_nan"] = n_nan_p
                diag["notes"]["final_pred_neg"] = n_neg_p
                if n_nan_p > 0:
                    diag["warnings"].append("final predictions contain NaN")
            except Exception as e:
                logger.exception("DIAG: inference run failed: %s", e)
                diag["errors"].append(f"inference_failed: {e}")
                diag["ok"] = False

    # Final summary
    if diag["ok"]:
        logger.info("DIAG: All checks passed (ok=True).")
    else:
        logger.error("DIAG: Some checks failed. See diag['errors'] and logs for details.")

    return diag


def diagnose_ensemble_batch(data_loader, num_patches=8):
    """
    Run a single-batch diagnostic for ensemble pipeline.
    This avoids spamming logs during full inference.
    """

    logger.info("\n" + "="*80)
    logger.info("🔍 ENSEMBLE PIPELINE DIAGNOSTICS (ONE BATCH)")
    logger.info("="*80)

    batch = next(iter(data_loader))

    # Robust unpack: new collate returns:
    # - with targets: (images, tabular, species_batch, targets_batch, sample_ids, target_names) -> len==6
    # - without targets: (images, tabular, species_batch, sample_ids, target_names) -> len==5
    if isinstance(batch, (tuple, list)):
        ln = len(batch)
    else:
        ln = 0

    if ln == 7:
        images, tabular, species_batch, target_idx_batch, targets, sample_ids, target_names = batch
    elif ln == 5:
        images, tabular, species_batch, sample_ids, target_names = batch
        targets = None
    else:
        # best-effort fallback (older/different signatures)
        try:
            # try to find sample_ids in last two items
            if ln >= 2:
                sample_ids = batch[-2]
                target_names = batch[-1]
            else:
                sample_ids = []
                target_names = []
            images = batch[0] if ln > 0 else None
            tabular = batch[1] if ln > 1 else None
        except Exception:
            logger.warning("diagnose_ensemble_batch: unexpected batch signature; aborting")
            return

    # ensure lists of strs
    try:
        sample_ids = [str(s) for s in sample_ids]
    except Exception:
        sample_ids = list(sample_ids)

    try:
        target_names = [str(t) for t in target_names]
    except Exception:
        target_names = list(target_names)

    total_patches = images.shape[0]
    unique_ids = list(dict.fromkeys(sample_ids))
    num_images = len(unique_ids)

    logger.info(f"Total patches:        {total_patches}")
    logger.info(f"Unique images:        {num_images}")
    logger.info(f"Expected patches/img: {num_patches}")

    # Check grouping
    grouped_ids = get_unique_sample_id(sample_ids, patches_per_image=num_patches)
    logger.info(f"Grouped sample_ids:   {grouped_ids}")
    logger.info(f"Grouped len:          {len(grouped_ids)}")

    # Patch grouping check
    rep_ok = all(sample_ids.count(uid) == num_patches for uid in unique_ids)
    if rep_ok:
        logger.info("✓ Perfect patch repetition detected per image")
    else:
        logger.warning("⚠ Patch repetition inconsistent!")

    # Target name grouping (take first of each image block)
    tn_grouped = []
    for i in range(0, total_patches, num_patches):
        if i < len(target_names):
            tn_grouped.append(target_names[i])
    logger.info(f"Grouped target_names: {tn_grouped}")

    # shapes
    logger.info(f"images.shape:   {images.shape}")
    logger.info(f"tabular.shape:  {tabular.shape}")
    if targets is not None:
        logger.info(f"targets.shape:  {targets.shape}")

    logger.info("="*80 + "\n")


def run_dataloader_diagnostics(data_loader, num_patches=8, device=device):
    """
    Runs a single-batch diagnostic on the dataloader to ensure that
    sample_ids, patch grouping, and shapes match the expected pipeline.
    """

    logger.info("\n" + "="*80)
    logger.info("🔍 RUNNING DATA LOADER DIAGNOSTICS")
    logger.info("="*80)

    batch = next(iter(data_loader))
    logger.info("✓ Pulled one batch from loader")

    # Robust unpacking - handle new collate return signatures
    if isinstance(batch, (tuple, list)):
        ln = len(batch)
    else:
        ln = 0

    if ln == 6:
        images, tabular, species_batch, targets, sample_ids, target_names = batch
    elif ln == 5:
        images, tabular, species_batch, sample_ids, target_names = batch
        targets = None
    else:
        # fallback best-effort
        try:
            images = batch[0] if ln > 0 else None
            tabular = batch[1] if ln > 1 else None
            sample_ids = batch[-2] if ln >= 2 else []
            target_names = batch[-1] if ln >= 1 else []
            targets = None
        except Exception:
            logger.warning("run_dataloader_diagnostics: unexpected batch signature; aborting")
            return

    # Convert to lists
    try:
        sample_ids_list = [str(s) for s in sample_ids]
    except Exception:
        sample_ids_list = list(sample_ids)

    # Basic counts
    total_patches = images.shape[0]
    unique_ids = list(dict.fromkeys(sample_ids_list))
    num_unique = len(unique_ids)

    logger.info(f"Total patches in batch      : {total_patches}")
    logger.info(f"Unique sample_ids           : {num_unique}")
    logger.info(f"Unique IDs (first 10)       : {unique_ids[:10]}")

    # Infer patches per image
    inferred_ppr = None
    if num_unique > 0:
        inferred_ppr = total_patches // num_unique

    logger.info(f"Inferred patches_per_image  : {inferred_ppr}")
    logger.info(f"Expected patches_per_image  : {num_patches}")

    # Check repetition pattern
    rep_ok = True
    for img_id in unique_ids:
        count = sample_ids_list.count(img_id)
        if count != num_patches:
            rep_ok = False
            break

    if rep_ok:
        logger.info("✓ Repetition pattern OK — each sample_id appears exactly num_patches times.")
    else:
        logger.warning("⚠ WARNING: Some sample_ids do NOT appear exactly num_patches times!")

    # Test grouping using your function
    grouped_ids = get_unique_sample_id(sample_ids_list, patches_per_image=num_patches)
    logger.info(f"Sample IDs after grouping   : {grouped_ids[:10]}")
    logger.info(f"Grouped count (num images)  : {len(grouped_ids)}")

    # Validate shapes of images/tabular/targets
    logger.info(f"images.shape                : {images.shape}")
    logger.info(f"tabular.shape               : {tabular.shape}")
    if targets is not None:
        logger.info(f"targets.shape               : {targets.shape}")

    # Check if reshaping will be valid
    if num_unique > 0 and (total_patches % num_unique == 0):
        logger.info("✓ Patch counts allow clean reshape to (num_images, num_patches)")
    else:
        logger.warning("⚠ Patch count NOT divisible by number of unique IDs — fallback grouping required!")

    logger.info("="*80)
    logger.info("Diagnostic complete.\n")


def cleanup_memory(model=None):
    """Aggressive GPU memory cleanup"""
    
    if model is not None:
        try:
            model = model.cpu()  
            logger.debug(f"✓ Model moved to CPU")
        except Exception as e:
            logger.warning(f"⚠ Error moving model to CPU: {e}")
        del model
    # Clear caches
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    # Log freed memory
    if torch.cuda.is_available():
        try:
            free_gb, total_gb = torch.cuda.mem_get_info()
            free_gb = free_gb / 1e9
            total_gb = total_gb / 1e9
            logger.info(f"✓ GPU Memory: {free_gb:.2f}GB / {total_gb:.2f}GB free")
        except:
            pass
            
def get_default_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def create_weights_zip_for_download_flexible(models_dir, output_dir, zip_name='trained_weights.zip'):
    """
    Create zip file with ONLY AVAILABLE weights.
    Skip missing files - you can add them later by re-running this function.
    Args:
        models_dir (Path): Directory containing trained model files
        output_dir (Path): Directory where zip file will be saved
        zip_name (str): Name of the zip file
    Returns:
        Path: Path to created zip file, or None if no files found
    """
    logger.info(f"\n{'='*80}")
    logger.info("CREATING WEIGHTS ZIP FILE FOR DOWNLOAD")
    logger.info(f"{'='*80}")
    
    models_dir = Path(models_dir)
    output_dir = Path('/kaggle/working')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    zip_path = output_dir / zip_name
    
    # Model files to include 
    model_files = [
        ('vit_best.pth', 'Vision Transformer (ViT-Base)'),
        ('resnet_best.pth', 'ResNet-50'),
        ('densenet_best.pth', 'DenseNet-121'),
        ('xgboost_best.pkl', 'XGBoost'),
        ('species_best.pth', 'Species Model'),
        ('metalearner_best.pth', 'Meta-Learner')
    ]
    logger.info(f"\nScanning for model files in: {models_dir}")
    
    # Find available files
    available_files = []
    missing_files = []
    total_available_size = 0
    
    for file_name, description in model_files:
        file_path = models_dir / file_name
        if file_path.exists():
            file_size = file_path.stat().st_size / (1024**2)
            logger.info(f"✓ Found: {file_name} ({file_size:.2f} MB) - {description}")
            available_files.append((file_name, file_path, description))
            total_available_size += file_size
        else:
            logger.warning(f"⚠ Missing: {file_name} - {description}")
            missing_files.append(file_name)

    # Check if any files available   
    if not available_files:
        logger.error("✗ No model files found!")
        logger.error(f"  Searched in: {models_dir}")
        logger.error("  Please save model files to this directory first.")
        return None
    
    logger.info(f"\n{'='*80}")
    logger.info(f"AVAILABLE: {len(available_files)}/{len(model_files)} models ({total_available_size:.2f} MB)")
    logger.info(f"{'='*80}")
    
    if missing_files:
        logger.warning(f"\nMissing {len(missing_files)} file(s) (will add when ready):")
        for file_name in missing_files:
            logger.warning(f"  - {file_name}")
    
    # Create zip with available files
    logger.info(f"\nCreating zip file: {zip_path}")
    
    try:
        # ✓ If zip already exists, remove it 
        if zip_path.exists():
            logger.info(f"  Removing old zip file: {zip_path}")
            zip_path.unlink()
        
        # Create zip with available files
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_name, file_path, description in available_files:
                zipf.write(file_path, arcname=file_name)
                logger.info(f"  ✓ Added {file_name}")
        
        # Verify zip creation
        zip_size = zip_path.stat().st_size / (1024**2)
        logger.info(f"\n{'='*80}")
        logger.info("✓ ZIP FILE CREATED SUCCESSFULLY!")
        logger.info(f"{'='*80}")
        logger.info(f"\nZip file: {zip_path}")
        logger.info(f"Size: {zip_size:.2f} MB")
        logger.info(f"Contains: {len(available_files)} model(s)")
        
        # List zip contents
        logger.info(f"\nZip contents:")
        with zipfile.ZipFile(zip_path, 'r') as zipf:
            for info in zipf.filelist:
                size_mb = info.file_size / (1024**2)
                logger.info(f"  - {info.filename} ({size_mb:.2f} MB)")
        
        # Show next steps
        if missing_files:
            logger.warning(f"\n{'='*80}")
            logger.warning("⚠ NEXT STEPS:")
            logger.warning(f"{'='*80}")
            logger.warning(f"\nStill missing {len(missing_files)} file(s):")
            for file_name in missing_files:
                logger.warning(f"  - {file_name}")
            logger.warning(f"\nOnce you have these files, re-run this function to update the zip!")
            logger.warning(f"The zip will be automatically recreated with all available files.")
        
        return zip_path
    
    except Exception as e:
        logger.error(f"✗ Error creating zip file: {e}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        raise


def copy_weights_to_output_for_download(models_dir, output_dir):
    """
    Copy all AVAILABLE weights to /kaggle/output directory for easy download.
    Skips missing files.
    """
    logger.info(f"\n{'='*80}")
    logger.info("COPYING AVAILABLE WEIGHTS TO OUTPUT DIRECTORY")
    logger.info(f"{'='*80}")
    
    models_dir = Path(models_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    model_files = [
        'vit_best.pth',
        'resnet_best.pth',
        'densenet_best.pth',
        'xgboost_best.pkl',
        'metalearner_best.pth',
        'species_best.pth'
    ]
    
    logger.info(f"Copying weights to: {output_dir}")
    
    copied_count = 0
    missing_count = 0
    
    for file_name in model_files:
        src = models_dir / file_name
        dst = output_dir / file_name
        
        if src.exists():
            shutil.copy(src, dst)
            file_size = dst.stat().st_size / (1024**2)
            logger.info(f"✓ Copied {file_name} ({file_size:.2f} MB)")
            copied_count += 1
        else:
            logger.warning(f"⚠ Skipping (not found): {file_name}")
            missing_count += 1
    
    logger.info(f"\n✓ Copied {copied_count} file(s) to /kaggle/output/")
    if missing_count > 0:
        logger.warning(f"⚠ Skipped {missing_count} missing file(s)")
    logger.info("  You can download them from Kaggle UI")
    
    return output_dir, copied_count, missing_count

def create_weights_manifest(models_dir, output_dir):
    """
    Create a JSON manifest describing available weights.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    manifest = {
        'created': str(datetime.now()),
        'models': {},
        'total_size_mb': 0,
        'status': 'partial'
    }
    
    model_files = [
        ('vit_best.pth', 'Vision Transformer (ViT-Base) - ImageNet pretrained'),
        ('resnet_best.pth', 'ResNet-50 - ImageNet pretrained'),
        ('densenet_best.pth', 'DenseNet-121 - ImageNet pretrained'),
        ('xgboost_best.pkl', 'XGBoost - Trained on NDVI + height (5 models)'),
        ('metalearner_best.pth', 'Meta-Learner - Trained on base predictions'),
        ('species_best.pth', 'Species Prediction Model')
        
    ]
    
    models_dir = Path(models_dir)
    
    available_count = 0
    missing_count = 0
    
    for file_name, description in model_files:
        file_path = models_dir / file_name
        if file_path.exists():
            size_mb = file_path.stat().st_size / (1024**2)
            manifest['models'][file_name] = {
                'description': description,
                'size_mb': round(size_mb, 2),
                'format': 'PyTorch (.pth)' if file_name.endswith('.pth') else 'Pickle (.pkl)',
                'status': 'available'
            }
            manifest['total_size_mb'] += size_mb
            available_count += 1
        else:
            manifest['models'][file_name] = {
                'description': description,
                'status': 'missing (will be added later)'
            }
            missing_count += 1
    
    manifest['total_size_mb'] = round(manifest['total_size_mb'], 2)
    manifest['summary'] = f"{available_count}/{len(model_files)} models available"
    
    # Update status
    if missing_count == 0:
        manifest['status'] = 'complete'
    elif available_count > 0:
        manifest['status'] = 'partial'
    else:
        manifest['status'] = 'empty'
    
    # Save manifest
    manifest_path = output_dir / 'manifest.json'
    with open(manifest_path, 'w') as f:
        json.dump(manifest, f, indent=2)
    
    logger.info(f"✓ Manifest created: {manifest_path}")
    return manifest

def prepare_and_download_weights(models_dir=None, output_dir=None):
    """
    Complete workflow to prepare weights for download.
    
    Handles:
    - Missing files (skips them)
    - Partial zips (creates with available files)
    - Future additions (re-run to update zip)
    
    Args:
        models_dir (Path): Directory with trained weights
        output_dir (Path): Directory for output files
    """
    if models_dir is None:
        models_dir = Path('/kaggle/working/models')
    if output_dir is None:
        output_dir = Path('/kaggle/output')
    
    logger.info("\n" + "="*80)
    logger.info("PREPARING WEIGHTS FOR DOWNLOAD (FLEXIBLE MODE)")
    logger.info("="*80)
    logger.info("\nThis will:")
    logger.info("  ✓ Create zip with AVAILABLE files")
    logger.info("  ✓ Skip missing files")
    logger.info("  ✓ Can be re-run to add new files")
    
    # Step 1: Create manifest
    logger.info("\n[Step 1/3] Creating manifest...")
    manifest = create_weights_manifest(models_dir, output_dir)
    
    # Step 2: Copy to output
    logger.info("\n[Step 2/3] Copying weights to output...")
    output_dir_result, copied, missing = copy_weights_to_output_for_download(models_dir, output_dir)
    
    # Step 3: Create zip file
    logger.info("\n[Step 3/3] Creating zip file...")
    zip_path = create_weights_zip_for_download_flexible(models_dir, output_dir)
    
    # Final summary
    logger.info("\n" + "="*80)
    logger.info("✓ DOWNLOAD PREPARATION COMPLETE!")
    logger.info("="*80)
    logger.info(f"\nFiles ready in {output_dir}:")
    logger.info(f"  - manifest.json (metadata)")
    
    if copied > 0:
        logger.info(f"  - {copied} model file(s) ready for download")
    
    if zip_path:
        logger.info(f"  - trained_weights.zip ({copied} models)")
    
    if missing > 0:
        logger.warning(f"\n⚠ Note: {missing} model(s) still missing")
    create_file()
    
    return zip_path
    
def offload_model_to_disk(model, model_name, save_dir):
    """
    Save model to disk and move to CPU to free GPU memory.
    Args:
        model: PyTorch model on GPU
        model_name: Name for checkpoint (e.g., 'vit', 'resnet')
        save_dir: Directory to save checkpoint
    
    Returns:
        Path to saved checkpoint
    """
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Move model to CPU
    model = model.cpu()
    
    # Save checkpoint
    checkpoint_path = save_dir / f'{model_name}_best.pth'
    torch.save(model.state_dict(), checkpoint_path)
    
    file_size_mb = checkpoint_path.stat().st_size / (1024**2)
    logger.info(f"✓ Saved {model_name} to disk ({file_size_mb:.2f} MB)")
    
    # Clean up
    del model
    cleanup_memory()
    
    return checkpoint_path

# INTERNET CONNECTIVITY CHECK
def check_internet_connection(timeout=5):
    """
    Check if internet connection is available.
    
    Args:
        timeout (int): Timeout in seconds
    
    Returns:
        bool: True if internet available, False otherwise
    """
    import socket
    
    try:
        # Try to connect to Google DNS 
        socket.create_connection(("8.8.8.8", 53), timeout=timeout)
        logger.info("✓ Internet connection detected")
        return True
    except (socket.timeout, socket.error):
        logger.warning("⚠ No internet connection detected")
        return False

def verify_pretrained_weights_exist(weights_dir):
    """
    Verify all required pretrained weights are cached locally.
    
    Args:
        weights_dir (Path): Directory with cached weights
    
    Returns:
        bool: True if all weights exist, False otherwise
    """
    weights_dir = Path(weights_dir)
    required_weights = {
        'vit': weights_dir / 'vit_best.pth',
        'resnet': weights_dir / 'resnet_best.pth',
        'densenet': weights_dir / 'densenet_best.pth', 
        'species': weights_dir / 'species_best.pth'
    }
    
    logger.info("\nVerifying cached pretrained weights...")
    all_exist = True
    
    for model_name, weight_path in required_weights.items():
        if weight_path.exists():
            size_mb = weight_path.stat().st_size / (1024**2)
            logger.info(f"  ✓ {model_name}: {size_mb:.2f} MB")
        else:
            logger.warning(f"  ✗ {model_name}: NOT FOUND")
            all_exist = False
    
    return all_exist

def load_from_offline_dataset(offline_path, model_name):
    """
    Load pretrained/fine-tuned weights from offline Kaggle dataset.
    
    UPDATED: Now includes species model support!
    
    Checks for BOTH _best.pth (fine-tuned) AND _pretrained.pth (ImageNet)
    
    Priority:
    1. _best.pth (fine-tuned models from previous run)
    2. _pretrained.pth (ImageNet pretrained)
    3. Return None if neither found
    
    Args:
        offline_path (Path): Path to offline dataset
        model_name (str): Model identifier ('vit', 'resnet', 'densenet', 'species', 'xgboost', 'metalearner')
    
    Returns:
        Path: Path to weights file if exists, None otherwise
    """
    offline_path = Path(offline_path)
    
    # Priority 1: Try _best.pth (fine-tuned models) 
    weight_file_best = offline_path / f'{model_name}_best.pth'
    if weight_file_best.exists():
        size_mb = weight_file_best.stat().st_size / (1024**2)
        logger.info(f"✓ Found fine-tuned weights: {weight_file_best.name} ({size_mb:.2f} MB)")
        return weight_file_best
    
    # Priority 2: Try _pretrained.pth (ImageNet pretrained) 
    weight_file_pretrained = offline_path / f'{model_name}_pretrained.pth'
    if weight_file_pretrained.exists():
        size_mb = weight_file_pretrained.stat().st_size / (1024**2)
        logger.info(f"✓ Found pretrained weights: {weight_file_pretrained.name} ({size_mb:.2f} MB)")
        return weight_file_pretrained
    
    logger.warning(f"⚠ Offline weights not found for {model_name}")
    return None


def prepare_pretrained_weights_with_internet_check(weights_cache_dir, offline_dataset_path=None):
    """
    Prepare pretrained weights with internet check and offline fallback.

    Priority:
    1. Check cache 
    2. Download if internet available
    3. Load from offline dataset if no internet
    4. Train from scratch if all else fails
    
    Args:
        weights_cache_dir (Path): Directory to cache weights
        offline_dataset_path (Path): Path to /kaggle/input/csiro-biomass-offline-assets
    
    Returns:
        bool: True if pretrained available, False otherwise
    """
    logger.info("="*80)
    logger.info("STEP 0: PREPARING PRETRAINED WEIGHTS")
    logger.info("="*80)
    
    weights_cache_dir = Path(weights_cache_dir)
    
    # STEP 0a: CHECK IF ALL WEIGHTS ALREADY CACHED  
    logger.info("\nStep 0a: Checking for cached pretrained weights...")
    if verify_pretrained_weights_exist(weights_cache_dir):
        logger.info("\n✓ All pretrained weights found in cache!")
        logger.info("  Using cached weights (fastest)")
        cleanup_memory()
        return True
    
    # STEP 0b: CHECK INTERNET CONNECTION   
    logger.info("\nStep 0b: Checking internet connection...")
    has_internet = check_internet_connection()
    
    if has_internet:
        logger.info("\n✓ Internet available - downloading pretrained weights")
        logger.info("(This happens only once - weights are cached for future runs)\n")
        try:
            # Download and cache all pretrained weights
            vit_pretrained_path = download_and_cache_pretrained_weights('vit', weights_cache_dir)
            resnet_pretrained_path = download_and_cache_pretrained_weights('resnet', weights_cache_dir)
            densenet_pretrained_path = download_and_cache_pretrained_weights('densenet', weights_cache_dir)
            species_pretrained_path = download_and_cache_pretrained_weights('species', weights_cache_dir) 
            
            cleanup_memory()
            logger.info("\n✓ All pretrained weights downloaded and cached successfully!\n")
            logger.info("Attempting to load from offline Kaggle dataset...\n")
            
            if offline_dataset_path is None:
                offline_dataset_path = Path('/kaggle/input/csiro-biomass-offline-assets')
            
            offline_dataset_path = Path(offline_dataset_path)
            
            # Check if offline dataset exists
            if not offline_dataset_path.exists():
                logger.warning(f"⚠ Offline dataset not found: {offline_dataset_path}")
                logger.warning("\nNo pretrained weights available!")
                logger.warning("Options:")
                logger.warning("  1. Upload to Kaggle: /kaggle/input/csiro-biomass-offline-assets/")
            
            # Try to load from offline dataset
            logger.info(f"Checking offline dataset: {offline_dataset_path}\n")
            
            #  Load species model too!
            vit_offline = load_from_offline_dataset(offline_dataset_path, 'vit')
            resnet_offline = load_from_offline_dataset(offline_dataset_path, 'resnet')
            densenet_offline = load_from_offline_dataset(offline_dataset_path, 'densenet')
            species_offline = load_from_offline_dataset(offline_dataset_path, 'species')
            xgb_offline = load_from_offline_dataset(offline_dataset_path, 'xgboost')
            metalearner_offline = load_from_offline_dataset(offline_dataset_path, 'metalearner')
            
            # If found, copy to cache
            if vit_offline and resnet_offline and densenet_offline:
                logger.info("\n✓ Core weights found in offline dataset!")
                
                #  Check species model too
                if species_offline:
                    logger.info("✓ Species model weights found!")
                else:
                    logger.warning("⚠ Species model weights not found (will train)")
    
                if xgb_offline:
                    logger.info("✓ xgb_offline model weights found!")
                else:
                    logger.warning("⚠ xgb_offline model weights not found (will train)")
                
                logger.info("Copying to cache directory for faster access...\n")
                
                try:
                    weights_cache_dir.mkdir(parents=True, exist_ok=True)
                    shutil.copy(vit_offline, weights_cache_dir / 'vit_best.pth')
                    shutil.copy(resnet_offline, weights_cache_dir / 'resnet_best.pth')
                    shutil.copy(densenet_offline, weights_cache_dir / 'densenet_best.pth')
                    #  Copy species model if found
                    if species_offline:
                        shutil.copy(species_offline, weights_cache_dir / 'species_best.pth')
                        logger.info("✓ Species model copied to cache")
                    
                    if xgb_offline:
                        shutil.copy(xgb_offline, weights_cache_dir / 'xgboost_best.pkl')
    
                    if metalearner_offline:
                        shutil.copy(metalearner_offline, weights_cache_dir / 'metalearner_best.pth')
                    
                    logger.info("✓ Weights copied to cache")
                    logger.info("✓ Next run will use cached weights (no copy needed)\n")
                    cleanup_memory()
                except Exception as e:
                    logger.error(f"✗ Error copying weights from offline dataset: {e}")
            return True                    
            
        except Exception as e:
            logger.error(f"\n✗ Error downloading pretrained weights: {e}")
            logger.warning("Falling back to offline dataset...\n")
            # Fall through to offline check
    

    # STEP 0c: NO INTERNET - TRY OFFLINE DATASET   
    if not has_internet:
        logger.warning("\n⚠ No internet connection detected")
        logger.info("Attempting to load from offline Kaggle dataset...\n")
        
        if offline_dataset_path is None:
            offline_dataset_path = Path('/kaggle/input/csiro-biomass-offline-assets')
        
        offline_dataset_path = Path(offline_dataset_path)
        
        # Check if offline dataset exists
        if not offline_dataset_path.exists():
            logger.warning(f"⚠ Offline dataset not found: {offline_dataset_path}")
            logger.warning("\nNo pretrained weights available!")
            logger.warning("Options:")
            logger.warning("  1. Upload to Kaggle: /kaggle/input/csiro-biomass-offline-assets/")
        
        # Try to load from offline dataset
        logger.info(f"Checking offline dataset: {offline_dataset_path}\n")
        
        #  Load species model too!
        vit_offline = load_from_offline_dataset(offline_dataset_path, 'vit')
        resnet_offline = load_from_offline_dataset(offline_dataset_path, 'resnet')
        densenet_offline = load_from_offline_dataset(offline_dataset_path, 'densenet')
        species_offline = load_from_offline_dataset(offline_dataset_path, 'species')
        xgb_offline = load_from_offline_dataset(offline_dataset_path, 'xgboost')
        metalearner_offline = load_from_offline_dataset(offline_dataset_path, 'metalearner')
        
        # If found, copy to cache
        if vit_offline and resnet_offline and densenet_offline:
            logger.info("\n✓ Core weights found in offline dataset!")
            
            #  Check species model too
            if species_offline:
                logger.info("✓ Species model weights found!")
            else:
                logger.warning("⚠ Species model weights not found (will train)")

            if xgb_offline:
                logger.info("✓ xgb_offline model weights found!")
            else:
                logger.warning("⚠ xgb_offline model weights not found (will train)")
            
            logger.info("Copying to cache directory for faster access...\n")

            
            try:
                weights_cache_dir.mkdir(parents=True, exist_ok=True)
                shutil.copy(vit_offline, weights_cache_dir / 'vit_best.pth')
                shutil.copy(resnet_offline, weights_cache_dir / 'resnet_best.pth')
                shutil.copy(densenet_offline, weights_cache_dir / 'densenet_best.pth')
                #  Copy species model if found
                if species_offline:
                    shutil.copy(species_offline, weights_cache_dir / 'species_best.pth')
                    logger.info("✓ Species model copied to cache")
                
                if xgb_offline:
                    shutil.copy(xgb_offline, weights_cache_dir / 'xgboost_best.pkl')

                if metalearner_offline:
                    shutil.copy(metalearner_offline, weights_cache_dir / 'metalearner_best.pth')
                
                logger.info("✓ Weights copied to cache")
                logger.info("✓ Next run will use cached weights (no copy needed)\n")
                cleanup_memory()
                return True
                
            except Exception as e:
                logger.error(f"✗ Error copying weights: {e}")
                return False
        else:
            logger.warning("✗ Some weights missing from offline dataset")
            missing = []
            if not vit_offline: missing.append("ViT")
            if not resnet_offline: missing.append("ResNet")
            if not densenet_offline: missing.append("DenseNet")
            if not xgb_offline: missing.append("xgboost")
            if not metalearner_offline: missing.append("metalearner")
            if not species_offline: 
                logger.warning("  ⚠ Species model not found (optional)")
            
            logger.warning(f"  Missing core models: {', '.join(missing)}\n")
            return False
    
    # FALLBACK: NO PRETRAINED AVAILABLE
    logger.warning("\n" + "!"*80)
    logger.warning("⚠ NO PRETRAINED WEIGHTS AVAILABLE")
    logger.warning("!"*80)
    logger.warning("\nWill train models from SCRATCH")  
    return False


def create_file(name="Ashmin Dhungana"):
    """
    Create a Art describing my Name.
    """
    logger.info("")
    logger.info("╔" + "═" * 78 + "╗")
    logger.info("║" + " " * 78 + "║")
    logger.info("║" + f"Prepared by: {name}".center(78) + "║")
    logger.info("║" + " " * 78 + "║")
    logger.info("╚" + "═" * 78 + "╝")
    logger.info("")

def download_and_cache_pretrained_weights(model_name, weights_dir):
    """
    Download pretrained weights from ImageNet and cache to disk.
    
    Args:
        model_name (str): Model identifier ('vit', 'resnet', 'densenet')
        weights_dir (Path): Directory to cache weights
    
    Returns:
        Path: Path to cached weights file
    """
    weights_dir = Path(weights_dir)
    weights_dir.mkdir(parents=True, exist_ok=True)
    
    cache_file = weights_dir / f'{model_name}_pretrained.pth'
    
    # Check if already cached
    if cache_file.exists():
        try:
            size_bytes = cache_file.stat().st_size
            size_mb = float(size_bytes) / (1024 * 1024)
            logger.info(f"✓ Using cached {model_name} pretrained weights ({size_mb:.2f} MB)")
        except Exception as e:
            logger.warning(f"⚠ Could not read cache file size: {e}")
            logger.info(f"✓ Using cached {model_name} pretrained weights")
        return cache_file
    
    logger.info(f"\n📥 Downloading {model_name} pretrained weights...")
    
    try:
        # Create temporary model to download weights
        if model_name.lower() == 'vit':
            logger.info("  Loading ViT-Base from ImageNet...")
            model = create_model('vit_base_patch16_224', pretrained=True)
            state_dict = model.state_dict()
            torch.save(state_dict, cache_file)
            logger.info(f"✓ ViT-Base weights downloaded and cached")

        elif model_name.lower() == 'species':
            logger.info("  Loading ViT-Base from ImageNet...")
            model = create_model('vit_base_patch16_224', pretrained=True)
            state_dict = model.state_dict()
            torch.save(state_dict, cache_file)
            logger.info(f"✓ Species ViT-Base weights downloaded and cached")
            
        elif model_name.lower() == 'resnet':
            logger.info("  Loading ResNet-50 from ImageNet...")
            model = resnet50(pretrained=True)
            state_dict = model.state_dict()
            torch.save(state_dict, cache_file)
            logger.info(f"✓ ResNet-50 weights downloaded and cached")
            
        elif model_name.lower() == 'densenet':
            logger.info("  Loading DenseNet-121 from ImageNet...")
            model = densenet121(pretrained=True)
            state_dict = model.state_dict()
            torch.save(state_dict, cache_file)
            logger.info(f"✓ DenseNet-121 weights downloaded and cached")
        del model
        del state_dict
        cleanup_memory()

        try:
            size_bytes = cache_file.stat().st_size
            size_mb = float(size_bytes) / (1024 * 1024)
            logger.info(f"  Size: {size_mb:.2f} MB")
        except Exception as e:
            logger.warning(f"⚠ Could not calculate file size: {e}")
        
        return cache_file
        
    except Exception as e:
        logger.error(f"✗ Error downloading {model_name} weights: {e}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        raise

def load_from_offline_dataset(offline_path, model_name):
    """
    Load pretrained/fine-tuned weights from offline Kaggle dataset.
    
    Checks for BOTH _best.pth (fine-tuned) AND _pretrained.pth (ImageNet)
    
    Priority:
    1. _best.pth (fine-tuned models from previous run)
    2. _pretrained.pth (ImageNet pretrained)
    3. Return None if neither found
    
    Args:
        offline_path (Path): Path to offline dataset
        model_name (str): Model identifier ('vit', 'resnet', 'densenet')
    
    Returns:
        Path: Path to weights file if exists, None otherwise
    """
    offline_path = Path(offline_path)

    # Priority 0: Try _best.pkl 
    weight_file_best = offline_path / f'{model_name}_best.pkl'
    if weight_file_best.exists():
        size_mb = weight_file_best.stat().st_size / (1024**2)
        logger.info(f"✓ Found fine-tuned weights: {weight_file_best.name} ({size_mb:.2f} MB)")
        return weight_file_best
    
    # Priority 1: Try _best.pth (fine-tuned models) 
    weight_file_best = offline_path / f'{model_name}_best.pth'
    if weight_file_best.exists():
        size_mb = weight_file_best.stat().st_size / (1024**2)
        logger.info(f"✓ Found fine-tuned weights: {weight_file_best.name} ({size_mb:.2f} MB)")
        return weight_file_best
    
    # Priority 2: Try _pretrained.pth (ImageNet pretrained) 
    weight_file_pretrained = offline_path / f'{model_name}_pretrained.pth'
    if weight_file_pretrained.exists():
        size_mb = weight_file_pretrained.stat().st_size / (1024**2)
        logger.info(f"✓ Found pretrained weights: {weight_file_pretrained.name} ({size_mb:.2f} MB)")
        return weight_file_pretrained
    
    logger.warning(f"⚠ Offline weights not found for {model_name}")
    return None


def load_model_with_pretrained(model_class, model_name, weights_dir, device, use_mil=CONFIG["use_mil"]):
    """
    SMART loading with PRIORITY:
    1. Check _best.pth FIRST (resume/fine-tune from saved weights)
    2. Check _pretrained.pth (first run with ImageNet)
    3. Train from scratch
    """
    logger.info(f"\nLoading {model_name}...")
    if use_mil == True and model_name in ['vit', 'densenet', 'resnet', 'species']:
        model = model_class(pretrained=False, use_mil=CONFIG["use_mil"])
    else:
        model = model_class(pretrained=False)
    best_path = Path(weights_dir) / f'{model_name}_best.pth'
    pretrained_path = Path(weights_dir) / f'{model_name}_pretrained.pth'
    
    # PRIORITY 1: _best.pth from saved dataset
    if best_path.exists():
        logger.info(f"✓ Found: {best_path.name} (FROM SAVED DATASET)")
        logger.info(f"  [Mode: FULL-LOAD]")
        logger.info(f"  Both backbone AND regression head already trained!")
        state = torch.load(best_path, map_location='cpu')
        model.load_state_dict(state, strict=False)
        logger.info(f"✓ Loaded successfully - ready for fine-tuning!\n")
    
    # PRIORITY 2: _pretrained.pth from download
    elif pretrained_path.exists():
        logger.info(f"✓ Found: {pretrained_path.name} (FROM IMAGENET)")
        logger.info(f"  [Mode: BACKBONE-ONLY]")
        logger.info(f"  Only backbone loaded, regression head random!")
        logger.info(f"  Loading for: First-time training with new architecture")
        state = torch.load(pretrained_path, map_location='cpu')
        incompatible = model.load_state_dict(state, strict=False)
        if incompatible.missing_keys:
            logger.info(f"✓ Backbone loaded")
            logger.info(f"  Skipped: {len(incompatible.missing_keys)} regression head layers")
    
    # PRIORITY 3: Train from scratch
    else:
        logger.warning(f"⚠ No weights found - training from scratch\n")
    model = model.to(device)
    return model

weights_cache_dir = MODELS_DIR 
offline_dataset_path = ASSETS
    
has_pretrained = prepare_pretrained_weights_with_internet_check(
        weights_cache_dir,
        offline_dataset_path)


# STEP 0: Load data
(train_df_org, test_df_org, STATE_COLS_org, SPECIES_COLS_org, TARGET_TYPE_COLS_org, 
    TOTAL_TABULAR_DIM_org, lookups_dict_org, original_test_df_org) = load_and_explore_data()

# Check for pretrained species weights first
if has_pretrained:
    logger.info("  ✓ Attempting to load pretrained species weights")
    species_model = load_model_with_pretrained(
    SpeciesClassificationViT, 'species', weights_cache_dir, device, use_mil=CONFIG["use_mil"]
    )
else:
    logger.info("  Training from scratch (no pretrained weights)")
    species_model = SpeciesClassificationViT(pretrained=False, use_mil=CONFIG["use_mil"])
    species_model = species_model.to(device)

# STEP 2.0: For Training Specis Model
train_loader_old, val_loader_old, test_loader_old, train_images_old, val_images_old = \
    create_data_loaders(train_df_org, test_df_org, lookups_dict_org, TOTAL_TABULAR_DIM_org, model=species_model)

agg = compute_and_save_ndvi_stats_with_sklearn(
    train_df_org,
    base_dir= RAW_DATA_DIR,  
    image_path_col="image_path",
    species_col="Species",
    ndvi_col_candidates=('Pre_GSHH_NDVI','ndvi','NDVI','vndvi'), 
    out_path= OUTPUTS_DIR / 'ndvidata.csv',
    resize_to=(512,512),
    max_images=None,        
    min_samples_for_ml=20,   
    force_recompute=True
)


# Train the species model
species_history = train_species_model(
        train_loader_old, val_loader_old,
        train_df=train_df_org, 
        num_epochs=CONFIG.get('species_vit_epochs', 50),
        device=device,
        model= species_model
    )

logger.info("\nOffloading Species Model to disk...")
offload_model_to_disk(species_model, 'species', MODELS_DIR)
logger.info("✓ Species model training completed\n")  

species_model = SpeciesClassificationViT(pretrained=False, use_mil=CONFIG["use_mil"])
species_model.load_state_dict(torch.load(MODELS_DIR / 'species_best.pth', map_location=device))
species_model = species_model.to(device)
logger.info("✓ ViT loaded")


synthetic_df = generate_synthetic_train_df_matching_schema(
    train_df=train_df_org,
    species_model=species_model,
    lookups_dict=lookups_dict_org,
    raw_dir=RAW_DATA_DIR,
    device=device
)

try:
    (train_df, test_df, STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS, 
        TOTAL_TABULAR_DIM, lookups_dict, original_test_df) = load_and_explore_data(TRAINDATA=SYN_CSV)

except Exception as e:
    (train_df, test_df, STATE_COLS, SPECIES_COLS, TARGET_TYPE_COLS, 
        TOTAL_TABULAR_DIM, lookups_dict, original_test_df) = load_and_explore_data()
    logger.info(f"Synthetic_df Load Failed: {e}")
                   

# STEP 1.2: Create loaders synthetic
train_loader, val_loader, test_loader, train_images, val_images = \
    create_data_loaders(train_df, test_df, lookups_dict, TOTAL_TABULAR_DIM, model=species_model)



train_split_df, val_split_df = train_test_split(
        train_df,
        test_size=0.15,  
        random_state=42,
        stratify=train_df['target_name'] 
        )


#_____________________________________________________
# Comment Out Later During Submition
#______________________________________________________
#Dataloder Diagnostics
run_dataloader_diagnostics(train_loader, num_patches=8)
run_dataloader_diagnostics(test_loader, num_patches=8)
diagnose_ensemble_batch(test_loader, num_patches=8)
diagnose_ensemble_batch(train_loader, num_patches=8)
#_____________________________________________________

logger.info("✓ Helper Function Defined")